# Modular Financial RAG System

NEW USER (no database):
Section 1 (Setup) → Section 3-4 (Process docs) → Section 5+ (RAG)

RETURNING USER (existing database):
Section 1 (Setup) → Section 5+ (RAG)

## 1. Core Setup

#### 1.1. Install Required Libraries

In [1]:
# Install neccessary libraries
!pip install --quiet sentence-transformers transformers chromadb pymupdf rank-bm25 python-docx ipywidgets spacy

#### 1.2 Enable jupyter extensions

In [2]:
# Enable widgets
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


#### 1.3 Import Libraries

In [3]:
# Essential imports
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from pathlib import Path
import os, json, re, threading
from typing import Dict, List, Optional, Any, Tuple
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import requests
import docx

from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder
from transformers import pipeline
import chromadb
from chromadb.config import Settings
import fitz  # PyMuPDF

from rank_bm25 import BM25Okapi
import pickle
import re
from IPython.display import display, clear_output
import ipywidgets as widgets
import datetime
import spacy
from collections import defaultdict
import shutil

#### Section 1.4. Local Config

This section establishes the core directory structure and configuration settings for the RAG system. It defines the foundational paths and validates the environment setup.

Run this cell once. Make sure the path to the desired data directory is correct and uncommented. This will be the dataset processed in the following sections. If another dataset have already been processed and stored, then comment out the path to those files. This allows to create different collections in the vector database.

**Directory Configuration:**
- Sets `ROOT` as the current working directory using `Path.cwd()`
- Defines three critical directories: `DATA_DIR` (source documents), `STORE_DIR` (ChromaDB storage), and `MODELS_DIR` (model cache)
- Creates `file_directories` list specifying which subdirectories to scan for documents

**Environment Validation:**
- Performs existence checks on all defined paths
- Scans and reports ChromaDB files if the store directory exists
- Provides diagnostic output showing directory status and file counts

**System Configuration:**
- Establishes `CONFIG` dictionary with key parameters:
  - `embedding_model`: BGE large model for semantic search
  - `finbert_model`: Financial sentiment analysis model
  - `llm_model`: Ollama model for response generation
  - `chunk_size` and `overlap`: Text processing parameters

In [4]:
# Core configuration to work from the current directory
ROOT = Path.cwd() 
DATA_DIR = ROOT / 'data'
STORE_DIR = ROOT / 'chroma_store'
MODELS_DIR = ROOT / 'models'

# Verify paths exist
print("Path verification:")
print(f"   ROOT: {ROOT}")
print(f"   DATA_DIR exists: {DATA_DIR.exists()}")
print(f"   STORE_DIR exists: {STORE_DIR.exists()}")
print(f"   MODELS_DIR exists: {MODELS_DIR.exists()}")

if STORE_DIR.exists():
    chroma_files = list(STORE_DIR.glob('*'))
    print(f"   ChromaDB files found: {len(chroma_files)}")
    for file in chroma_files[:5]:  # Show first 5 files
        print(f"      {file.name}")

file_directories = [
    DATA_DIR / 'pra_rulebook',
    # Add back other directories when needed:
    # DATA_DIR / 'earnings_transcripts',
    # DATA_DIR / 'transcripts_analyst_meeting',
]

CONFIG = {
    'embedding_model': 'BAAI/bge-large-en-v1.5',
    'finbert_model': 'yiyanghkust/finbert-tone',
    'llm_model': 'llama3.1:8b',
    'chunk_size': 600,
    'overlap': 100
}

print(f"Config: {len(CONFIG)} settings loaded")

Path verification:
   ROOT: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe
   DATA_DIR exists: True
   STORE_DIR exists: True
   MODELS_DIR exists: True
   ChromaDB files found: 4
      3f516e47-f1a9-48bb-adb7-79543f55aa64
      bm25_indices
      chroma.sqlite3
      c82d869e-4e75-4e96-af7b-b45927f70131
Config: 5 settings loaded


#### Section 1.5 Model Loading

This section initializes and loads the core machine learning models required for the RAG system's text processing and similarity operations.

**Model Loading Functions:**
- `load_embedding_model()`: Instantiates a SentenceTransformer model using the configured BGE large embedding model for semantic text representation
- `load_finbert_model()`: Creates a Transformers pipeline for financial sentiment analysis using the FinBERT model, configured to return all confidence scores

**Model Initialization:**
- Loads the BGE large English model (`BAAI/bge-large-en-v1.5`) for generating high-quality text embeddings
- Initializes FinBERT (`yiyanghkust/finbert-tone`) as a sentiment analysis pipeline specifically trained on financial text
- Both models are loaded into global variables (`embedding_model` and `finbert`) for system-wide access

**Error Handling:**
- Implements basic error handling during model loading
- Provides console feedback confirming successful model initialization

**Purpose:**
This section establishes the foundational AI models that enable the RAG system's core capabilities: semantic similarity search through embeddings and financial sentiment analysis. The embedding model is crucial for vector database operations and semantic search, while FinBERT provides domain-specific sentiment insights for financial documents. These models are loaded once at system startup and reused throughout the application lifecycle for optimal performance.

In [5]:
# Load embedding and sentiment models
def load_embedding_model(model_name: str) -> SentenceTransformer:
    """Load BGE embedding model locally"""
    return SentenceTransformer(model_name)

def load_finbert_model(model_name: str) -> pipeline:
    """Load FinBERT sentiment model locally"""
    return pipeline('sentiment-analysis', model=model_name, return_all_scores=True)

# Load models
embedding_model = load_embedding_model(CONFIG['embedding_model'])
finbert = load_finbert_model(CONFIG['finbert_model'])

print(f"Models loaded: BGE + FinBERT")

Device set to use mps:0


Models loaded: BGE + FinBERT


#### Section 1.6 — ChromaDB setup with auto-repair

Initialize a persistent ChromaDB client, ensure the store directory exists, handle common DB errors.

- Key functions
  - create_chroma_client(persist_dir: str) -> chromadb.Client
    - Ensures the persistence directory exists.
    - Attempts to instantiate a chromadb. PersistentClient and verify responsiveness via client.list_collections().
    - On failure, logs the error, removes the existing store directory (shutil.rmtree), recreates it, re-instantiates the PersistentClient, and reports repair success. Returns the client instance.

  - get_or_create_collection(client: chromadb.Client, name: str) -> chromadb.Collection
    - Tries to retrieve an existing collection by name.
    - If retrieval fails, creates a new collection with basic metadata (description) and returns it.
    - Encapsulates collection creation/retrieval with simple error handling.

  - display_database_status(client: chromadb.Client, store_dir: Path) -> None
    - Prints a structured status report: ChromaDB version, database path, total collections.
    - Iterates collections to query per-collection counts (collection.count()), aggregates total chunk/document counts, and prints per-collection diagnostics.
    - Catches and reports exceptions per collection and for the overall status check.

- Initialization flow (cell-level behavior)
  - Calls create_chroma_client with STORE_DIR; if it succeeds, sets a global client.
  - Optionally selects/creates a collection when CONFIG contains a collection_name, storing it in the global variable collection.
  - Calls display_database_status to output diagnostics and guide next steps (e.g., run document processing if no collections).
  - All major failure points print user-facing guidance (permissions, locks, readonly issues) and encourage re-running the repair path or restarting the kernel.

- Error-handling & repair strategy
  - Proactive: tests client responsiveness immediately after creation.
  - Recoverable-failure approach: on error tries a filesystem-level repair (delete+recreate store directory) and reinstantiates the client.
  - Informative: prints specific hints for common failure modes (readonly, permission, lock/busy).

- Side effects & outputs
  - Creates the persistence directory if missing.
  - May remove and recreate the chroma store directory during "repair".
  - Produces a human-readable database status report; returns client and collection objects for downstream cells.

In [6]:
# ChromaDB setup with auto-repair - Clean Implementation
def create_chroma_client(persist_dir: str) -> chromadb.Client:
    """Create persistent ChromaDB client with auto-repair capability."""
    persist_path = Path(persist_dir)
    persist_path.mkdir(parents=True, exist_ok=True)

    try:
        client = chromadb.PersistentClient(path=str(persist_path))
        # Test client responsiveness
        client.list_collections()
        return client
    except Exception as e:
        print(f"ChromaDB issue detected: {e}")
        print(f"Attempting database repair...")
        
        # Simple repair: recreate directory
        if persist_path.exists():
            shutil.rmtree(persist_path, ignore_errors=True)
        persist_path.mkdir(parents=True, exist_ok=True)
        
        client = chromadb.PersistentClient(path=str(persist_path))
        print(f"Database repaired successfully")
        return client

def get_or_create_collection(client: chromadb.Client, name: str) -> chromadb.Collection:
    """Get existing collection or create new one."""
    try:
        return client.get_collection(name)
    except Exception:
        return client.create_collection(
            name=name,
            metadata={"description": "Financial documents collection"}
        )

def display_database_status(client: chromadb.Client, store_dir: Path) -> None:
    """Display comprehensive database status information."""
    print(f"ChromaDB Status Report")
    print(f"{'='*50}")
    
    try:
        # Get collections
        collections = client.list_collections()
        print(f"Database location: {store_dir}")
        print(f"ChromaDB version: {chromadb.__version__}")
        print(f"Total collections: {len(collections)}")
        
        if collections:
            print(f"\nCollection Details:")
            total_documents = 0
            
            for i, col in enumerate(collections, 1):
                try:
                    collection_obj = client.get_collection(col.name)
                    count = collection_obj.count()
                    total_documents += count
                    print(f"   {i}. {col.name}: {count:,} chunks")
                except Exception as e:
                    print(f"   {i}. {col.name}: Error ({str(e)[:30]}...)")
            
            print(f"\nTotal documents: {total_documents:,}")
        else:
            print(f"\nNo collections found")
            print(f"Run document processing (Sections 3-4) to populate database")
            
    except Exception as e:
        print(f"Status check failed: {e}")
        
    print(f"{'='*50}")

# Initialize ChromaDB client
print("Initializing ChromaDB...")
try:
    client = create_chroma_client(str(STORE_DIR))
    print(f"ChromaDB client ready")
    
    # Initialize collection variable for global use
    collection = None
    if CONFIG.get('collection_name'):
        try:
            collection = get_or_create_collection(client, CONFIG['collection_name'])
            print(f"Collection '{CONFIG['collection_name']}' ready")
        except Exception as e:
            print(f"Collection setup warning: {e}")
    
    # Display comprehensive status
    display_database_status(client, STORE_DIR)
    
except Exception as e:
    print(f"ChromaDB initialization failed: {e}")
    print(f"Check your environment and permissions")

Initializing ChromaDB...
ChromaDB client ready
ChromaDB Status Report
Database location: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/chroma_store
ChromaDB version: 1.0.21
Total collections: 2

Collection Details:
   1. transcrips_barclasys: 5,591 chunks
   2. pra_rules: 2,991 chunks

Total documents: 8,582


#### 1.7 Core Function Definitions

##### 1.7.1 Document Discovery Function

Locate supported document files in one or more data directories and return a list of Path objects for downstream processing (extraction, chunking, ingestion).

- Key function
  - discover_documents(data_dirs: List[Path]) -> List[Path]
    - Signature and return: accepts a list of Path objects, returns a list of Path objects for files found.
    - Supported extensions: [.pdf, .docx, .txt, .md, .csv] (defined inline).

- Usage in notebook
  - Called with file_directories to populate variable files and drive subsequent extraction/processing cells.
  - Debug prints guide users when directories are empty/missing or when glob patterns yield unexpected results.

Summary: a single, explicit discovery utility that enumerates top-level files of specific extensions across configured directories with heavy debug output to help diagnose data-path and glob issues prior to processing.

In [7]:
# Document discovery function with enhanced logging
def discover_documents(data_dirs: List[Path]) -> List[Path]:
    """Find all supported files in data directories"""
    supported_extensions = ['.pdf', '.docx', '.txt', '.md', '.csv']
    files = []
    print("Supported file extensions:", supported_extensions)
    print(f"Data directories provided: {data_dirs}")

    for dir_path in data_dirs:
        if dir_path.exists():
            print(f"Searching directory: {dir_path}")
            try:
                print(f"  Contents of {dir_path}: {[item.name for item in dir_path.iterdir()]}")
            except Exception as e:
                print(f"  Could not list contents of {dir_path}: {e}")

            for ext in supported_extensions:
                pattern = f'*{ext}'
                print(f"  Searching for pattern '{pattern}' in {dir_path}")
                found_files = list(dir_path.glob(pattern))
                print(f"  Found {len(found_files)} file(s) matching '{pattern}'")
                if found_files:
                    print(f"  Sample found files: {[f.name for f in found_files[:3]]}")
                files.extend(found_files)
        else:
            print(f"Warning: Directory not found: {dir_path}")
    return files

##### 1.7.2 Metadata Extraction Functions

Extract structured metadata from filenames and document content, merge extraction outputs, score quality, and resolve conflicts between filename-derived and content-derived metadata.

- extract_file_metadata(filepath: Path) -> Dict[str, str]
  - Parses filename stem and extension.
  - Returns core file-level fields: filename, company (first underscore part), quarter (second part or unknown), doc_type (heuristic by extension/name), file_extension.
  - Lightweight, filename-first fallback used when content extraction is unavailable.

In [8]:
# Metadata extraction function 
def extract_file_metadata(filepath: Path) -> Dict[str, str]:
    """Extract company, quarter, year from filename"""
    name = filepath.stem.lower()
    parts = name.split('_')

    # Determine document type based on file extension and content
    doc_type = 'unknown'
    if filepath.suffix.lower() == '.pdf':
        if 'transcript' in name:
            doc_type = 'earnings_transcript'
        elif 'meeting' in name:
            doc_type = 'analyst_meeting'
        else:
            doc_type = 'financial_document'
    elif filepath.suffix.lower() in ['.docx', '.txt', '.md']:
        doc_type = 'text_document'
    elif filepath.suffix.lower() == '.csv':
        doc_type = 'data_file'

    return {
        'filename': filepath.name,
        'company': parts[0] if parts else 'unknown',
        'quarter': parts[1] if len(parts) > 1 else 'unknown',
        'doc_type': doc_type,
        'file_extension': filepath.suffix.lower()
    }

##### 1.7.3 Merge Metadata Function

- merge_metadata(file_meta: Dict, extraction_meta: Optional[Dict]) -> Dict
  - Safely combines file-level metadata with extraction metadata.
  - Preserves core file fields (filename, company, quarter, doc_type, file_extension) unless they are empty/unknown.
  - Merges nested dicts, flattens/adds extraction-specific fields, and returns the merged dictionary.


In [9]:
# Metadata merging function
def merge_metadata(file_meta: Dict[str, Any], extraction_meta: Optional[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Safely merge file-level metadata with extraction metadata.

    Args:
        file_meta: Output from extract_file_metadata() - contains filename, company, quarter, doc_type, file_extension
        extraction_meta: Output from EnhancedDocumentProcessor.extract_text() - contains extraction_mode, layout, etc.

    Returns:
        Merged metadata dictionary preserving file-level info and adding extraction details
    """
    merged = dict(file_meta) if isinstance(file_meta, dict) else {}
    if not extraction_meta:
        return merged

    for k, v in extraction_meta.items():
        if k in merged and isinstance(merged[k], dict) and isinstance(v, dict):
            # Merge nested dictionaries
            merged[k] = {**merged[k], **v}
        elif k in ('filename', 'company', 'quarter', 'doc_type', 'file_extension'):
            # Don't overwrite core file metadata unless it's empty/unknown
            if not merged.get(k) or merged.get(k) in (None, '', 'unknown'):
                merged[k] = v
        else:
            # Add extraction-specific fields (extraction_mode, layout, etc.)
            merged[k] = v

    return merged

##### 1.7.4 Enhanced Document Processor

Robust, format-aware text extraction from PDFs, DOCX, TXT/MD and CSV with optional PDF layout parsing, basic cleaning, extraction metadata, and batch processing with progress reporting and error capture.

- clean_text(text: str) -> str
  - Normalizes input to a minimal unicode string.
  - Handles bytes decoding (utf-8 with fallback), removes NULs, normalizes CRLF/CR to LF, collapses excessive blank lines, and strips surrounding whitespace.
  - Returns empty string for falsy input.

- _extract_pdf_with_layout(file_path: str) -> (str, List[Dict])
  - Detailed PDF extraction using PyMuPDF (fitz) returning a concatenated text and a layout list.
  - Iterates pages → blocks → lines → spans; for each non-empty span collects text and layout attributes (page, bbox, font_size, font, is_bold, is_italic).
  - Returns (full_text, layout_data). Raises RuntimeError on failure.

- _extract_pdf_simple(file_path: str) -> str
  - Fast PDF extraction using page.get_text() for each page, concatenated.
  - Simpler, faster path for non-complex PDFs. Raises RuntimeError on failure.

- extract_text(file_path: str, pdf_mode: str = "auto") -> (str, Optional[Dict])
  - Main per-file extraction orchestrator.
  - Validates file existence and dispatches by suffix:
    - '.pdf': supports pdf_mode "auto" | "layout" | "simple".
      - "auto": heuristics determine layout vs simple (e.g., page_count > 3 → layout).
      - Falls back to simple extraction on exceptions.
      - Metadata includes extraction_mode and 'layout' when applicable.
    - '.docx': reads paragraphs via python-docx; metadata extraction_mode 'docx'.
    - '.txt' / '.md': reads text file; extraction_mode 'text'.
    - '.csv': reads with pandas and converts to CSV text; extraction_mode 'csv'.
    - other suffixes: raises ValueError.
  - Cleans extracted text via clean_text; raises ValueError if resulting text is empty.
  - Returns (cleaned_text, metadata) where metadata is a dict containing extraction_mode and optional layout list for PDFs.

- process_files(file_paths, pdf_mode: str = "auto", show_progress: bool = True) -> List[Tuple[Path, Optional[str], Optional[Dict]]]
  - Batch processor that iterates files, calls extract_text for each, and collects tuples: (Path, text or None, metadata or {'error': ...}).
  - Uses tqdm for progress; counts successes and failures; prints summary when done.
  - Captures exceptions per-file and records error metadata instead of aborting the whole run.

Design notes / error handling
- Defensive: clear error reporting per-file, raises on missing file or when no text extracted.
- PDF extraction has two tiers (layout vs simple) and automatic fallback on extraction errors.
- Metadata is returned alongside text to indicate extraction mode and provide layout spans for downstream use (sanitization needed before DB storage).
- Intended for integration into a larger pipeline (chunking, metadata merger, embeddings) where layout data can improve chunk boundaries and provenance.

In [10]:
# Enhanced document processing class
class EnhancedDocumentProcessor:
    def __init__(self):
        self.supported_formats = ['.pdf', '.docx', '.txt', '.md', '.csv']

    def clean_text(self, text: str) -> str:
        """Return a minimal cleaned unicode string. Returns empty string for falsy input."""
        if not text:
            return ''
        # handle bytes
        if isinstance(text, (bytes, bytearray)):
            try:
                text = text.decode('utf8')
            except Exception:
                text = text.decode('utf8', errors='replace')
        # remove nulls, normalize newlines, collapse long runs
        text = text.replace('\x00', '')
        text = re.sub(r"\r\n|\r", "\n", text)
        text = re.sub(r"\n{3,}", "\n\n", text)
        return text.strip()

    def _extract_pdf_with_layout(self, file_path: str) -> Tuple[str, List[Dict]]:
        """Enhanced PDF extraction with layout metadata (Function A logic)"""
        full_text, layout_data = [], []
        try:
            with fitz.open(file_path) as doc:
                for page_num, page in enumerate(doc, 1):
                    blocks = page.get_text("dict").get("blocks", [])
                    for block in blocks:
                        for line in block.get("lines", []):
                            for span in line.get("spans", []):
                                text = span.get("text", "").strip()
                                if text:
                                    font = span.get("font", "") or ""
                                    layout_data.append({
                                        "text": text,
                                        "page": page_num,
                                        "bbox": span.get("bbox"),
                                        "font_size": span.get("size"),
                                        "font": font,
                                        "is_bold": "bold" in font.lower(),
                                        "is_italic": "italic" in font.lower()
                                    })
                                    full_text.append(text)
            return "\n".join(full_text), layout_data
        except Exception as e:
            raise RuntimeError(f"Layout PDF extraction failed: {e}")

    def _extract_pdf_simple(self, file_path: str) -> str:
        """Simple PDF extraction (Function B logic)"""
        try:
            doc = fitz.open(file_path)
            parts = [page.get_text() for page in doc]
            return '\n'.join(parts)
        except Exception as e:
            raise RuntimeError(f"Simple PDF extraction failed: {e}")

    def extract_text(self, file_path: str, pdf_mode: str = "auto") -> Tuple[str, Optional[Dict]]:
        """
        Extract text from various file formats with smart PDF handling

        Args:
            file_path: Path to the file
            pdf_mode: "simple" (fast), "layout" (detailed), or "auto" (detect complexity)

        Returns:
            Tuple of (extracted_text, metadata) where metadata contains layout info for PDFs
        """
        p = Path(file_path)
        if not p.exists():
            raise ValueError(f'File not found: {file_path}')

        suffix = p.suffix.lower()
        metadata = None

        if suffix == '.pdf':
            # Auto-detect if we need layout extraction
            if pdf_mode == "auto":
                # Simple heuristic: use layout for potentially complex PDFs
                try:
                    with fitz.open(file_path) as doc:
                        # Check if PDF has multiple columns or complex layout
                        if doc.page_count > 3:  # Longer documents might be complex
                            text, layout_data = self._extract_pdf_with_layout(file_path)
                            metadata = {"layout": layout_data, "extraction_mode": "layout_auto"}
                        else:
                            text = self._extract_pdf_simple(file_path)
                            metadata = {"extraction_mode": "simple_auto"}
                except:
                    # Fallback to simple extraction
                    text = self._extract_pdf_simple(file_path)
                    metadata = {"extraction_mode": "simple_fallback"}

            elif pdf_mode == "layout":
                text, layout_data = self._extract_pdf_with_layout(file_path)
                metadata = {"layout": layout_data, "extraction_mode": "layout_forced"}

            else:  # simple
                text = self._extract_pdf_simple(file_path)
                metadata = {"extraction_mode": "simple_forced"}

        elif suffix in ('.docx',):
            doc = docx.Document(str(p))
            parts = [para.text for para in doc.paragraphs]
            text = '\n'.join(parts)
            metadata = {"extraction_mode": "docx"}

        elif suffix in ('.txt', '.md'):
            text = p.read_text(encoding='utf-8', errors='ignore')
            metadata = {"extraction_mode": "text"}

        elif suffix in ('.csv',):
            df = pd.read_csv(p)
            text = df.to_csv(index=False)
            metadata = {"extraction_mode": "csv"}

        else:
            raise ValueError(f'Unhandled file type: {suffix}')

        cleaned = self.clean_text(text)
        if not cleaned:
            raise ValueError(f'No text extracted from: {file_path}')

        return cleaned, metadata

    def process_files(self, file_paths, pdf_mode: str = "auto", show_progress: bool = True) -> List[Tuple[Path, Optional[str], Optional[Dict]]]:
        """Process multiple files, return (Path, text, metadata) tuples"""
        files = list(file_paths)
        results = []
        success = 0

        with tqdm(total=len(files), desc="Processing files", disable=not show_progress) as pbar:
            for fp in files:
                try:
                    text, metadata = self.extract_text(str(fp), pdf_mode=pdf_mode)
                    results.append((Path(fp), text, metadata))
                    success += 1
                except Exception as e:
                    results.append((Path(fp), None, {"error": str(e)}))
                finally:
                    pbar.update(1)
                    pbar.set_postfix(success=success, failed=len(results)-success)

        if show_progress:
            print(f'Processed {len(files)} files ({success} succeeded, {len(files) - success} failed)')

        return results

##### 1.7.5 Chunking Function

Split cleaned document text into overlapping, metadata-rich chunks suitable for embeddings, sentiment analysis and DB storage.

- Input handling: calls a clean_text helper to normalize/strip input; returns [] if input is empty after cleaning.

- Boundary-aware slicing:
  - Primary window: start..end = idx .. min(idx+chunk_size, len(text)).
  - If respect_boundaries is True and end < len(text), searches backward from near-end for preferred break patterns (paragraph breaks, single newlines, sentence/clause separators) to move end to a natural boundary within the overlap window.
  - search window is limited (end - overlap - 50) to prefer nearby boundaries.

- Overlap and stepping:
  - After emitting a chunk, the next start index advances to max(end - overlap, start + 1) so chunks overlap by the configured amount and do not stall on zero-length progress.

- Chunk record schema (each chunk is a dict):
  - chunk_id: f"{chunk_id_prefix}_{i}"
  - text: trimmed chunk text
  - offset: {"start": start, "end": end}
  - size: len(text)
  - boundary_type: "natural" (if ended at a found boundary), "hard" (forced at window end) or "merged" (after small-chunk merge)

- Small-chunk merging:
  - After initial segmentation, adjacent tiny chunks (<100 chars) are merged into the previous chunk when combined size remains reasonable (< chunk_size * 1.5). Merged chunks update offsets, size and boundary_type.

- Design/compatibility notes:
  - Default chunk_size=600 and overlap=100 are tuned for downstream embedding models.
  - chunk_id_prefix allows stable, document-scoped IDs (e.g., using file stem when called from processing loop).
  - Output is a list of chunk dicts ready for sentiment labeling, embedding generation, and sanitized storage.

In [11]:
# Enhanced chunking function
def chunk_text_enhanced(
    text: str, chunk_size: int = 600, overlap: int = 100,
    chunk_id_prefix: str = "chunk", respect_boundaries: bool = True
) -> List[Dict]:
    """Enhanced chunking with boundary respect and metadata - compatible with embeddings and sentiment analysis."""
    if not text:
        return []

    # Clean input text
    text = clean_text(text)
    if not text:
        return []

    chunks, idx, i = [], 0, 0

    while idx < len(text):
        start, end = idx, min(idx + chunk_size, len(text))

        # Find natural boundaries if requested
        if respect_boundaries and end < len(text):
            # Look for paragraph breaks, sentences, clauses
            for pattern in [r'\n\n', r'\n', r'\. ', r'; ', r', ']:
                search_start = max(start, end - overlap - 50)
                matches = list(re.finditer(pattern, text[search_start:end]))
                if matches and (boundary_pos := search_start + matches[-1].end()) > start + chunk_size // 2:
                    end = boundary_pos
                    break

        chunk_text = text[start:end].strip()
        if chunk_text:  # Only add non-empty chunks
            chunks.append({
                "chunk_id": f"{chunk_id_prefix}_{i}",
                "text": chunk_text,
                "offset": {"start": start, "end": end},
                "size": len(chunk_text),
                "boundary_type": "natural" if respect_boundaries and end < len(text) else "hard"
            })
            i += 1

        # Move to next position with overlap
        idx = max(end - overlap, start + 1)

    # Merge small chunks to avoid very short pieces
    merged = []
    for chunk in chunks:
        # If chunk is small and previous chunk exists and combined size is reasonable
        if (merged and len(chunk['text']) < 100 and
            len(merged[-1]['text']) + len(chunk['text']) < chunk_size * 1.5):

            prev = merged[-1]
            prev['text'] += '\n\n' + chunk['text']
            prev['offset']['end'] = chunk['offset']['end']
            prev['size'] = len(prev['text'])
            prev['boundary_type'] = 'merged'
        else:
            merged.append(chunk)

    return merged

##### 1.7.6 Sentiment Analysis functions

Provide a small, robust wrapper around a Transformers sentiment pipeline (FinBERT) to produce per-chunk financial sentiment labels and confidence scores; plus a simple batch runner with progress reporting.

- analyze_fin_sentiment(text: str, model: pipeline) -> Dict[str, float]
  - Inputs: a text string and a Transformers pipeline instance (expected FinBERT-style pipeline returning label/score entries).
  - Preprocessing: truncates input to the first 512 characters to respect model/token limits.
  - Invocation: calls model(text) and expects results shaped as a list containing score dictionaries.
  - Selection: chooses the label with the highest score (max by 'score').
  - Output: returns {'sentiment': <label.lower()>, 'confidence': <rounded_score>} where confidence is rounded to 3 decimals.
  - Error handling: on any exception returns a safe fallback {'sentiment': 'neutral', 'confidence': 0.0}.

- batch_sentiment_analysis(chunks: List[str], model: pipeline) -> List[Dict[str, float]]
  - Inputs: list of chunk texts and a pipeline model.
  - Behavior: iterates over chunks, runs analyze_fin_sentiment for each, and collects results.
  - Progress: displays progress with tqdm ("Sentiment").
  - Output: list of per-chunk sentiment dicts in the same order as input.

- Design notes / assumptions
  - Built for FinBERT-style outputs where model(text) → [[{label, score}, ...]]; the wrapper extracts the best-scoring label accordingly.
  - Truncation to 512 chars prevents token-limit failures at the cost of possibly losing context on very long chunks.
  - Conservative fallback ensures downstream pipeline robustness when the sentiment model fails.

In [12]:
def analyze_fin_sentiment(text: str, model: pipeline) -> Dict[str, float]:
    """Get financial sentiment with confidence scores"""
    try:
        text = text[:512]  # Truncate to model limit
        results = model(text)
        best_result = max(results[0], key=lambda x: x['score'])
        return {
            'sentiment': best_result['label'].lower(),
            'confidence': round(best_result['score'], 3)
        }
    except Exception:
        return {'sentiment': 'neutral', 'confidence': 0.0}

def batch_sentiment_analysis(chunks: List[str], model: pipeline) -> List[Dict[str, float]]:
    """Analyze sentiment for multiple chunks"""
    results = []
    for chunk in tqdm(chunks, desc="Sentiment"):
        results.append(analyze_fin_sentiment(chunk, model))
    return results

##### 1.7.7 Embedding Generation Functions

Produce vector embeddings for text chunks using a SentenceTransformer (BGE) model, expose a thread-safe singleton loader, batch encoding with progress, and a helper to attach embeddings back to chunk dictionaries for downstream storage/search.

I chose BAAI/bge-large-en-v1.5, because it excels at capturing deep semantic meaning rather than just keywords, allowing it to identify conceptually related content across documents even when phrased differently. Its strong performance on retrieval benchmarks and optimization for asymmetric search (matching short queries to long documents) makes it highly effective at finding relevant passages for accurate summarization and cross-document comparison in RAG systems.

- generate_embeddings(chunks: List[str], model: SentenceTransformer) -> np.ndarray
  - Inputs: list of texts and a loaded SentenceTransformer instance.
  - Behavior: returns model.encode(...) for the whole list; uses show_progress_bar=True and normalize_embeddings=True.
  - Output: numpy array of embeddings; returns empty np.array if input list is empty.

- embed_texts(texts: List[str], model_name: str = 'BAAI/bge-large-en-v1.5', batch_size: int = 32, show_progress: bool = True, normalize: bool = True) -> List[List[float]]
  - obtains model via get_embedding_model(model_name), iterates in batches, encodes with normalize_embeddings=normalize, optionally displays a tqdm progress bar over batches, and collects embeddings as Python lists.
  - Output: list-of-list embeddings (float), or [] for empty input.

- embed_chunks(chunks: List[Dict], **kwargs) -> List[Dict]
  - extracts chunk['text'] for all chunks, calls embed_texts(...), then returns a new list of chunk dicts with an 'embedding' key added (zipped by order).
  - Note: downstream code expects embeddings to be numeric arrays/lists and coerces types before DB storage.

Design notes / important details
- Normalization: encoding calls default to normalized embeddings (cosine-compatible) unless overridden.
- Batch processing: embed_texts supports configurable batch_size and progress reporting to handle large corpora without exhausting memory.
- Singleton model: reduces repeated model load time and memory footprint; lock prevents race conditions in multi-threaded environments.
- Empty-input guards: functions return empty arrays/lists immediately for empty inputs to simplify caller logic.
- Interoperability: outputs are converted to Python lists before storage (other sections coerce numpy arrays to lists and handle missing embeddings).

In [13]:
# Generate embeddings function
def generate_embeddings(chunks: List[str], model: SentenceTransformer) -> np.ndarray:
    """Generate BGE embeddings for text chunks"""
    if not chunks:
        return np.array([])
    return model.encode(chunks, show_progress_bar=True, normalize_embeddings=True)


# Embedding model singleton
_embed_model, _model_lock = None, threading.Lock()
# Get singleton embedding model
def get_embedding_model(name: str = 'BAAI/bge-large-en-v1.5') -> SentenceTransformer:
    global _embed_model
    if _embed_model is None:
        with _model_lock:
            if _embed_model is None:
                _embed_model = SentenceTransformer(name)
                print(f"Loaded: {name}")
    return _embed_model

# Embed texts with progress and normalization
def embed_texts(
    texts: List[str], model_name: str = 'BAAI/bge-large-en-v1.5',
    batch_size: int = 32, show_progress: bool = True, normalize: bool = True
) -> List[List[float]]:
    """Embed texts with progress and normalization."""
    if not texts: return []

    model = get_embedding_model(model_name)
    vectors, iter_range = [], range(0, len(texts), batch_size)

    if show_progress:
        iter_range = tqdm(iter_range, desc="Embedding", unit="batch")

    for i in iter_range:
        batch = texts[i:i+batch_size]
        emb = model.encode(batch, show_progress_bar=False, normalize_embeddings=normalize)
        vectors.extend(emb.tolist())

    return vectors

# Embed chunks with metadata preservation
def embed_chunks(chunks: List[Dict], **kwargs) -> List[Dict]:
    """Embed chunks while preserving metadata."""
    texts = [chunk['text'] for chunk in chunks]
    embeddings = embed_texts(texts, **kwargs)
    return [{**chunk, 'embedding': emb} for chunk, emb in zip(chunks, embeddings)]

##### 1.7.8 Vector Database Ingestion Functions

Utilities to sanitize/flatten document metadata, coerce/normalize embeddings, build per-chunk metadata, generate stable IDs, and add enriched chunk records into a ChromaDB collection.

Key functions

- sanitize_metadata_value(key, value) -> Dict[str, Any]
  - Passes primitives through, flattens nested dicts into prefixed keys, special-cases layout lists (span count + sample texts), JSON-serializes other lists/complex values and truncates long strings to safe length.

- sanitize_metadata(metadata) -> Dict[str, Any]
  - Purpose: Apply sanitize_metadata_value across a metadata dict.
  - Behavior: Returns an empty dict for falsy input; produces a flat, primitive-only metadata dict suitable for DB storage.

- normalize_embedding(embedding, target_dim=1024) -> List[float]
  - Purpose: Coerce various embedding representations to a Python list of floats.
  - Behavior: Accepts numpy arrays, lists, objects with tolist(), or iterable; falls back to a zero vector of target_dim if coercion fails. Used to ensure ChromaDB receives numeric lists.

- build_chunk_metadata(base_meta, chunk, sentiment) -> Dict[str, Any]
  - Adds chunk_id, sentiment, confidence, size, boundary type, start/end offsets, ensures filename present, then runs sanitize_metadata before returning.

- store_enriched_chunks(collection, chunks, metadata) -> int
  - Purpose: Main ingestion routine that prepares and writes chunk documents to a ChromaDB collection.
  - Behavior:
    - Sanitizes base document metadata.
    - Builds stable ids using filename stem + chunk_id (keeps existing id if it already contains filename stem).
    - Extracts chunk texts and sentiments.
    - Coerces chunk embeddings to float lists using a local helper; detects embedding dimension from the first valid embedding and replaces missing/invalid embeddings with zero vectors of that dimension.
    - Builds per-chunk sanitized metadata via a shallow copy of base meta + chunk-specific fields.
    - Calls collection.add(ids=..., documents=..., embeddings=..., metadatas=...).
    - Returns number of chunks stored.
  - Assumptions: collection supports ChromaDB .add API; callers provide chunks in order matching any precomputed embeddings/sentiments.

Design notes / important behaviors
- Metadata flattening: prevents nested/complex objects from being stored directly; layout spans are summarized rather than persisted in full.
- Embedding handling: coercion to float lists, auto-detection of embedding dimension, and zero-vector fallback avoid DB errors and preserve consistent dimensionality.
- Stable IDs: filename-based prefixing gives document-scoped, stable chunk identifiers to help deduplication and provenance.
- Simplicity and interoperability: outputs are plain Python types (lists, dicts, primitives) to match ChromaDB expectations and downstream code.
- Empty-input guards: functions return early for empty metadata/chunk lists to simplify caller logic.

In [15]:
# Vector storage functions with metadata sanitization
def sanitize_metadata_value(key: str, value: Any) -> Dict[str, Any]:
    """Convert a single metadata value to storage-safe primitives."""
    result = {}
    
    # Primitives pass through
    if isinstance(value, (str, int, float, bool)) or value is None:
        result[key] = value
        return result
    
    # Nested dict: flatten with prefixed keys
    if isinstance(value, dict):
        for k, v in value.items():
            flat_key = f"{key}_{k}"
            if isinstance(v, (str, int, float, bool)) or v is None:
                result[flat_key] = v
            else:
                # Stringify complex nested values
                result[flat_key] = str(v)[:1000]
        return result
    
    # Lists: special handling for layout data
    if isinstance(value, list):
        if key.lower() == 'layout' and value and isinstance(value[0], dict):
            result['layout_span_count'] = len(value)
            sample_texts = [s.get('text', '')[:200] for s in value[:3] if isinstance(s, dict)]
            result['layout_sample_texts'] = ' | '.join(sample_texts)[:1000]
        else:
            result[key] = json.dumps(value, ensure_ascii=False)[:1000]
        return result
    
    # Fallback: stringify
    result[key] = str(value)[:1000]
    return result

# Sanitize entire metadata dict
def sanitize_metadata(metadata: Optional[Dict[str, Any]]) -> Dict[str, Any]:
    """Flatten and sanitize metadata for ChromaDB storage."""
    if not metadata:
        return {}
    
    sanitized = {}
    for key, value in metadata.items():
        sanitized.update(sanitize_metadata_value(key, value))
    
    return sanitized

# Normalize embedding to list of floats
def normalize_embedding(embedding: Any, target_dim: int = 1024) -> List[float]:
    """Convert embedding to list of floats, filling with zeros if missing."""
    # Handle numpy arrays
    if isinstance(embedding, np.ndarray):
        return [float(x) for x in embedding.tolist()]
    
    # Handle lists
    if isinstance(embedding, list):
        return [float(x) for x in embedding]
    
    # Handle objects with tolist
    if hasattr(embedding, 'tolist'):
        return [float(x) for x in embedding.tolist()]
    
    # Try iteration
    try:
        return [float(x) for x in list(embedding)]
    except:
        # Fallback: zero vector
        return [0.0] * target_dim

# Build per-chunk metadata
def build_chunk_metadata(base_meta: Dict, chunk: Dict, sentiment: Dict) -> Dict[str, Any]:
    """Build per-chunk metadata by merging base metadata with chunk-specific fields."""
    chunk_meta = dict(base_meta)  # Copy base
    
    # Add chunk-specific fields with safe type coercion
    chunk_meta.update({
        'chunk_id': str(chunk.get('chunk_id', 'unknown')),
        'sentiment': sentiment.get('sentiment', 'neutral') if isinstance(sentiment, dict) else 'neutral',
        'confidence': float(sentiment.get('confidence', 0.0)) if isinstance(sentiment, dict) else 0.0,
        'chunk_size': int(chunk.get('size', 0) or 0),
        'boundary_type': str(chunk.get('boundary_type', 'unknown')),
        'start_offset': int(chunk.get('offset', {}).get('start', 0) if isinstance(chunk.get('offset'), dict) else 0),
        'end_offset': int(chunk.get('offset', {}).get('end', 0) if isinstance(chunk.get('offset'), dict) else 0)
    })
    
    # Ensure filename exists
    if 'filename' not in chunk_meta or not chunk_meta.get('filename'):
        chunk_meta['filename'] = base_meta.get('filename', 'unknown')
    
    return sanitize_metadata(chunk_meta)

# Store enriched chunks in ChromaDB
def store_enriched_chunks(
    collection: chromadb.Collection,
    chunks: List[Dict],
    metadata: Dict[str, Any]
) -> int:
    """
    Store chunks with embeddings, sentiment, and metadata in ChromaDB.
    
    Args:
        collection: ChromaDB collection to store in
        chunks: List of chunk dicts with text, embedding, sentiment, etc.
        metadata: Base document-level metadata
    
    Returns:
        Number of chunks stored
    """
    if not chunks:
        return 0
    
    # Prepare base metadata
    base_meta = sanitize_metadata(metadata)
    filename_stem = Path(base_meta.get('filename', 'unknown')).stem
    
    # Extract data from chunks
    chunk_texts = [c.get('text', '') for c in chunks]
    sentiments = [c.get('sentiment', {'sentiment': 'neutral', 'confidence': 0.0}) for c in chunks]
    
    # Build stable IDs (filename_stem_chunk_id)
    ids = []
    for i, chunk in enumerate(chunks):
        chunk_id = str(chunk.get('chunk_id', f'chunk_{i}'))
        # Use chunk_id as-is if it already contains filename stem
        if filename_stem in chunk_id:
            ids.append(chunk_id)
        else:
            ids.append(f"{filename_stem}_{chunk_id}")
    
    # Normalize embeddings
    embeddings_raw = [c.get('embedding') for c in chunks]
    
    # Detect embedding dimension from first valid embedding
    emb_dim = 1024  # default
    for emb in embeddings_raw:
        normalized = normalize_embedding(emb, emb_dim)
        if normalized and normalized != [0.0] * emb_dim:
            emb_dim = len(normalized)
            break
    
    # Normalize all embeddings with detected dimension
    embeddings = [normalize_embedding(e, emb_dim) for e in embeddings_raw]
    
    # Build per-chunk metadata
    metadatas = [
        build_chunk_metadata(base_meta, chunk, sentiment)
        for chunk, sentiment in zip(chunks, sentiments)
    ]
    
    # Store in ChromaDB
    collection.add(
        ids=ids,
        documents=chunk_texts,
        embeddings=embeddings,
        metadatas=metadatas
    )
    
    return len(chunks)


print("✅ Vector Database Ingestion Functions ready")
print("   📦 Metadata sanitization with nested structure support")
print("   🔢 Embedding normalization with auto-dimension detection")
print("   🏷️ Stable ID generation (filename-based)")
print("   💾 ChromaDB storage with enriched metadata")

✅ Vector Database Ingestion Functions ready
   📦 Metadata sanitization with nested structure support
   🔢 Embedding normalization with auto-dimension detection
   🏷️ Stable ID generation (filename-based)
   💾 ChromaDB storage with enriched metadata


##### 1.7.9 Collection management functions

Small utility layer that wraps common ChromaDB collection operations with defensive checks, friendly messages and simple error handling. Exposes four primary helpers for listing, creating, inspecting and selecting collections.

Key functions

- list_existing_collections(client) -> List[str]
  - Calls client.list_collections(), extracts collection.name; on exception prints an error and returns an empty list.

- create_new_collection(client, collection_name, description: str = None) -> chromadb.Collection | None
  - Purpose: Create (or retrieve) a named collection and attach simple metadata.
  - Behavior:
    - First checks existing collections (via list_existing_collections) and returns the existing collection if present.
    - Otherwise calls client.create_collection(name=..., metadata={...}).
    - Contains targeted error handling: inspects exception text and prints actionable messages for common states (read-only store, permission issues, DB locks/busy); returns None on failure.

- get_collection_info(collection: chromadb.Collection) -> Dict[str, Any]
  - Purpose: Fetch lightweight diagnostics for a collection.
  - Behavior: Calls collection.get() and returns a dict with name, metadata, document_count (len(ids) if present) and sample_ids (first 5). On error returns a dict with the collection name and an error string.

- select_or_create_collection(client, collection_name: str = None) -> chromadb.Collection
  - Purpose: Convenience chooser that uses a supplied name or CONFIG['collection_name'] to pick or create a collection.
  - Behavior: Lists existing collections, returns existing collection if present, otherwise calls create_new_collection to create and return a new collection (prints status messages).


In [16]:
# Collection management functions with enhanced error handling
def list_existing_collections(client: chromadb.Client) -> List[str]:
    """List all existing collections in the ChromaDB instance"""
    try:
        collections = client.list_collections()
        return [collection.name for collection in collections]
    except Exception as e:
        print(f"Error listing collections: {e}")
        return []
# Create new collection with error handling
def create_new_collection(client: chromadb.Client, collection_name: str, description: str = None) -> chromadb.Collection:
    """Create a new collection with the specified name, with enhanced error handling"""
    try:
        # Check if collection already exists
        existing_collections = list_existing_collections(client)
        if collection_name in existing_collections:
            print(f"Collection '{collection_name}' already exists. Retrieving existing collection.")
            return client.get_collection(collection_name)

        # Create new collection
        metadata = {"description": description or f"Financial documents collection: {collection_name}"}
        collection = client.create_collection(name=collection_name, metadata=metadata)
        print(f"Created new collection: '{collection_name}'")
        return collection

    except Exception as e:
        error_msg = str(e).lower()

        # Handle specific database errors
        if "readonly" in error_msg or "1032" in error_msg:
            print(f"Database is read-only. Attempting to repair...")
            print("Run the database repair cell above to fix this issue.")
            print("Alternative: Restart the notebook kernel and run the repair cell.")

        elif "permission" in error_msg or "access" in error_msg:
            print(f"Permission denied. Database directory may need permission fixes.")
            print("Run the database repair cell above to fix permissions.")

        elif "lock" in error_msg or "busy" in error_msg:
            print(f"Database is locked (another process may be using it).")
            print("Close other notebook instances and try again.")

        else:
            print(f"Unexpected error creating collection '{collection_name}': {e}")
            print("Try running the database repair cell above.")

        return None
# Get collection info with document count
def get_collection_info(collection: chromadb.Collection) -> Dict[str, Any]:
    """Get information about a collection including document count"""
    try:
        data = collection.get()
        return {
            'name': collection.name,
            'metadata': collection.metadata,
            'document_count': len(data['ids']) if data['ids'] else 0,
            'sample_ids': data['ids'][:5] if data['ids'] else []
        }
    except Exception as e:
        return {'name': collection.name, 'error': str(e)}

def select_or_create_collection(client: chromadb.Client, collection_name: str = None) -> chromadb.Collection:
    """Select existing collection or create new one based on name"""
    if not collection_name:
        collection_name = CONFIG['collection_name']

    existing_collections = list_existing_collections(client)

    if collection_name in existing_collections:
        print(f"Using existing collection: '{collection_name}'")
        return client.get_collection(collection_name)
    else:
        print(f"Collection '{collection_name}' not found. Creating new collection.")
        return create_new_collection(client, collection_name)



#### 1.8 Rag workflow variables for section 5-7.

Initialize variables that would normally be created in Section 3 (document processing)

- Small compatibility/bootstrapping cell that ensures later sections (5–7) can run even if document-processing (Sections 3–4) was skipped. It defines a set of global placeholders, preserves any existing values, and emits brief runtime hints.

- all_chunks -> dict  
  - Document-indexed structure mapping each file path to its extracted text, chunk list and merged metadata. Populated in Section 3 but initialized here as an empty dict to avoid NameError in downstream code.

- all_embeddings -> list  
  - Linear list/array of embeddings for all chunks. Downstream code expects embeddings to be present; initialized empty.

- all_sentiments -> list  
  - Per-chunk sentiment result list (FinBERT outputs). Initialized empty so sentiment-dependent steps can still run defensively.

- results -> list  
  - General-purpose list used earlier for (path, text, metadata) tuples from extraction. Initialized empty for compatibility.

- all_chunk_texts -> list  
  - Flat list of chunk texts (used by embedding/sentiment batch runners and Section 4.6). Added explicitly for Section 4.6 compatibility.

- bm25_index -> None or index object  
  - Placeholder for an in-memory BM25 index; set to None when not built.

Behavior and design notes
- Guarded initialization: each variable is only created if absent in globals() so re-running cells or preserving already-processed data is safe.

In [17]:
# Initialize empty variables for Section 5-7 compatibility
if 'all_chunks' not in globals():
    all_chunks = {}  # Empty dict - would be populated by Section 3
    print("Initialized all_chunks (empty - populate via Section 3 if needed)")

if 'all_embeddings' not in globals():
    all_embeddings = []  # Empty list - would be populated by Section 3
    print("Initialized all_embeddings (empty - populate via Section 3 if needed)")
if 'all_sentiments' not in globals():
    all_sentiments = []  # Empty list - would be populated by Section 3
    print("Initialized all_sentiments (empty - populate via Section 3 if needed)")

if 'results' not in globals():
    results = []  # Empty list - would be populated by Section 3
    print("Initialized results (empty - populate via Section 3 if needed)")
if 'all_chunk_texts' not in globals():
    all_chunk_texts = []  # Empty list - would be populated by Section 3.10.3
    print("Initialized all_chunk_texts (empty - populate via Section 3 if needed)")

if 'bm25_index' not in globals():
    bm25_index = None  # Would be built in Section 5
    print("Initialized bm25_index (None - will be built in Section 5)")
print("All core functions defined and available")
print("Document discovery and metadata extraction")
print("Enhanced document processing (PDF, DOCX, TXT, MD, CSV)")
print("Text chunking with boundary respect")
print("Financial sentiment analysis")
print("Embedding generation with BGE model")
print("ChromaDB storage with metadata sanitization")
print("Collection management utilities")
print("Ready for use across all workflow scenarios!")
print("Section 5-7 compatibility variables initialized")
print("Added all_chunk_texts for section 4.6 compatibility")

Initialized all_chunks (empty - populate via Section 3 if needed)
Initialized all_embeddings (empty - populate via Section 3 if needed)
Initialized all_sentiments (empty - populate via Section 3 if needed)
Initialized results (empty - populate via Section 3 if needed)
Initialized all_chunk_texts (empty - populate via Section 3 if needed)
Initialized bm25_index (None - will be built in Section 5)
All core functions defined and available
Document discovery and metadata extraction
Enhanced document processing (PDF, DOCX, TXT, MD, CSV)
Text chunking with boundary respect
Financial sentiment analysis
Embedding generation with BGE model
ChromaDB storage with metadata sanitization
Collection management utilities
Ready for use across all workflow scenarios!
Section 5-7 compatibility variables initialized
Added all_chunk_texts for section 4.6 compatibility


## 2. Document Processing

This section includes functions for loading files, extracting and parsing text, chunking, doing sentiment analysis on chunks, creating embeddings, and storing them in ChromaDB.

- Designed to handle text extraction from different file types (PDF, DOCX, TXT, .MD, .CSV)
- Currently, it has been tested with PDF files only.

#### 2.2 Directory Verification

Quick diagnostics to verify the presence and contents of the configured data directories before running document processing.

- Inputs: uses globals established in Section 1.4 — DATA_DIR (root data folder) and file_directories (list of subdirectory Path objects).

In [21]:
print("Data directories:")
for dir_path in file_directories:
    if dir_path.exists():
        files = [f for f in dir_path.glob('*.pdf')]  # Adjust extension as needed
        print(f"{dir_path.name}: {len(files)} PDF files")
    else:
        print(f"{dir_path.name}: Not found")

Data directories:
pra_rulebook: 41 PDF files


#### 2.3 Extract Metadata from file names

In [ ]:
#### Advanced Metadata Extraction System

class ConsolidatedMetadataExtractor:
    """Unified metadata extraction combining filename parsing and content analysis"""

    def __init__(self):
        self.nlp = self._load_nlp_model()

        # Financial terms dictionary
        self.financial_terms = {
            'metrics': ['revenue', 'profit', 'earnings', 'ebitda', 'cash flow', 'roi', 'roe', 'margin', 'growth', 'loss'],
            'instruments': ['bonds', 'stocks', 'derivatives', 'securities', 'loans', 'credit', 'debt', 'equity'],
            'regulations': ['compliance', 'regulatory', 'pra', 'basel', 'mifid', 'gdpr', 'aml', 'kyc'],
            'operations': ['risk management', 'capital adequacy', 'stress testing', 'audit', 'governance']
        }

        # Document type patterns
        self.document_type_patterns = {
            'earnings_transcript': ['earnings call', 'quarterly results', 'q1', 'q2', 'q3', 'q4'],
            'annual_report': ['annual report', 'form 10-k', '10-k', 'yearly results'],
            'regulatory_filing': ['form 8-k', '8-k', 'sec filing', 'regulatory submission'],
            'analyst_meeting': ['analyst meeting', 'investor day', 'analyst call'],
            'press_release': ['press release', 'news release', 'announcement'],
            'policy_document': ['policy', 'procedure', 'guidelines', 'framework'],
            'compliance_document': ['compliance', 'regulatory requirement', 'audit report'],
            'financial_statement': ['balance sheet', 'income statement', 'cash flow statement']
        }

        # Company name patterns
        self.company_patterns = [
            r'\b([A-Z][a-z]+ (?:Bank|Banking|Financial|Capital|Investment|Securities))\b',
            r'\b(Barclays|HSBC|Lloyds|RBS|Santander|Standard Chartered)\b',
            r'\b([A-Z]{2,5})\s+(?:plc|PLC|Inc|Corporation|Corp|Ltd|Limited)\b'
        ]

    def _load_nlp_model(self):
        """Load spaCy model with fallback"""
        try:
            import spacy
            try:
                return spacy.load("en_core_web_sm")
            except OSError:
                print("⚠️ spaCy English model not found. Installing...")
                import subprocess
                subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"], check=True)
                return spacy.load("en_core_web_sm")
        except ImportError:
            print("⚠️ spaCy not available. Installing...")
            import subprocess
            subprocess.run(["pip", "install", "spacy"], check=True)
            subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"], check=True)
            import spacy
            return spacy.load("en_core_web_sm")
        except Exception as e:
            print(f"❌ Could not load spaCy model: {e}")
            return None

    def extract_comprehensive_metadata(self, filepath: Path, text: str = None) -> Dict[str, Any]:
        """
        Extract comprehensive metadata from both filename and content

        Args:
            filepath: Path to the document
            text: Document content (optional, for content-based extraction)

        Returns:
            Comprehensive metadata dictionary
        """
        # Start with filename-based metadata
        metadata = self._extract_filename_metadata(filepath)

        # Add content-based metadata if text is provided
        if text:
            content_metadata = self._extract_content_metadata(text)
            metadata.update(content_metadata)

            # Resolve conflicts and enhance with content analysis
            metadata = self._resolve_metadata_conflicts(metadata, text)

        return metadata

    def _extract_filename_metadata(self, filepath: Path) -> Dict[str, Any]:
        """Extract metadata from filename - Enhanced version"""
        name = filepath.stem.lower()
        parts = name.split('_')

        # Basic file information
        metadata = {
            'filename': filepath.name,
            'file_extension': filepath.suffix.lower(),
            'filename_parts': parts,
            'extraction_source': 'filename'
        }

        # Extract company/organization
        if parts:
            metadata['company_filename'] = parts[0]

        # Extract quarter information
        quarter_info = self._extract_quarter_from_filename(name)
        metadata.update(quarter_info)

        # Extract year information
        year_matches = re.findall(r'\b(20\d{2})\b', name)
        if year_matches:
            metadata['year_filename'] = year_matches[0]
            metadata['years_filename'] = list(set(year_matches))

        # Determine basic document type from file extension and name
        metadata['doc_type_filename'] = self._classify_doc_type_from_filename(filepath, name)

        return metadata

    def _extract_quarter_from_filename(self, filename: str) -> Dict[str, Any]:
        """Extract quarter information from filename"""
        quarter_info = {}

        # Direct quarter matches
        quarter_patterns = [
            r'\b(q[1-4])\b',
            r'\b([1-4]q)\b',
            r'\b(first|second|third|fourth).*quarter\b'
        ]

        for pattern in quarter_patterns:
            matches = re.findall(pattern, filename)
            if matches:
                quarter_info['quarter_filename'] = matches[0].upper()
                break

        return quarter_info

    def _classify_doc_type_from_filename(self, filepath: Path, name: str) -> str:
        """Classify document type from filename and extension"""
        if filepath.suffix.lower() == '.pdf':
            if 'transcript' in name:
                return 'earnings_transcript'
            elif 'meeting' in name:
                return 'analyst_meeting'
            elif 'annual' in name or 'yearly' in name:
                return 'annual_report'
            else:
                return 'financial_document'
        elif filepath.suffix.lower() in ['.docx', '.txt', '.md']:
            return 'text_document'
        elif filepath.suffix.lower() == '.csv':
            return 'data_file'
        else:
            return 'unknown'

    def _extract_content_metadata(self, text: str) -> Dict[str, Any]:
        """Extract metadata from document content"""
        content_metadata = {'extraction_source': 'content'}

        # 1. Date extraction from content
        dates_info = self._extract_dates_from_content(text)
        content_metadata.update(dates_info)

        # 2. Document type classification from content
        doc_type_info = self._classify_document_type_from_content(text)
        content_metadata.update(doc_type_info)

        # 3. Entity extraction
        entities_info = self._extract_entities(text)
        content_metadata.update(entities_info)

        # 4. Financial terms extraction
        financial_info = self._extract_financial_terms(text)
        content_metadata.update(financial_info)

        # 5. Content statistics
        content_stats = self._calculate_content_statistics(text)
        content_metadata.update(content_stats)

        return content_metadata

    def _extract_dates_from_content(self, text: str) -> Dict[str, Any]:
        """Extract dates from document content"""
        date_patterns = {
            'full_date': [
                r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},?\s+\d{4}\b',
                r'\b\d{1,2}[/-]\d{1,2}[/-]\d{4}\b',
                r'\b\d{4}[/-]\d{1,2}[/-]\d{1,2}\b'
            ],
            'quarter': [
                r'\b(?:Q[1-4]|[1-4]Q|First|Second|Third|Fourth)\s+(?:quarter|Quarter)\s+\d{4}\b',
            ],
            'year': [
                r'\bfor\s+the\s+year\s+ended?\s+\d{4}\b',
                r'\b(?:fiscal|financial)\s+year\s+\d{4}\b'
            ]
        }

        extracted_dates = {}

        for date_type, patterns in date_patterns.items():
            dates = []
            for pattern in patterns:
                matches = re.finditer(pattern, text, re.IGNORECASE)
                for match in matches:
                    dates.append(match.group().strip())

            if dates:
                extracted_dates[f'{date_type}_content'] = dates[:5]

        # Extract years for indexing
        year_matches = re.findall(r'\b(20\d{2})\b', text)
        if year_matches:
            years = list(set(year_matches))
            extracted_dates['years_content'] = sorted(years, reverse=True)[:5]
            extracted_dates['primary_year_content'] = years[0]

        # Extract quarters
        quarter_matches = re.findall(r'\b([Q1-4]|[1-4]Q)\b', text, re.IGNORECASE)
        if quarter_matches:
            extracted_dates['quarters_content'] = list(set([q.upper() for q in quarter_matches]))

        return {'content_dates': extracted_dates}

    def _classify_document_type_from_content(self, text: str) -> Dict[str, Any]:
        """Classify document type based on content patterns"""
        text_lower = text.lower()

        type_scores = {}
        for doc_type, patterns in self.document_type_patterns.items():
            score = 0
            matched_patterns = []

            for pattern in patterns:
                count = len(re.findall(re.escape(pattern.lower()), text_lower))
                if count > 0:
                    score += count
                    matched_patterns.append(pattern)

            if score > 0:
                type_scores[doc_type] = {
                    'score': score,
                    'matched_patterns': matched_patterns
                }

        primary_type = max(type_scores.keys(), key=lambda k: type_scores[k]['score']) if type_scores else None

        return {
            'doc_type_content': primary_type,
            'doc_type_scores': type_scores,
            'doc_type_confidence': type_scores[primary_type]['score'] / 10.0 if primary_type else 0.0
        }

    def _extract_entities(self, text: str) -> Dict[str, Any]:
        """Extract entities using spaCy and custom patterns"""
        entities = {
            'companies': [],
            'people': [],
            'organizations': [],
            'money_amounts': [],
            'percentages': [],
            'locations': []
        }

        # Use spaCy if available
        if self.nlp:
            try:
                doc = self.nlp(text[:5000])
                for ent in doc.ents:
                    if ent.label_ in ['PERSON']:
                        entities['people'].append(ent.text)
                    elif ent.label_ in ['ORG']:
                        entities['organizations'].append(ent.text)
                    elif ent.label_ in ['MONEY']:
                        entities['money_amounts'].append(ent.text)
                    elif ent.label_ in ['PERCENT']:
                        entities['percentages'].append(ent.text)
                    elif ent.label_ in ['GPE', 'LOC']:
                        entities['locations'].append(ent.text)
            except Exception as e:
                print(f"spaCy entity extraction failed: {e}")

        # Custom patterns
        for pattern in self.company_patterns:
            matches = re.findall(pattern, text)
            for match in matches:
                if isinstance(match, tuple):
                    entities['companies'].extend([m for m in match if m])
                else:
                    entities['companies'].append(match)

        # Clean and limit entities
        for key in entities:
            entities[key] = list(set(entities[key]))[:10]

        return {
            'extracted_entities': entities,
            'entity_counts': {k: len(v) for k, v in entities.items()}
        }

    def _extract_financial_terms(self, text: str) -> Dict[str, Any]:
        """Extract and categorize financial terms"""
        text_lower = text.lower()

        found_terms = {}
        term_frequencies = {}

        for category, terms in self.financial_terms.items():
            found_in_category = []
            frequencies = {}

            for term in terms:
                count = len(re.findall(rf'\b{re.escape(term)}\b', text_lower))
                if count > 0:
                    found_in_category.append(term)
                    frequencies[term] = count

            if found_in_category:
                found_terms[category] = found_in_category
                term_frequencies[category] = frequencies

        total_unique_terms = sum(len(terms) for terms in found_terms.values())
        financial_complexity = min(total_unique_terms / 20.0, 1.0)

        return {
            'financial_terms': found_terms,
            'financial_term_frequencies': term_frequencies,
            'financial_complexity_score': financial_complexity,
            'total_financial_terms': total_unique_terms
        }

    def _calculate_content_statistics(self, text: str) -> Dict[str, Any]:
        """Calculate content statistics"""
        word_count = len(text.split())
        sentence_count = len(re.split(r'[.!?]+', text))
        paragraph_count = len([p for p in text.split('\n\n') if p.strip()])

        avg_words_per_sentence = word_count / max(sentence_count, 1)

        has_numbers = bool(re.search(r'\d', text))
        has_tables = bool(re.search(r'\t|\s{3,}', text))
        has_financial_symbols = bool(re.search(r'[£$€%]', text))

        return {
            'content_statistics': {
                'word_count': word_count,
                'sentence_count': sentence_count,
                'paragraph_count': paragraph_count,
                'avg_words_per_sentence': round(avg_words_per_sentence, 1),
                'has_numbers': has_numbers,
                'has_tables': has_tables,
                'has_financial_symbols': has_financial_symbols
            }
        }

    def _resolve_metadata_conflicts(self, metadata: Dict[str, Any], text: str) -> Dict[str, Any]:
        """Resolve conflicts between filename and content metadata"""
        resolved = metadata.copy()

        # Resolve document type (content takes precedence if confident)
        doc_type_filename = metadata.get('doc_type_filename')
        doc_type_content = metadata.get('doc_type_content')
        doc_type_confidence = metadata.get('doc_type_confidence', 0)

        if doc_type_content and doc_type_confidence > 0.5:
            resolved['doc_type'] = doc_type_content
            resolved['doc_type_source'] = 'content'
        else:
            resolved['doc_type'] = doc_type_filename
            resolved['doc_type_source'] = 'filename'

        # Resolve company information
        company_filename = metadata.get('company_filename')
        extracted_companies = metadata.get('extracted_entities', {}).get('companies', [])

        if extracted_companies:
            resolved['company'] = extracted_companies[0]  # Use first extracted company
            resolved['company_source'] = 'content'
            resolved['all_companies'] = extracted_companies
        else:
            resolved['company'] = company_filename if company_filename else 'unknown'
            resolved['company_source'] = 'filename'

        # Resolve year information (use content if available, fallback to filename)
        year_content = metadata.get('content_dates', {}).get('primary_year_content')
        year_filename = metadata.get('year_filename')

        if year_content:
            resolved['year'] = year_content
            resolved['year_source'] = 'content'
        elif year_filename:
            resolved['year'] = year_filename
            resolved['year_source'] = 'filename'
        else:
            resolved['year'] = 'unknown'
            resolved['year_source'] = 'none'

        # Resolve quarter information (similar logic)
        quarter_content = metadata.get('content_dates', {}).get('quarters_content')
        quarter_filename = metadata.get('quarter_filename')

        if quarter_content:
            resolved['quarter'] = quarter_content[0]
            resolved['quarter_source'] = 'content'
        elif quarter_filename:
            resolved['quarter'] = quarter_filename
            resolved['quarter_source'] = 'filename'
        else:
            resolved['quarter'] = 'unknown'
            resolved['quarter_source'] = 'none'

        # Add metadata quality score
        resolved['metadata_quality_score'] = self._calculate_metadata_quality(resolved)
        resolved['advanced_extraction_applied'] = True

        return resolved

    def _calculate_metadata_quality(self, metadata: Dict[str, Any]) -> float:
        """Calculate metadata quality score"""
        quality_factors = []

        # Check completeness
        key_fields = ['company', 'year', 'quarter', 'doc_type']
        completeness = sum(1 for field in key_fields if metadata.get(field) != 'unknown') / len(key_fields)
        quality_factors.append(completeness)

        # Check content-based extraction
        if metadata.get('extracted_entities', {}).get('entity_counts', {}).get('companies', 0) > 0:
            quality_factors.append(0.8)

        # Check financial complexity
        financial_complexity = metadata.get('financial_complexity_score', 0)
        quality_factors.append(financial_complexity)

        # Check content statistics
        content_stats = metadata.get('content_statistics', {})
        if content_stats.get('word_count', 0) > 100:
            quality_factors.append(0.7)

        return sum(quality_factors) / len(quality_factors) if quality_factors else 0.5

# Initialize the consolidated metadata extractor
try:
    consolidated_metadata_extractor = ConsolidatedMetadataExtractor()
    print("✅ CONSOLIDATED Metadata Extraction System initialized!")
    print("   🔄 Unified filename + content analysis")
    print("   📅 Enhanced date extraction")
    print("   🏢 Smart entity extraction")
    print("   💼 Financial domain optimization")
    print("   ⚖️ Conflict resolution between sources")
    print("   📊 Metadata quality scoring")

    if consolidated_metadata_extractor.nlp:
        print("   🧠 spaCy NLP model loaded successfully")
    else:
        print("   ⚠️ spaCy NLP model not available (basic extraction only)")

except Exception as e:
    print(f"❌ Consolidated metadata extractor initialization failed: {e}")
    consolidated_metadata_extractor = None


✅ CONSOLIDATED Metadata Extraction System initialized!
   🔄 Unified filename + content analysis
   📅 Enhanced date extraction
   🏢 Smart entity extraction
   💼 Financial domain optimization
   ⚖️ Conflict resolution between sources
   📊 Metadata quality scoring
   🧠 spaCy NLP model loaded successfully


#### 2.4 Document Text Extraction and Parsing

In [ ]:
# Enhanced Document Parser
class EnhancedDocumentProcessor:
    def __init__(self):
        self.supported_formats = ['.pdf', '.docx', '.txt', '.md', '.csv']

    def clean_text(self, text: str) -> str:
        """Return a minimal cleaned unicode string. Returns empty string for falsy input."""
        if not text:
            return ''
        # handle bytes
        if isinstance(text, (bytes, bytearray)):
            try:
                text = text.decode('utf8')
            except Exception:
                text = text.decode('utf8', errors='replace')
        # remove nulls, normalize newlines, collapse long runs
        text = text.replace('\x00', '')
        text = re.sub(r"\r\n|\r", "\n", text)
        text = re.sub(r"\n{3,}", "\n\n", text)
        return text.strip()

    def _extract_pdf_with_layout(self, file_path: str) -> Tuple[str, List[Dict]]:
        """Enhanced PDF extraction with layout metadata (Function A logic)"""
        full_text, layout_data = [], []
        try:
            with fitz.open(file_path) as doc:
                for page_num, page in enumerate(doc, 1):
                    blocks = page.get_text("dict").get("blocks", [])
                    for block in blocks:
                        for line in block.get("lines", []):
                            for span in line.get("spans", []):
                                text = span.get("text", "").strip()
                                if text:
                                    font = span.get("font", "") or ""
                                    layout_data.append({
                                        "text": text,
                                        "page": page_num,
                                        "bbox": span.get("bbox"),
                                        "font_size": span.get("size"),
                                        "font": font,
                                        "is_bold": "bold" in font.lower(),
                                        "is_italic": "italic" in font.lower()
                                    })
                                    full_text.append(text)
            return "\n".join(full_text), layout_data
        except Exception as e:
            raise RuntimeError(f"Layout PDF extraction failed: {e}")

    def _extract_pdf_simple(self, file_path: str) -> str:
        """Simple PDF extraction (Function B logic)"""
        try:
            doc = fitz.open(file_path)
            parts = [page.get_text() for page in doc]
            return '\n'.join(parts)
        except Exception as e:
            raise RuntimeError(f"Simple PDF extraction failed: {e}")

    def extract_text(self, file_path: str, pdf_mode: str = "auto") -> Tuple[str, Optional[Dict]]:
        """
        Extract text from various file formats with smart PDF handling

        Args:
            file_path: Path to the file
            pdf_mode: "simple" (fast), "layout" (detailed), or "auto" (detect complexity)

        Returns:
            Tuple of (extracted_text, metadata) where metadata contains layout info for PDFs
        """
        p = Path(file_path)
        if not p.exists():
            raise ValueError(f'File not found: {file_path}')

        suffix = p.suffix.lower()
        metadata = None

        if suffix == '.pdf':
            # Auto-detect if we need layout extraction
            if pdf_mode == "auto":
                # Simple heuristic: use layout for potentially complex PDFs
                try:
                    with fitz.open(file_path) as doc:
                        # Check if PDF has multiple columns or complex layout
                        if doc.page_count > 3:  # Longer documents might be complex
                            text, layout_data = self._extract_pdf_with_layout(file_path)
                            metadata = {"layout": layout_data, "extraction_mode": "layout_auto"}
                        else:
                            text = self._extract_pdf_simple(file_path)
                            metadata = {"extraction_mode": "simple_auto"}
                except:
                    # Fallback to simple extraction
                    text = self._extract_pdf_simple(file_path)
                    metadata = {"extraction_mode": "simple_fallback"}

            elif pdf_mode == "layout":
                text, layout_data = self._extract_pdf_with_layout(file_path)
                metadata = {"layout": layout_data, "extraction_mode": "layout_forced"}

            else:  # simple
                text = self._extract_pdf_simple(file_path)
                metadata = {"extraction_mode": "simple_forced"}

        elif suffix in ('.docx',):
            doc = docx.Document(str(p))
            parts = [para.text for para in doc.paragraphs]
            text = '\n'.join(parts)
            metadata = {"extraction_mode": "docx"}

        elif suffix in ('.txt', '.md'):
            text = p.read_text(encoding='utf-8', errors='ignore')
            metadata = {"extraction_mode": "text"}

        elif suffix in ('.csv',):
            df = pd.read_csv(p)
            text = df.to_csv(index=False)
            metadata = {"extraction_mode": "csv"}

        else:
            raise ValueError(f'Unhandled file type: {suffix}')

        cleaned = self.clean_text(text)
        if not cleaned:
            raise ValueError(f'No text extracted from: {file_path}')

        return cleaned, metadata

    def process_files(self, file_paths, pdf_mode: str = "auto", show_progress: bool = True) -> List[Tuple[Path, Optional[str], Optional[Dict]]]:
        """Process multiple files, return (Path, text, metadata) tuples"""
        files = list(file_paths)
        results = []
        success = 0

        with tqdm(total=len(files), desc="Processing files", disable=not show_progress) as pbar:
            for fp in files:
                try:
                    text, metadata = self.extract_text(str(fp), pdf_mode=pdf_mode)
                    results.append((Path(fp), text, metadata))
                    success += 1
                except Exception as e:
                    results.append((Path(fp), None, {"error": str(e)}))
                finally:
                    pbar.update(1)
                    pbar.set_postfix(success=success, failed=len(results)-success)

        if show_progress:
            print(f'Processed {len(files)} files ({success} succeeded, {len(files) - success} failed)')

        return results

#### 2.5 Run Data Processor

In [ ]:
# Default: auto-detect PDF complexity. Options: "auto", "layout", "simple". Auto should detect complexity of files (needs to be tested more)
processor = EnhancedDocumentProcessor()
results = processor.process_files(files, pdf_mode="auto")

Processing files:   0%|          | 0/41 [00:00<?, ?it/s]

Processed 41 files (41 succeeded, 0 failed)


#### 2.6 Check results

In [ ]:
# Check results with layout data
for path, text, metadata in results[:1]:
    if metadata and 'layout' in metadata:
        print(f"\nPDF with layout data: {path.name}")
        print(f"Extracted {len(metadata['layout'])} text spans")
        print(f"Sample text: {text[:500]}...")
    elif text:
        print(f"\nFile: {path.name}")
        print(f"Text: {text[:500]}...")


PDF with layout data: Market Risk_16-09-2025.pdf
Extracted 204 text spans
Sample text: Prudential Regulation Authority Rulebook
Part
Market Risk
Printed on: 16/09/2025
Rulebook at: 16/09/2025
Related links
PS7/13 - Strengthening capital standar
ds: implementing CRD IV, feedback an
d final rules
https://www.bankofengland.co.uk/prudential-regula
tion/publication/2013/strengthening-capital-standar
ds-implementing-crd-4
PS20/21 - Financial holding companie
s: Further implementation
https://www.bankofengland.co.uk/prudential-regula
tion/publication/2021/june/financial-holding-compa
nie...


#### 2.7 Merge metadata

In [ ]:
# Apply consolidated metadata extraction to all files
for path, text, extraction_meta in results:
    # Use the new consolidated extractor
    if consolidated_metadata_extractor:
        try:
            # Extract comprehensive metadata (filename + content)
            comprehensive_metadata = consolidated_metadata_extractor.extract_comprehensive_metadata(path, text)

            # Merge with any existing extraction metadata
            if extraction_meta:
                comprehensive_metadata.update(extraction_meta)

            # Store in results (replace the old metadata)
            # Update the results tuple with new metadata
            for i, (result_path, result_text, result_meta) in enumerate(results):
                if result_path == path:
                    results[i] = (result_path, result_text, comprehensive_metadata)
                    break

        except Exception as e:
            print(f"❌ Consolidated metadata extraction failed for {path.name}: {e}")
            # Fallback to basic extraction
            file_meta = extract_file_metadata(path)
            merged_meta = merge_metadata(file_meta, extraction_meta)

            for i, (result_path, result_text, result_meta) in enumerate(results):
                if result_path == path:
                    results[i] = (result_path, result_text, merged_meta)
                    break
    else:
        print("⚠️ Consolidated extractor not available, using basic extraction")
        file_meta = extract_file_metadata(path)
        merged_meta = merge_metadata(file_meta, extraction_meta)

        for i, (result_path, result_text, result_meta) in enumerate(results):
            if result_path == path:
                results[i] = (result_path, result_text, merged_meta)
                break

print("✅ Consolidated metadata extraction applied to all documents!")

# Display sample of enhanced metadata
if results:
    sample_path, sample_text, sample_metadata = results[0]
    print(f"\n📋 Enhanced Metadata Sample for '{sample_path.name}':")
    print("=" * 60)

    # Core information
    print(f"Company: {sample_metadata.get('company', 'Unknown')} (source: {sample_metadata.get('company_source', 'unknown')})")
    print(f"Year: {sample_metadata.get('year', 'Unknown')} (source: {sample_metadata.get('year_source', 'unknown')})")
    print(f"Quarter: {sample_metadata.get('quarter', 'Unknown')} (source: {sample_metadata.get('quarter_source', 'unknown')})")
    print(f"Document Type: {sample_metadata.get('doc_type', 'Unknown')} (source: {sample_metadata.get('doc_type_source', 'unknown')})")

    # Quality metrics
    print(f"Metadata Quality Score: {sample_metadata.get('metadata_quality_score', 0):.2f}")
    print(f"Financial Complexity: {sample_metadata.get('financial_complexity_score', 0):.2f}")

    # Entity counts
    entity_counts = sample_metadata.get('entity_counts', {})
    if entity_counts:
        print(f"Entities Found: {dict(entity_counts)}")

    # Content statistics
    content_stats = sample_metadata.get('content_statistics', {})
    if content_stats:
        print(f"Content: {content_stats.get('word_count', 0)} words, {content_stats.get('sentence_count', 0)} sentences")

✅ Consolidated metadata extraction applied to all documents!

📋 Enhanced Metadata Sample for 'Market Risk_16-09-2025.pdf':
Company: Internal Capital (source: content)
Year: 2021 (source: content)
Quarter: 3 (source: content)
Document Type: financial_document (source: filename)
Metadata Quality Score: 0.65
Financial Complexity: 0.25
Entities Found: {'companies': 1, 'people': 0, 'organizations': 9, 'money_amounts': 0, 'percentages': 1, 'locations': 1}
Content: 707 words, 57 sentences


#### 2.8 Chunking Function


##### 2.8.1 Chunking with Overlap

In [ ]:
# Enhanced chunking function with compatibility fixes
def clean_text(text: str) -> str:
    """Clean text for processing"""
    if not text:
        return ''
    # Handle bytes
    if isinstance(text, (bytes, bytearray)):
        try:
            text = text.decode('utf8')
        except Exception:
            text = text.decode('utf8', errors='replace')
    # Clean text
    text = text.replace('\x00', '')
    text = re.sub(r"\r\n|\r", "\n", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def chunk_text_enhanced(
    text: str, chunk_size: int = 600, overlap: int = 100,
    chunk_id_prefix: str = "chunk", respect_boundaries: bool = True
) -> List[Dict]:
    """Enhanced chunking with boundary respect and metadata - compatible with embeddings and sentiment analysis."""
    if not text:
        return []

    # Clean input text
    text = clean_text(text)
    if not text:
        return []

    chunks, idx, i = [], 0, 0

    while idx < len(text):
        start, end = idx, min(idx + chunk_size, len(text))

        # Find natural boundaries if requested
        if respect_boundaries and end < len(text):
            # Look for paragraph breaks, sentences, clauses
            for pattern in [r'\n\n', r'\n', r'\. ', r'; ', r', ']:
                search_start = max(start, end - overlap - 50)
                matches = list(re.finditer(pattern, text[search_start:end]))
                if matches and (boundary_pos := search_start + matches[-1].end()) > start + chunk_size // 2:
                    end = boundary_pos
                    break

        chunk_text = text[start:end].strip()
        if chunk_text:  # Only add non-empty chunks
            chunks.append({
                "chunk_id": f"{chunk_id_prefix}_{i}",
                "text": chunk_text,
                "offset": {"start": start, "end": end},
                "size": len(chunk_text),
                "boundary_type": "natural" if respect_boundaries and end < len(text) else "hard"
            })
            i += 1

        # Move to next position with overlap
        idx = max(end - overlap, start + 1)

    # Merge small chunks to avoid very short pieces
    merged = []
    for chunk in chunks:
        # If chunk is small and previous chunk exists and combined size is reasonable
        if (merged and len(chunk['text']) < 100 and
            len(merged[-1]['text']) + len(chunk['text']) < chunk_size * 1.5):

            prev = merged[-1]
            prev['text'] += '\n\n' + chunk['text']
            prev['offset']['end'] = chunk['offset']['end']
            prev['size'] = len(prev['text'])
            prev['boundary_type'] = 'merged'
        else:
            merged.append(chunk)

    return merged

print(f"Enhanced chunking function ready (compatible with embeddings & sentiment)")

Enhanced chunking function ready (compatible with embeddings & sentiment)


##### 2.8.2 Call Chunking Function on all documents

In [ ]:
# Call Chunk on all texts with proper metadata merging
all_chunks = {}
total_files = len(results)
total_chunks = 0

with tqdm(total=total_files, desc="Chunking files", unit="file") as pbar:
    for path, text, extraction_meta in results:
        # Extract file metadata from path - THIS IS THE FIX
        file_meta = extract_file_metadata(path)

        # Merge with extraction metadata - THIS IS THE FIX
        merged_meta = merge_metadata(file_meta, extraction_meta)

        # Use the file stem as chunk_id prefix so chunk ids are stable and include file context
        chunks = chunk_text_enhanced(text, chunk_size=600, overlap=100, chunk_id_prefix=path.stem)

        # Store canonical path with PROPERLY MERGED metadata - THIS IS THE FIX
        all_chunks[str(path)] = {"text": text, "chunks": chunks, "metadata": merged_meta}

        total_chunks += len(chunks)
        pbar.set_postfix({'file': path.name, 'chunks': len(chunks), 'total_chunks': total_chunks})
        pbar.update(1)

print(f"Chunking complete: processed {total_files} file(s), generated {total_chunks} chunks")

Chunking files:   0%|          | 0/41 [00:00<?, ?file/s]

Chunking complete: processed 41 file(s), generated 2991 chunks


##### 2.8.3 Display sample chunk with metadata

In [ ]:
# show a small sample for quick inspection
for name, doc in list(all_chunks.items())[:1]:
    if doc.get('chunks'):
        print(f"\nSample chunks from '{name}' (first 2):\n---\n")
        for i, chunk_dict in enumerate(doc['chunks'][:2]):
            chunk_text = chunk_dict['text']  # Extract text from the dictionary
            print(f"Chunk {i+1} (id={chunk_dict['chunk_id']}, len={len(chunk_text)}):")
            print(f"Boundary: {chunk_dict['boundary_type']}, Offset: {chunk_dict['offset']}")
            print(f"Text:\n{chunk_text}\n---\n")


Sample chunks from '/Users/rodrigograndy/Desktop/coding_projects/rag_fin/data/pra_rulebook/Market Risk_16-09-2025.pdf' (first 2):
---

Chunk 1 (id=Market Risk_16-09-2025_0, len=582):
Boundary: natural, Offset: {'start': 0, 'end': 583}
Text:
Prudential Regulation Authority Rulebook
Part
Market Risk
Printed on: 16/09/2025
Rulebook at: 16/09/2025
Related links
PS7/13 - Strengthening capital standar
ds: implementing CRD IV, feedback an
d final rules
https://www.bankofengland.co.uk/prudential-regula
tion/publication/2013/strengthening-capital-standar
ds-implementing-crd-4
PS20/21 - Financial holding companie
s: Further implementation
https://www.bankofengland.co.uk/prudential-regula
tion/publication/2021/june/financial-holding-compa
nies-further-implementation
Legislation.gov.uk
http://www.legislation.gov.uk/
Eur-Lex
---

Chunk 2 (id=Market Risk_16-09-2025_1, len=553):
Boundary: natural, Offset: {'start': 483, 'end': 1037}
Text:
holding-compa
nies-further-implementation
Legislation.gov.uk


#### 2.9 Sentiment Analysis

##### 2.9.1 Sentiment analysis on Chunks

In [ ]:
# Financial sentiment analysis
def analyze_fin_sentiment(text: str, model: pipeline) -> Dict[str, float]:
    """Get financial sentiment with confidence scores"""
    try:
        text = text[:512]  # Truncate to model limit
        results = model(text)
        best_result = max(results[0], key=lambda x: x['score'])
        return {
            'sentiment': best_result['label'].lower(),
            'confidence': round(best_result['score'], 3)
        }
    except Exception:
        return {'sentiment': 'neutral', 'confidence': 0.0}

def batch_sentiment_analysis(chunks: List[str], model: pipeline) -> List[Dict[str, float]]:
    """Analyze sentiment for multiple chunks"""
    results = []
    for chunk in tqdm(chunks, desc="Sentiment"):
        results.append(analyze_fin_sentiment(chunk, model))
    return results

print(f"Sentiment analysis functions ready")

Sentiment analysis functions ready


##### 2.9.2 Apply Sentiment Analysis to all chunks

In [ ]:
# Apply sentiment analysis to all chunks
print("Applying sentiment analysis to all chunks...")

# Collect all chunks from all documents for batch processing
all_chunk_texts = []
chunk_file_mapping = []  # Keep track of which file each chunk belongs to

for name, doc in all_chunks.items():
    if doc.get('chunks'):
        for chunk in doc['chunks']:
            all_chunk_texts.append(chunk['text'])
            chunk_file_mapping.append(name)

# Process all chunks with a single progress bar
print(f"Processing {len(all_chunk_texts)} chunks across {len(all_chunks)} files...")
all_sentiments = []
for chunk_text in tqdm(all_chunk_texts, desc="Sentiment analysis"):
    sentiment = analyze_fin_sentiment(chunk_text, finbert)
    all_sentiments.append(sentiment)

# Assign sentiment results back to chunks
sentiment_idx = 0
for name, doc in all_chunks.items():
    if doc.get('chunks'):
        for chunk in doc['chunks']:
            chunk['sentiment'] = all_sentiments[sentiment_idx]
            sentiment_idx += 1

print(f"Sentiment analysis complete for all {len(all_chunk_texts)} chunks")

Applying sentiment analysis to all chunks...
Processing 2991 chunks across 41 files...


Sentiment analysis:   0%|          | 0/2991 [00:00<?, ?it/s]

Sentiment analysis complete for all 2991 chunks


##### 2.9.3 Display sample chunk with sentiment

In [ ]:
# Print a sample chunk with sentiment for verification
for name, doc in list(all_chunks.items())[:1]:
    if doc.get('chunks'):
        print(f"\nSample chunk with sentiment from '{Path(name).name}':\n" + "="*60 + "\n")
        sample_chunk = doc['chunks'][0]
        print(f"Chunk ID: {sample_chunk['chunk_id']}")
        print(f"Text Length: {len(sample_chunk['text'])} characters")

        # Check if sentiment data exists
        sentiment_data = sample_chunk.get('sentiment', {})
        if sentiment_data and 'sentiment' in sentiment_data:
            sentiment_label = sentiment_data['sentiment'].title()
            confidence = sentiment_data.get('confidence', 0.0)
            print(f"Sentiment: {sentiment_label} (confidence: {confidence:.1%})")
        else:
            print(f"Sentiment: Not available (run sentiment analysis first)")

        print(f"Boundary Type: {sample_chunk.get('boundary_type', 'unknown')}")
        print(f"Offset: {sample_chunk.get('offset', {})}")
        print(f"\nText Sample:\n{sample_chunk['text'][:300]}{'...' if len(sample_chunk['text']) > 300 else ''}\n")
        print(f"Document Metadata:")
        if doc['metadata']:
            for key, value in doc['metadata'].items():
                if key == 'layout':
                    print(f"   {key}: {len(value)} layout spans" if isinstance(value, list) else f"   {key}: {value}")
                else:
                    print(f"   {key}: {value}")
        else:
            print("   No metadata available")
        print("="*60 + "\n")


Sample chunk with sentiment from 'Market Risk_16-09-2025.pdf':

Chunk ID: Market Risk_16-09-2025_0
Text Length: 582 characters
Sentiment: Neutral (confidence: 100.0%)
Boundary Type: natural
Offset: {'start': 0, 'end': 583}

Text Sample:
Prudential Regulation Authority Rulebook
Part
Market Risk
Printed on: 16/09/2025
Rulebook at: 16/09/2025
Related links
PS7/13 - Strengthening capital standar
ds: implementing CRD IV, feedback an
d final rules
https://www.bankofengland.co.uk/prudential-regula
tion/publication/2013/strengthening-capit...

Document Metadata:
   filename: Market Risk_16-09-2025.pdf
   company: market risk
   quarter: 16-09-2025
   doc_type: financial_document
   file_extension: .pdf
   filename_parts: ['market risk', '16-09-2025']
   extraction_source: content
   company_filename: market risk
   year_filename: 2025
   years_filename: ['2025']
   doc_type_filename: financial_document
   content_dates: {'full_date_content': ['16/09/2025', '16/09/2025', '17/09/2021', '17/09/2

#### 2.10 Embedding Generation

##### 2.10.1 Embedding Generation Function

In [ ]:
# Embedding generation functions
def generate_embeddings(chunks: List[str], model: SentenceTransformer) -> np.ndarray:
    """Generate BGE embeddings for text chunks"""
    if not chunks:
        return np.array([])
    return model.encode(chunks, show_progress_bar=True, normalize_embeddings=True)

print(f"Embedding generation function ready")

Embedding generation function ready


##### 2.10.2 Choose Embedding Model

In [ ]:
# Configuration: Chose model based on quality and speed needs.
EMBEDDING_MODEL = ['all-MiniLM-L6-v2',
    'thenlper/gte-large',   # Very good quality
    'sentence-transformers/all-mpnet-base-v2',  # Great balance
    'BAAI/bge-large-en-v1.5',  # State-of-art (requires Transformers lib)
]

# Model singleton
_embed_model, _model_lock = None, threading.Lock()

def get_embedding_model(name: str = EMBEDDING_MODEL[3]) -> SentenceTransformer:
    global _embed_model
    if _embed_model is None:
        with _model_lock:
            if _embed_model is None:
                _embed_model = SentenceTransformer(name)
                print(f"Loaded: {name}")
    return _embed_model

def embed_texts(
    texts: List[str], model_name: str = EMBEDDING_MODEL[3],
    batch_size: int = 32, show_progress: bool = True, normalize: bool = True
) -> List[List[float]]:
    """Embed texts with progress and normalization."""
    if not texts: return []

    model = get_embedding_model(model_name)
    vectors, iter_range = [], range(0, len(texts), batch_size)

    if show_progress:
        iter_range = tqdm(iter_range, desc="Embedding", unit="batch")

    for i in iter_range:
        batch = texts[i:i+batch_size]
        emb = model.encode(batch, show_progress_bar=False, normalize_embeddings=normalize)
        vectors.extend(emb.tolist())

    return vectors

def embed_chunks(chunks: List[Dict], **kwargs) -> List[Dict]:
    """Embed chunks while preserving metadata."""
    texts = [chunk['text'] for chunk in chunks]
    embeddings = embed_texts(texts, **kwargs)
    return [{**chunk, 'embedding': emb} for chunk, emb in zip(chunks, embeddings)]

print(f"Embedding utility functions ready")

Embedding utility functions ready


##### 2.10.3 Generate Embeddings for all chunks

In [ ]:
# Apply embedding generation to all chunks
print("Applying embedding generation to all chunks...")

# Collect all chunk texts from all documents for batch processing
all_chunk_texts = []
chunk_file_mapping = []  # Keep track of which file each chunk belongs to

for name, doc in all_chunks.items():
    if doc.get('chunks'):
        for chunk in doc['chunks']:
            all_chunk_texts.append(chunk['text'])
            chunk_file_mapping.append(name)

# Process all chunks with a single progress bar
print(f"Processing {len(all_chunk_texts)} chunks across {len(all_chunks)} files...")
all_embeddings = []

# Generate embeddings for all chunks at once
if all_chunk_texts:
    all_embeddings = generate_embeddings(all_chunk_texts, embedding_model)

# Assign embedding results back to chunks
embedding_idx = 0
total_chunks_embedded = 0

for name, doc in all_chunks.items():
    if doc.get('chunks'):
        for chunk in doc['chunks']:
            if embedding_idx < len(all_embeddings):
                chunk['embedding'] = all_embeddings[embedding_idx]
                embedding_idx += 1
                total_chunks_embedded += 1

print(f"Embedding generation complete for all {total_chunks_embedded} chunks")

Applying embedding generation to all chunks...
Processing 2991 chunks across 41 files...


Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Embedding generation complete for all 2991 chunks


##### 2.10.4 Display sample embedding

In [ ]:
# Print a sample chunk with embeddings for verification
for name, doc in list(all_chunks.items())[:1]:
    if doc.get('chunks'):
        print(f"\nSample chunk with embeddings from '{Path(name).name}':\n---\n")
        sample_chunk = doc['chunks'][0]
        print(f"Chunk ID: {sample_chunk['chunk_id']}")
        print(f"Text Length: {len(sample_chunk['text'])} characters")
        print(f"Text Sample: {sample_chunk['text'][:200]}...")

        embedding = sample_chunk.get('embedding')
        if embedding is not None:
            if isinstance(embedding, np.ndarray):
                print(f"Embedding: numpy array of shape {embedding.shape}")
                print(f"   First 5 values: {embedding[:5]}")
            elif isinstance(embedding, list):
                print(f"Embedding: list of length {len(embedding)}")
                print(f"   First 5 values: {embedding[:5]}")
            else:
                print(f"Embedding: {type(embedding)}")
        else:
            print("No embedding found! (run embedding generation first)")

        # Check sentiment data
        sentiment = sample_chunk.get('sentiment', {})
        if sentiment and 'sentiment' in sentiment:
            print(f"Sentiment: {sentiment['sentiment'].title()} (confidence: {sentiment.get('confidence', 0.0):.1%})")
        else:
            print("Sentiment: Not available (run sentiment analysis first)")

        print("---\n")


Sample chunk with embeddings from 'Market Risk_16-09-2025.pdf':
---

Chunk ID: Market Risk_16-09-2025_0
Text Length: 582 characters
Text Sample: Prudential Regulation Authority Rulebook
Part
Market Risk
Printed on: 16/09/2025
Rulebook at: 16/09/2025
Related links
PS7/13 - Strengthening capital standar
ds: implementing CRD IV, feedback an
d fin...
Embedding: numpy array of shape (1024,)
   First 5 values: [-0.00994055 -0.00976403 -0.01132176  0.00842187  0.0186065 ]
Sentiment: Neutral (confidence: 100.0%)
---



## 4. Database Storage

#### 4.1 Collection Selection for Storage


In [37]:
# Collection Management Functions
def list_existing_collections(client: chromadb.Client) -> List[str]:
    """List all existing collections in the ChromaDB instance"""
    try:
        collections = client.list_collections()
        return [collection.name for collection in collections]
    except Exception as e:
        print(f"Error listing collections: {e}")
        return []

def create_new_collection(client: chromadb.Client, collection_name: str, description: str = None) -> chromadb.Collection:
    """Create a new collection with the specified name, with enhanced error handling"""
    try:
        # Check if collection already exists
        existing_collections = list_existing_collections(client)
        if collection_name in existing_collections:
            print(f"Collection '{collection_name}' already exists. Retrieving existing collection.")
            return client.get_collection(collection_name)

        # Create new collection
        metadata = {"description": description or f"Financial documents collection: {collection_name}"}
        collection = client.create_collection(name=collection_name, metadata=metadata)
        print(f"Created new collection: '{collection_name}'")
        return collection

    except Exception as e:
        error_msg = str(e).lower()

        # Handle specific database errors
        if "readonly" in error_msg or "1032" in error_msg:
            print(f"Database is read-only. Attempting to repair...")
            print("Run the database repair cell above to fix this issue.")
            print("Alternative: Restart the notebook kernel and run the repair cell.")

        elif "permission" in error_msg or "access" in error_msg:
            print(f"Permission denied. Database directory may need permission fixes.")
            print("Run the database repair cell above to fix permissions.")

        elif "lock" in error_msg or "busy" in error_msg:
            print(f"Database is locked (another process may be using it).")
            print("Close other notebook instances and try again.")

        else:
            print(f"Unexpected error creating collection '{collection_name}': {e}")
            print("Try running the database repair cell above.")

        return None

def get_collection_info(collection: chromadb.Collection) -> Dict[str, Any]:
    """Get information about a collection including document count"""
    try:
        data = collection.get()
        return {
            'name': collection.name,
            'metadata': collection.metadata,
            'document_count': len(data['ids']) if data['ids'] else 0,
            'sample_ids': data['ids'][:5] if data['ids'] else []
        }
    except Exception as e:
        return {'name': collection.name, 'error': str(e)}

def select_or_create_collection(client: chromadb.Client, collection_name: str = None) -> chromadb.Collection:
    """Select existing collection or create new one based on name"""
    if not collection_name:
        collection_name = CONFIG['collection_name']

    existing_collections = list_existing_collections(client)

    if collection_name in existing_collections:
        print(f"Using existing collection: '{collection_name}'")
        return client.get_collection(collection_name)
    else:
        print(f"Collection '{collection_name}' not found. Creating new collection.")
        return create_new_collection(client, collection_name)

print("Collection management functions ready")

Collection management functions ready


#### 4.2 Instantiate Collection Manager and Store Documents

In [38]:
# Interactive Collection Selection Interface
class CollectionSelector:
    def __init__(self, client: chromadb.Client):
        self.client = client
        self.selected_collection = None
        self.existing_collections = list_existing_collections(client)

        # Create UI components
        self.collection_mode = widgets.RadioButtons(
            options=['Use Existing Collection', 'Create New Collection'],
            value='Use Existing Collection',
            description='Action:',
            style={'description_width': 'initial'}
        )

        self.existing_dropdown = widgets.Dropdown(
            options=self.existing_collections or ['No collections found'],
            description='Select:',
            disabled=len(self.existing_collections) == 0
        )

        self.new_name_text = widgets.Text(
            value='',
            placeholder='Enter new collection name',
            description='New Name:',
            disabled=True
        )

        self.new_description_text = widgets.Textarea(
            value='',
            placeholder='Optional: Enter collection description',
            description='Description:',
            disabled=True,
            rows=2
        )

        self.confirm_button = widgets.Button(
            description='Confirm Selection',
            button_style='success',
            disabled=False
        )

        self.output = widgets.Output()

        # Set up event handlers
        self.collection_mode.observe(self._on_mode_change, names='value')
        self.confirm_button.on_click(self._on_confirm_click)

        # Initialize UI state
        self._update_ui_state()

    def _on_mode_change(self, change):
        """Handle mode change between existing and new collection"""
        self._update_ui_state()

    def _update_ui_state(self):
        """Update UI component states based on selected mode"""
        is_new = self.collection_mode.value == 'Create New Collection'

        self.existing_dropdown.disabled = is_new or len(self.existing_collections) == 0
        self.new_name_text.disabled = not is_new
        self.new_description_text.disabled = not is_new

        if is_new:
            self.new_name_text.value = ''
            self.new_description_text.value = ''

    def _on_confirm_click(self, button):
        """Handle confirm button click"""
        with self.output:
            clear_output()

            if self.collection_mode.value == 'Use Existing Collection':
                if not self.existing_collections:
                    print("No existing collections available. Please create a new one.")
                    return

                collection_name = self.existing_dropdown.value
                if collection_name == 'No collections found':
                    print("No valid collection selected.")
                    return

                try:
                    self.selected_collection = self.client.get_collection(collection_name)
                    info = get_collection_info(self.selected_collection)
                    print(f"Selected existing collection: '{collection_name}'")
                    print(f"   Documents: {info['document_count']}")
                    if info.get('metadata'):
                        print(f"   Description: {info['metadata'].get('description', 'No description')}")
                except Exception as e:
                    print(f"Error selecting collection '{collection_name}': {e}")
                    return

            else:  # Create New Collection
                collection_name = self.new_name_text.value.strip()

                if not collection_name:
                    print("Please enter a collection name.")
                    return

                # Validate collection name (basic validation)
                if not collection_name.replace('_', '').replace('-', '').isalnum():
                    print("Collection name should contain only letters, numbers, hyphens, and underscores.")
                    return

                description = self.new_description_text.value.strip()

                try:
                    self.selected_collection = create_new_collection(self.client, collection_name, description)
                    if self.selected_collection:
                        print(f"Created new collection: '{collection_name}'")
                        if description:
                            print(f"   Description: {description}")
                    else:
                        print(f"Failed to create collection '{collection_name}'")
                        return
                except Exception as e:
                    print(f"Error creating collection '{collection_name}': {e}")
                    return

            # Update the global collection variable
            global collection
            collection = self.selected_collection
            print(f"Active collection set to: '{self.selected_collection.name}'")

    def display(self):
        """Display the collection selection interface"""
        print("Collection Selection for Storage")
        print("=" * 50)

        # Show existing collections info
        if self.existing_collections:
            print(f"Found {len(self.existing_collections)} existing collections:")
            for coll_name in self.existing_collections:
                try:
                    temp_collection = self.client.get_collection(coll_name)
                    info = get_collection_info(temp_collection)
                    print(f"  • {coll_name}: {info['document_count']} documents")
                except:
                    print(f"  • {coll_name}: (info unavailable)")
        else:
            print("No existing collections found.")

        print("\n" + "=" * 50 + "\n")

        # Display widgets
        display(widgets.VBox([
            self.collection_mode,
            self.existing_dropdown,
            self.new_name_text,
            self.new_description_text,
            self.confirm_button,
            self.output
        ]))

# Create collection selector instance
selector = CollectionSelector(client)

print("Collection selector interface ready")

Collection selector interface ready


#### 4.3 Choose collection to Store Documents in ChromaDB

In [39]:
# Display Collection Selection Interface
# Run this cell to choose your target collection for storing processed documents
selector.display()

Collection Selection for Storage
Found 2 existing collections:
  • transcrips_barclasys: 5591 documents
  • pra_rules: 2991 documents




#### 4.4 Store Documents in ChromaDB

In [40]:
# Store enriched chunks in database
def store_enriched_chunks(
    collection: chromadb.Collection,
    chunks: List[Dict],
    metadata: Dict[str, Any],
) -> int:
    """Store chunks with embeddings, sentiment, and metadata.

    Ensure stable ids and numeric-only embeddings for ChromaDB compatibility.
    """

    def sanitize_meta_for_storage(meta: Optional[Dict[str, Any]]) -> Dict[str, Any]:
        """Flatten/sanitize metadata so all values are primitives or short strings."""
        if not meta:
            return {}
        out = {}
        for k, v in meta.items():
            # Primitive types pass through
            if isinstance(v, (str, int, float, bool)) or v is None:
                out[k] = v
                continue

            # Nested dict: try to pull out primitive children with prefixed keys
            if isinstance(v, dict):
                for kk, vv in v.items():
                    key = f"{k}_{kk}"
                    if isinstance(vv, (str, int, float, bool)) or vv is None:
                        out[key] = vv
                    else:
                        try:
                            s = json.dumps(vv, ensure_ascii=False)
                        except Exception:
                            s = str(vv)
                        out[key] = s[:1000]
                continue

            # Lists: special-case layout lists (common from PDF extraction)
            if isinstance(v, list):
                if k.lower() == 'layout' and len(v) > 0 and isinstance(v[0], dict):
                    out['layout_span_count'] = len(v)
                    try:
                        sample_texts = [span.get('text', '') for span in v[:3] if isinstance(span, dict)]
                        out['layout_sample_texts'] = ' | '.join(sample_texts)[:1000]
                    except Exception:
                        out['layout_sample_texts'] = str(v[:3])[:1000]
                else:
                    try:
                        out[k] = json.dumps(v, ensure_ascii=False)[:1000]
                    except Exception:
                        out[k] = str(v)[:1000]
                continue

            # Fallback for other complex types
            try:
                out[k] = json.dumps(v, ensure_ascii=False)[:1000]
            except Exception:
                out[k] = str(v)[:1000]

        return out

    # Prepare base sanitized metadata for this document
    base_meta = sanitize_meta_for_storage(metadata)
    # Use filename stem for stable id prefixes
    base_filename = str(base_meta.get('filename', 'unknown'))
    try:
        base_filename_stem = Path(base_filename).stem
    except Exception:
        base_filename_stem = base_filename

    # Collect chunk texts and sentiments
    chunk_texts = [chunk.get('text', '') for chunk in chunks]
    sentiments = [chunk.get('sentiment', {'sentiment': 'neutral', 'confidence': 0.0}) for chunk in chunks]

    # Build robust ids: if chunk_id already contains filename stem use it, else prefix
    ids = []
    for i, chunk in enumerate(chunks):
        raw_cid = chunk.get('chunk_id') or f'chunk_{i}'
        cid = str(raw_cid)
        if base_filename_stem and base_filename_stem in cid:
            ids.append(cid)
        else:
            ids.append(f"{base_filename_stem}_{cid}")

    # Helper to coerce embeddings into lists of floats
    def _to_float_list(e):
        if e is None:
            return None
        # numpy arrays
        if isinstance(e, np.ndarray):
            try:
                return [float(x) for x in e.tolist()]
            except Exception:
                return None
        # lists
        if isinstance(e, list):
            try:
                return [float(x) for x in e]
            except Exception:
                return None
        # objects with tolist
        if hasattr(e, 'tolist'):
            try:
                lst = e.tolist()
                return [float(x) for x in lst]
            except Exception:
                return None
        # try to iterate
        try:
            return [float(x) for x in list(e)]
        except Exception:
            return None

    # Extract embeddings and coerce to floats
    embeddings_list: List[Optional[List[float]]] = []
    for chunk in chunks:
        embedding = chunk.get('embedding')
        emb_list = _to_float_list(embedding)
        embeddings_list.append(emb_list)

    # Determine embedding dimension from first non-empty embedding
    emb_dim = None
    for e in embeddings_list:
        if isinstance(e, list) and len(e) > 0:
            emb_dim = len(e)
            break
    if emb_dim is None:
        emb_dim = 1024

    # Replace missing or invalid embeddings with zero vectors
    for i, e in enumerate(embeddings_list):
        if not isinstance(e, list):
            embeddings_list[i] = [0.0] * emb_dim

    # Build per-chunk metadata by copying base_meta and adding chunk-level fields
    metadatas = []
    for i, (chunk, sentiment) in enumerate(zip(chunks, sentiments)):
        chunk_meta = dict(base_meta)  # shallow copy

        # Add chunk-specific metadata with safe types
        chunk_meta.update({
            'chunk_id': str(chunk.get('chunk_id', f'chunk_{i}')),
            'sentiment': (sentiment.get('sentiment') if isinstance(sentiment, dict) else 'neutral'),
            'confidence': float(sentiment.get('confidence', 0.0)) if isinstance(sentiment, dict) else 0.0,
            'chunk_size': int(chunk.get('size', 0) or 0),
            'boundary_type': str(chunk.get('boundary_type', 'unknown')),
            'start_offset': int(chunk.get('offset', {}).get('start', 0) if isinstance(chunk.get('offset'), dict) else 0),
            'end_offset': int(chunk.get('offset', {}).get('end', 0) if isinstance(chunk.get('offset'), dict) else 0),
        })

        if 'filename' not in chunk_meta or not chunk_meta.get('filename'):
            chunk_meta['filename'] = base_meta.get('filename', 'unknown')

        chunk_meta = sanitize_meta_for_storage(chunk_meta)
        metadatas.append(chunk_meta)

    # Add to collection (ensure embeddings are numeric lists)
    collection.add(
        ids=ids,
        documents=chunk_texts,
        embeddings=embeddings_list,
        metadatas=metadatas
    )

    return len(chunks)

print(f"Storage function ready")

Storage function ready


#### 4.5 Collection status

In [41]:
# Collection Status Check Before Storage
def verify_collection_ready() -> bool:
    """Verify that a collection has been properly selected for storage"""
    global collection

    if collection is None:
        print("No collection selected!")
        print("Please run the collection selector interface above to choose or create a collection.")
        return False

    try:
        # Test collection access
        collection.count()
        info = get_collection_info(collection)
        print(f"Collection ready: '{collection.name}'")
        print(f"Current documents: {info['document_count']}")
        return True
    except Exception as e:
        print(f"Collection error: {e}")
        print("Please re-run the collection selector interface above.")
        return False

print("Collection verification function ready")

Collection verification function ready


#### 4.6 Store Documents in ChromaDB with Metadata and Sentiment

In [44]:
# Store all enriched chunks into the selected collection
print(len(all_chunk_texts))
if verify_collection_ready():
    print("Collection is ready. Starting document storage...")
    total_stored = 0
    # Iterate through each document in all_chunks and store its chunks
    for filename, doc_data in all_chunks.items():
        chunks_to_store = doc_data.get('chunks', [])
        metadata = doc_data.get('metadata', {})

        if chunks_to_store and collection:
            try:
                num_stored = store_enriched_chunks(collection, chunks_to_store, metadata)
                print(f"Stored {num_stored} chunks from '{Path(filename).name}'")
                total_stored += num_stored
            except Exception as e:
                print(f"Error storing chunks from '{Path(filename).name}': {e}")
        elif not chunks_to_store:
            print(f"No chunks found for '{Path(filename).name}', skipping storage.")
        elif not collection:
             print("Error: No collection available for storage.")


    print(f"\nDocument storage process complete. Total chunks stored: {total_stored}")

else:
    print("Collection not ready. Please select or create a collection using the interface above.")

0
No collection selected!
Please run the collection selector interface above to choose or create a collection.
Collection not ready. Please select or create a collection using the interface above.


## 5. Enhanced RAG System Implementation

#### 5.1 Enhanced Collections Manager with BM25 Indexing

In [45]:
# Enhanced Collections Manager with BM25 and Metadata Schema
# Check if CONFIG section has been run
required_vars = ['STORE_DIR', 'embedding_model', 'client']
missing_vars = [var for var in required_vars if var not in globals()]

if missing_vars:
    print(f"❌ Required variables not found: {missing_vars}")
    print("💡 Please run these cells first:")
    print("   - Cell 11: CONFIG and paths setup")
    print("   - Cell 16: ChromaDB client initialization")
    print("   - Cell 18: Models loading")
else:
    print("✅ All required variables available")

class EnhancedCollectionsManager:
    """Manages multiple ChromaDB collections with BM25 indexing and structured metadata"""

    def __init__(self, client, store_dir: Path):
        self.client = client
        self.store_dir = Path(store_dir)
        self.bm25_dir = self.store_dir / "bm25_indices"
        self.bm25_dir.mkdir(exist_ok=True)
        self.collections_cache = {}
        self.bm25_indices = {}

    def get_collection(self, collection_name: str):
        """Get collection with caching"""
        if collection_name not in self.collections_cache:
            try:
                self.collections_cache[collection_name] = self.client.get_collection(collection_name)
            except Exception as e:
                print(f"Collection {collection_name} not found: {e}")
                return None
        return self.collections_cache[collection_name]

    def get_available_collections(self) -> List[str]:
        """Get list of available collection names"""
        try:
            return [c.name for c in self.client.list_collections()]
        except:
            return []

    def build_bm25_index(self, collection_name: str, force_rebuild: bool = False):
        """Build BM25 index for collection with optimized parameters"""
        index_path = self.bm25_dir / f"{collection_name}_bm25.pkl"

        if index_path.exists() and not force_rebuild:
            try:
                with open(index_path, 'rb') as f:
                    self.bm25_indices[collection_name] = pickle.load(f)
                print(f"✅ Loaded existing BM25 index for {collection_name}")
                return
            except:
                pass

        collection = self.get_collection(collection_name)
        if not collection:
            return

        # Get all documents
        data = collection.get(include=['documents', 'metadatas'])
        documents = data.get('documents', [])

        if not documents:
            print(f"No documents found in {collection_name}")
            return

        # Tokenize documents for BM25 (financial domain optimized)
        tokenized_docs = []
        for doc in documents:
            # Financial-specific preprocessing
            doc_clean = re.sub(r'[^\w\s\.]', ' ', doc.lower())
            # Preserve financial terms and numbers
            tokens = [token for token in doc_clean.split() if len(token) > 1]
            tokenized_docs.append(tokens)

        # Create BM25 with financial domain parameters (k1=1.2, b=0.75)
        bm25 = BM25Okapi(tokenized_docs, k1=1.2, b=0.75)
        self.bm25_indices[collection_name] = bm25

        # Save index
        with open(index_path, 'wb') as f:
            pickle.dump(bm25, f)

        print(f"✅ Built BM25 index for {collection_name} ({len(documents)} docs)")

    def search_bm25(self, query: str, collection_name: str, top_k: int = 10) -> List[Dict]:
        """BM25 keyword search with financial optimization - Updated to use pre-built indices"""

        # Check if index is already loaded in memory
        if collection_name not in self.bm25_indices:
            # Try to load pre-built index first
            index_path = self.bm25_dir / f"{collection_name}_bm25.pkl"
            if index_path.exists():
                try:
                    with open(index_path, 'rb') as f:
                        self.bm25_indices[collection_name] = pickle.load(f)
                    print(f"✅ Loaded pre-built BM25 index for {collection_name}")
                except Exception as e:
                    print(f"⚠️ Failed to load pre-built index for {collection_name}: {e}")
                    # Fall back to building new index
                    self.build_bm25_index(collection_name)
            else:
                # No pre-built index exists, build new one
                print(f"📝 No pre-built index found for {collection_name}, building new one...")
                self.build_bm25_index(collection_name)

        # Get the BM25 index
        bm25 = self.bm25_indices.get(collection_name)
        if not bm25:
            print(f"❌ No BM25 index available for {collection_name}")
            return []

        # Tokenize query (using same financial optimization as build process)
        query_tokens = [token for token in re.sub(r'[^\w\s\.]', ' ', query.lower()).split() if len(token) > 1]

        if not query_tokens:
            return []

        # Get BM25 scores
        scores = bm25.get_scores(query_tokens)

        # Get top results
        top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]

        # Get documents and metadata from collection
        collection = self.get_collection(collection_name)
        if not collection:
            return []

        try:
            data = collection.get(include=['documents', 'metadatas'])
            documents = data.get('documents', [])
            metadatas = data.get('metadatas', [])

            results = []
            for idx in top_indices:
                if idx < len(documents) and scores[idx] > 0:
                    results.append({
                        'text': documents[idx],
                        'metadata': metadatas[idx] if idx < len(metadatas) else {},
                        'bm25_score': float(scores[idx]),
                        'collection': collection_name
                    })

            return results

        except Exception as e:
            print(f"Error retrieving documents for BM25 search in {collection_name}: {e}")
            return []

    def semantic_search(self, query: str, collection_name: str, top_k: int = 10) -> List[Dict]:
        """Semantic search using ChromaDB with proper embedding model"""
        collection = self.get_collection(collection_name)
        if not collection:
            return []

        try:
            # Use the global embedding model from CONFIG to ensure dimension consistency
            if 'embedding_model' in globals():
                query_embedding = embedding_model.encode([query])
                results = collection.query(
                    query_embeddings=query_embedding.tolist(),
                    n_results=top_k,
                    include=['documents', 'metadatas', 'distances']
                )
            else:
                # Fallback to query_texts (may cause dimension mismatch)
                results = collection.query(
                    query_texts=[query],
                    n_results=top_k,
                    include=['documents', 'metadatas', 'distances']
                )

            formatted_results = []
            if results['documents'] and results['documents'][0]:
                for i, doc in enumerate(results['documents'][0]):
                    formatted_results.append({
                        'text': doc,
                        'metadata': results['metadatas'][0][i] or {},
                        'semantic_score': 1.0 - results['distances'][0][i],  # Convert distance to similarity
                        'collection': collection_name
                    })

            return formatted_results

        except Exception as e:
            print(f"Error in semantic search: {e}")
            return []

# Initialize Enhanced Collections Manager
if missing_vars:
    print("❌ Cannot initialize collections manager - required variables missing")
else:
    try:
        collections_manager = EnhancedCollectionsManager(client, STORE_DIR)
        print("✅ Enhanced Collections Manager initialized")

        available_collections = collections_manager.get_available_collections()
        print(f"   Available collections: {available_collections}")

        if not available_collections:
            print("   ⚠️ No collections found - you may need to process documents first")

    except Exception as e:
        print(f"❌ Failed to initialize collections manager: {e}")
        print("💡 Make sure to run the CONFIG and ChromaDB setup cells first")

✅ All required variables available
✅ Enhanced Collections Manager initialized
   Available collections: ['transcrips_barclasys', 'pra_rules']


#### 5.2 BM25 Index building

##### 5.2.1 BM25 Index Building Function

In [46]:
# BM25 Index Pre-Building System for Startup Optimization
import time
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

class BM25IndexPreBuilder:
    """Centralized BM25 index pre-building system for startup optimization"""

    def __init__(self, collections_manager: EnhancedCollectionsManager):
        self.manager = collections_manager
        self.build_stats = {}

    def prebuild_all_indices(self, force_rebuild: bool = False, parallel: bool = True) -> Dict[str, Any]:
        """
        Pre-build BM25 indices for all available collections

        Args:
            force_rebuild: Force rebuild even if indices exist
            parallel: Build indices in parallel (faster)

        Returns:
            Build statistics and results
        """
        print("🚀 PRE-BUILDING BM25 INDICES FOR ALL COLLECTIONS")
        print("=" * 60)

        available_collections = self.manager.get_available_collections()

        if not available_collections:
            print("⚠️ No collections found to index")
            return {'status': 'no_collections', 'collections': []}

        print(f"📂 Found {len(available_collections)} collections: {available_collections}")

        start_time = time.time()

        if parallel and len(available_collections) > 1:
            results = self._build_indices_parallel(available_collections, force_rebuild)
        else:
            results = self._build_indices_sequential(available_collections, force_rebuild)

        total_time = time.time() - start_time

        # Calculate statistics
        successful_builds = sum(1 for r in results.values() if r.get('success', False))
        total_documents = sum(r.get('document_count', 0) for r in results.values())

        build_summary = {
            'status': 'completed',
            'total_collections': len(available_collections),
            'successful_builds': successful_builds,
            'failed_builds': len(available_collections) - successful_builds,
            'total_documents_indexed': total_documents,
            'total_build_time': total_time,
            'average_time_per_collection': total_time / len(available_collections),
            'parallel_processing': parallel,
            'force_rebuild': force_rebuild,
            'results': results
        }

        self._print_build_summary(build_summary)

        return build_summary

    def _build_indices_sequential(self, collections: List[str], force_rebuild: bool) -> Dict[str, Dict]:
        """Build indices sequentially"""
        results = {}

        for i, collection_name in enumerate(collections, 1):
            print(f"\n🔨 Building index {i}/{len(collections)}: {collection_name}")
            result = self._build_single_index(collection_name, force_rebuild)
            results[collection_name] = result

        return results

    def _build_indices_parallel(self, collections: List[str], force_rebuild: bool) -> Dict[str, Dict]:
        """Build indices in parallel using thread pool"""
        print("⚡ Using parallel processing for faster builds")

        results = {}
        max_workers = min(len(collections), 4)  # Limit to 4 threads

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all build tasks
            future_to_collection = {
                executor.submit(self._build_single_index, collection_name, force_rebuild): collection_name
                for collection_name in collections
            }

            # Collect results as they complete
            for future in tqdm(future_to_collection, desc="Building indices", unit="collection"):
                collection_name = future_to_collection[future]
                try:
                    result = future.result(timeout=300)  # 5 minute timeout per collection
                    results[collection_name] = result
                except Exception as e:
                    results[collection_name] = {
                        'success': False,
                        'error': str(e),
                        'document_count': 0,
                        'build_time': 0
                    }

        return results

    def _build_single_index(self, collection_name: str, force_rebuild: bool) -> Dict[str, Any]:
        """Build BM25 index for a single collection"""
        start_time = time.time()

        try:
            # Check if index already exists
            index_path = self.manager.bm25_dir / f"{collection_name}_bm25.pkl"

            if index_path.exists() and not force_rebuild:
                # Load existing index
                try:
                    with open(index_path, 'rb') as f:
                        bm25_index = pickle.load(f)
                    self.manager.bm25_indices[collection_name] = bm25_index

                    # Get document count from collection
                    collection = self.manager.get_collection(collection_name)
                    doc_count = collection.count() if collection else 0

                    build_time = time.time() - start_time

                    return {
                        'success': True,
                        'action': 'loaded_existing',
                        'document_count': doc_count,
                        'build_time': build_time,
                        'index_file': str(index_path)
                    }

                except Exception as e:
                    print(f"⚠️ Failed to load existing index for {collection_name}: {e}")
                    # Fall through to rebuild

            # Build new index
            collection = self.manager.get_collection(collection_name)
            if not collection:
                return {
                    'success': False,
                    'error': 'Collection not found',
                    'document_count': 0,
                    'build_time': time.time() - start_time
                }

            # Get documents
            data = collection.get(include=['documents'])
            documents = data.get('documents', [])

            if not documents:
                return {
                    'success': False,
                    'error': 'No documents found in collection',
                    'document_count': 0,
                    'build_time': time.time() - start_time
                }

            # Tokenize documents (using same logic as manager)
            tokenized_docs = []
            for doc in documents:
                doc_clean = re.sub(r'[^\w\s\.]', ' ', doc.lower())
                tokens = [token for token in doc_clean.split() if len(token) > 1]
                tokenized_docs.append(tokens)

            # Create BM25 index with financial optimization
            bm25 = BM25Okapi(tokenized_docs, k1=1.2, b=0.75)

            # Store in manager
            self.manager.bm25_indices[collection_name] = bm25

            # Save to disk
            self.manager.bm25_dir.mkdir(exist_ok=True)
            with open(index_path, 'wb') as f:
                pickle.dump(bm25, f)

            build_time = time.time() - start_time

            return {
                'success': True,
                'action': 'built_new',
                'document_count': len(documents),
                'build_time': build_time,
                'index_file': str(index_path),
                'tokenized_documents': len(tokenized_docs)
            }

        except Exception as e:
            return {
                'success': False,
                'error': str(e),
                'document_count': 0,
                'build_time': time.time() - start_time
            }

    def _print_build_summary(self, summary: Dict[str, Any]):
        """Print comprehensive build summary"""
        print("\n" + "=" * 60)
        print("📊 BM25 INDEX BUILD SUMMARY")
        print("=" * 60)

        print(f"✅ Successfully built: {summary['successful_builds']}/{summary['total_collections']} collections")

        if summary['failed_builds'] > 0:
            print(f"❌ Failed builds: {summary['failed_builds']}")

        print(f"📄 Total documents indexed: {summary['total_documents_indexed']:,}")
        print(f"⏱️ Total build time: {summary['total_build_time']:.1f} seconds")
        print(f"📈 Average time per collection: {summary['average_time_per_collection']:.1f} seconds")
        print(f"⚡ Parallel processing: {'Yes' if summary['parallel_processing'] else 'No'}")
        print(f"🔄 Force rebuild: {'Yes' if summary['force_rebuild'] else 'No'}")

        print(f"\n📋 DETAILED RESULTS:")
        for collection_name, result in summary['results'].items():
            status = "✅" if result['success'] else "❌"
            action = result.get('action', 'failed')
            doc_count = result.get('document_count', 0)
            build_time = result.get('build_time', 0)

            print(f"  {status} {collection_name}: {action} ({doc_count:,} docs, {build_time:.1f}s)")

            if not result['success'] and 'error' in result:
                print(f"      Error: {result['error']}")

        print("\n🎯 SYSTEM STATUS:")
        if summary['successful_builds'] == summary['total_collections']:
            print("🚀 All BM25 indices built successfully - System ready for optimal performance!")
        elif summary['successful_builds'] > 0:
            print("⚠️ Partial success - Some indices built, but system will work with degraded performance")
        else:
            print("❌ No indices built - System will fall back to semantic search only")

        print("=" * 60)

    def check_index_status(self) -> Dict[str, Any]:
        """Check status of all BM25 indices"""
        collections = self.manager.get_available_collections()
        status = {
            'total_collections': len(collections),
            'indexed_collections': 0,
            'missing_indices': [],
            'existing_indices': [],
            'index_files': []
        }

        for collection_name in collections:
            index_path = self.manager.bm25_dir / f"{collection_name}_bm25.pkl"

            if collection_name in self.manager.bm25_indices and index_path.exists():
                status['indexed_collections'] += 1
                status['existing_indices'].append(collection_name)
                status['index_files'].append(str(index_path))
            else:
                status['missing_indices'].append(collection_name)

        return status

# Initialize the pre-builder system
try:
    if 'collections_manager' in globals():
        bm25_prebuilder = BM25IndexPreBuilder(collections_manager)
        print("✅ BM25 Index Pre-Builder initialized!")
        print("   📁 Index storage directory:", collections_manager.bm25_dir)
        print("   🔧 Ready to build indices for all collections")
        print("\n💡 Usage:")
        print("   • bm25_prebuilder.prebuild_all_indices() - Build all indices")
        print("   • bm25_prebuilder.prebuild_all_indices(force_rebuild=True) - Force rebuild")
        print("   • bm25_prebuilder.check_index_status() - Check current status")
    else:
        print("❌ collections_manager not available - run Section 5.1 first")

except Exception as e:
    print(f"❌ BM25 Pre-Builder initialization failed: {e}")

✅ BM25 Index Pre-Builder initialized!
   📁 Index storage directory: /Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/chroma_store/bm25_indices
   🔧 Ready to build indices for all collections

💡 Usage:
   • bm25_prebuilder.prebuild_all_indices() - Build all indices
   • bm25_prebuilder.prebuild_all_indices(force_rebuild=True) - Force rebuild
   • bm25_prebuilder.check_index_status() - Check current status


##### 5.2.2 BM25 Index Version Check

- Verify all indices are healthy
status = index_status_manager.verify_all_indices_status()

- Fix any missing or corrupted indices
index_status_manager.fix_missing_indices(auto_fix=True)

- Test search performance
performance = index_status_manager.get_search_performance_metrics()

In [47]:
#### 5.2.1 Index Status Verification and Management System

class BM25IndexStatusManager:
    """Comprehensive BM25 index status verification and management system"""

    def __init__(self, collections_manager: EnhancedCollectionsManager, prebuilder: BM25IndexPreBuilder = None):
        self.manager = collections_manager
        self.prebuilder = prebuilder

    def verify_all_indices_status(self) -> Dict[str, Any]:
        """Comprehensive verification of all BM25 indices"""
        print("🔍 VERIFYING BM25 INDEX STATUS")
        print("=" * 50)

        available_collections = self.manager.get_available_collections()

        if not available_collections:
            print("⚠️ No collections found in the database")
            return {'status': 'no_collections', 'collections': []}

        verification_results = {
            'total_collections': len(available_collections),
            'verified_indices': 0,
            'missing_indices': 0,
            'corrupted_indices': 0,
            'healthy_indices': 0,
            'collection_details': {},
            'recommendations': [],
            'system_health': 'unknown'
        }

        print(f"📂 Found {len(available_collections)} collections to verify:")

        for collection_name in available_collections:
            print(f"\n🔍 Verifying: {collection_name}")
            collection_status = self._verify_single_collection_index(collection_name)
            verification_results['collection_details'][collection_name] = collection_status

            # Update counters
            if collection_status['status'] == 'healthy':
                verification_results['healthy_indices'] += 1
                verification_results['verified_indices'] += 1
                print(f"   ✅ Status: {collection_status['status']}")
            elif collection_status['status'] == 'missing':
                verification_results['missing_indices'] += 1
                verification_results['recommendations'].append(f"Build index for {collection_name}")
                print(f"   ❌ Status: {collection_status['status']}")
            elif collection_status['status'] == 'corrupted':
                verification_results['corrupted_indices'] += 1
                verification_results['recommendations'].append(f"Rebuild index for {collection_name}")
                print(f"   ⚠️ Status: {collection_status['status']}")

            # Display key metrics
            print(f"   📄 Documents: {collection_status['document_count']:,}")
            print(f"   📊 Index size: {collection_status['index_size_mb']:.1f} MB")
            print(f"   📅 Last modified: {collection_status['last_modified']}")

        # Calculate system health
        health_ratio = verification_results['healthy_indices'] / verification_results['total_collections']
        if health_ratio >= 0.9:
            verification_results['system_health'] = 'excellent'
        elif health_ratio >= 0.7:
            verification_results['system_health'] = 'good'
        elif health_ratio >= 0.5:
            verification_results['system_health'] = 'fair'
        else:
            verification_results['system_health'] = 'poor'

        self._print_verification_summary(verification_results)

        return verification_results

    def _verify_single_collection_index(self, collection_name: str) -> Dict[str, Any]:
        """Verify BM25 index for a single collection"""
        result = {
            'collection_name': collection_name,
            'status': 'unknown',
            'document_count': 0,
            'index_size_mb': 0,
            'last_modified': 'Unknown',
            'index_path': None,
            'memory_loaded': False,
            'can_search': False,
            'error': None
        }

        try:
            # Check collection exists and get document count
            collection = self.manager.get_collection(collection_name)
            if not collection:
                result['status'] = 'collection_missing'
                result['error'] = 'Collection not found in database'
                return result

            result['document_count'] = collection.count()

            # Check index file exists
            index_path = self.manager.bm25_dir / f"{collection_name}_bm25.pkl"
            result['index_path'] = str(index_path)

            if not index_path.exists():
                result['status'] = 'missing'
                return result

            # Check file size and modification time
            file_stat = index_path.stat()
            result['index_size_mb'] = file_stat.st_size / (1024 * 1024)
            result['last_modified'] = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(file_stat.st_mtime))

            # Check if index is loaded in memory
            result['memory_loaded'] = collection_name in self.manager.bm25_indices

            # Verify index integrity by loading it
            try:
                if not result['memory_loaded']:
                    with open(index_path, 'rb') as f:
                        bm25_index = pickle.load(f)
                else:
                    bm25_index = self.manager.bm25_indices[collection_name]

                # Test basic functionality
                test_query = ["test", "query"]
                scores = bm25_index.get_scores(test_query)

                if len(scores) == result['document_count']:
                    result['status'] = 'healthy'
                    result['can_search'] = True
                else:
                    result['status'] = 'corrupted'
                    result['error'] = f"Index size mismatch: {len(scores)} vs {result['document_count']} documents"

            except Exception as e:
                result['status'] = 'corrupted'
                result['error'] = f"Index corruption: {str(e)}"

        except Exception as e:
            result['status'] = 'error'
            result['error'] = f"Verification failed: {str(e)}"

        return result

    def _print_verification_summary(self, results: Dict[str, Any]):
        """Print comprehensive verification summary"""
        print("\n" + "=" * 50)
        print("📊 BM25 INDEX VERIFICATION SUMMARY")
        print("=" * 50)

        # Overall health
        health = results['system_health']
        health_icon = {
            'excellent': '🟢',
            'good': '🔵',
            'fair': '🟡',
            'poor': '🔴'
        }.get(health, '⚪')

        print(f"{health_icon} **System Health: {health.upper()}**")
        print(f"✅ Healthy indices: {results['healthy_indices']}/{results['total_collections']}")

        if results['missing_indices'] > 0:
            print(f"❌ Missing indices: {results['missing_indices']}")

        if results['corrupted_indices'] > 0:
            print(f"⚠️ Corrupted indices: {results['corrupted_indices']}")

        # Performance metrics
        total_docs = sum(details['document_count'] for details in results['collection_details'].values())
        total_size = sum(details['index_size_mb'] for details in results['collection_details'].values())

        print(f"\n📈 **Performance Metrics:**")
        print(f"   📄 Total documents indexed: {total_docs:,}")
        print(f"   💾 Total index size: {total_size:.1f} MB")
        print(f"   📊 Average index efficiency: {total_size/max(total_docs/1000, 1):.2f} MB per 1K docs")

        # Recommendations
        if results['recommendations']:
            print(f"\n💡 **Recommendations:**")
            for rec in results['recommendations']:
                print(f"   • {rec}")

        # Detailed status breakdown
        print(f"\n📋 **Detailed Status:**")
        for collection_name, details in results['collection_details'].items():
            status_icon = {
                'healthy': '✅',
                'missing': '❌',
                'corrupted': '⚠️',
                'error': '🔴'
            }.get(details['status'], '⚪')

            memory_status = "📲 Loaded" if details['memory_loaded'] else "📀 On Disk"

            print(f"   {status_icon} {collection_name}: {details['status']} | {details['document_count']:,} docs | {details['index_size_mb']:.1f}MB | {memory_status}")

            if details['error']:
                print(f"      ⚠️ {details['error']}")

        print("\n" + "=" * 50)

    def fix_missing_indices(self, auto_fix: bool = False) -> Dict[str, Any]:
        """Fix missing or corrupted indices"""
        print("🔧 FIXING MISSING/CORRUPTED INDICES")
        print("=" * 40)

        status = self.verify_all_indices_status()
        issues_found = []

        # Collect collections needing fixes
        for collection_name, details in status['collection_details'].items():
            if details['status'] in ['missing', 'corrupted']:
                issues_found.append({
                    'collection': collection_name,
                    'issue': details['status'],
                    'error': details.get('error', '')
                })

        if not issues_found:
            print("✅ No issues found - all indices are healthy!")
            return {'fixed': 0, 'issues': []}

        print(f"🔍 Found {len(issues_found)} indices requiring fixes:")
        for issue in issues_found:
            print(f"   • {issue['collection']}: {issue['issue']}")
            if issue['error']:
                print(f"     Error: {issue['error']}")

        if not auto_fix:
            print("\n💡 To fix these issues automatically, run:")
            print("   index_status_manager.fix_missing_indices(auto_fix=True)")
            return {'fixed': 0, 'issues': issues_found}

        # Auto-fix mode
        print(f"\n🔧 Auto-fixing {len(issues_found)} indices...")
        fixed_count = 0

        for issue in issues_found:
            collection_name = issue['collection']
            print(f"\n🔨 Fixing {collection_name}...")

            try:
                # Force rebuild the index
                self.manager.build_bm25_index(collection_name, force_rebuild=True)

                # Verify the fix
                verification = self._verify_single_collection_index(collection_name)
                if verification['status'] == 'healthy':
                    print(f"   ✅ Fixed successfully")
                    fixed_count += 1
                else:
                    print(f"   ❌ Fix failed: {verification.get('error', 'Unknown error')}")

            except Exception as e:
                print(f"   ❌ Fix failed with exception: {e}")

        print(f"\n📊 Fix Summary: {fixed_count}/{len(issues_found)} indices fixed")

        return {'fixed': fixed_count, 'issues': issues_found}

    def get_search_performance_metrics(self) -> Dict[str, Any]:
        """Get search performance metrics for all indices"""
        print("⚡ SEARCH PERFORMANCE ANALYSIS")
        print("=" * 40)

        collections = self.manager.get_available_collections()
        performance_data = {
            'collections': {},
            'overall_metrics': {},
            'recommendations': []
        }

        total_search_time = 0
        total_collections = 0

        for collection_name in collections:
            print(f"\n🧪 Testing {collection_name}...")

            # Test search performance
            perf_results = self._test_collection_performance(collection_name)
            performance_data['collections'][collection_name] = perf_results

            if perf_results['search_time'] > 0:
                total_search_time += perf_results['search_time']
                total_collections += 1

                # Performance assessment
                if perf_results['search_time'] > 1.0:
                    performance_data['recommendations'].append(
                        f"Consider optimizing {collection_name} - slow search time ({perf_results['search_time']:.2f}s)"
                    )

                print(f"   ⏱️ Search time: {perf_results['search_time']:.3f}s")
                print(f"   📊 Results found: {perf_results['results_count']}")

        # Calculate overall metrics
        if total_collections > 0:
            performance_data['overall_metrics'] = {
                'average_search_time': total_search_time / total_collections,
                'total_collections_tested': total_collections,
                'performance_grade': self._calculate_performance_grade(total_search_time / total_collections)
            }

        self._print_performance_summary(performance_data)

        return performance_data

    def _test_collection_performance(self, collection_name: str) -> Dict[str, Any]:
        """Test search performance for a single collection"""
        try:
            # Ensure index is loaded
            if collection_name not in self.manager.bm25_indices:
                self.manager.build_bm25_index(collection_name)

            # Test query
            test_query = "financial performance earnings"

            start_time = time.time()
            results = self.manager.search_bm25(test_query, collection_name, 10)
            search_time = time.time() - start_time

            return {
                'search_time': search_time,
                'results_count': len(results),
                'index_loaded': True,
                'status': 'success'
            }

        except Exception as e:
            return {
                'search_time': 0,
                'results_count': 0,
                'index_loaded': False,
                'status': 'error',
                'error': str(e)
            }

    def _calculate_performance_grade(self, avg_time: float) -> str:
        """Calculate performance grade based on average search time"""
        if avg_time < 0.1:
            return 'A+ (Excellent)'
        elif avg_time < 0.5:
            return 'A (Very Good)'
        elif avg_time < 1.0:
            return 'B (Good)'
        elif avg_time < 2.0:
            return 'C (Fair)'
        else:
            return 'D (Needs Improvement)'

    def _print_performance_summary(self, data: Dict[str, Any]):
        """Print performance analysis summary"""
        metrics = data.get('overall_metrics', {})

        if not metrics:
            print("❌ No performance data available")
            return

        print(f"\n📊 PERFORMANCE SUMMARY")
        print("=" * 30)
        print(f"⏱️ Average search time: {metrics['average_search_time']:.3f}s")
        print(f"🎯 Performance grade: {metrics['performance_grade']}")
        print(f"📈 Collections tested: {metrics['total_collections_tested']}")

        if data['recommendations']:
            print(f"\n💡 Recommendations:")
            for rec in data['recommendations']:
                print(f"   • {rec}")

# Initialize the Index Status Manager
try:
    if 'collections_manager' in globals() and 'bm25_prebuilder' in globals():
        index_status_manager = BM25IndexStatusManager(
            collections_manager=collections_manager,
            prebuilder=bm25_prebuilder
        )
        print("✅ BM25 Index Status Manager initialized!")
        print("   🔍 Comprehensive index verification")
        print("   🔧 Automatic issue detection and fixing")
        print("   ⚡ Search performance analysis")
        print("   📊 Health monitoring and recommendations")
        print("\n💡 Usage:")
        print("   • index_status_manager.verify_all_indices_status() - Check all indices")
        print("   • index_status_manager.fix_missing_indices(auto_fix=True) - Fix issues")
        print("   • index_status_manager.get_search_performance_metrics() - Performance test")

    else:
        print("❌ Required components not available (collections_manager, bm25_prebuilder)")
        print("   Please run Sections 5.1 and 5.2 first")

except Exception as e:
    print(f"❌ Index Status Manager initialization failed: {e}")


# Checking status
index_status_manager.verify_all_indices_status()

✅ BM25 Index Status Manager initialized!
   🔍 Comprehensive index verification
   🔧 Automatic issue detection and fixing
   ⚡ Search performance analysis
   📊 Health monitoring and recommendations

💡 Usage:
   • index_status_manager.verify_all_indices_status() - Check all indices
   • index_status_manager.fix_missing_indices(auto_fix=True) - Fix issues
   • index_status_manager.get_search_performance_metrics() - Performance test
🔍 VERIFYING BM25 INDEX STATUS
📂 Found 2 collections to verify:

🔍 Verifying: transcrips_barclasys
   ✅ Status: healthy
   📄 Documents: 5,591
   📊 Index size: 3.7 MB
   📅 Last modified: 2025-11-18 17:06:08

🔍 Verifying: pra_rules
   ✅ Status: healthy
   📄 Documents: 2,991
   📊 Index size: 1.6 MB
   📅 Last modified: 2025-11-18 17:06:07

📊 BM25 INDEX VERIFICATION SUMMARY
🟢 **System Health: EXCELLENT**
✅ Healthy indices: 2/2

📈 **Performance Metrics:**
   📄 Total documents indexed: 8,582
   💾 Total index size: 5.3 MB
   📊 Average index efficiency: 0.62 MB per 1K do

{'total_collections': 2,
 'verified_indices': 2,
 'missing_indices': 0,
 'corrupted_indices': 0,
 'healthy_indices': 2,
 'collection_details': {'transcrips_barclasys': {'collection_name': 'transcrips_barclasys',
   'status': 'healthy',
   'document_count': 5591,
   'index_size_mb': 3.701578140258789,
   'last_modified': '2025-11-18 17:06:08',
   'index_path': '/Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/chroma_store/bm25_indices/transcrips_barclasys_bm25.pkl',
   'memory_loaded': False,
   'can_search': True,
   'error': None},
  'pra_rules': {'collection_name': 'pra_rules',
   'status': 'healthy',
   'document_count': 2991,
   'index_size_mb': 1.633096694946289,
   'last_modified': '2025-11-18 17:06:07',
   'index_path': '/Users/rodrigograndy/Desktop/coding_projects/posts_website_drafts/rag_boe/chroma_store/bm25_indices/pra_rules_bm25.pkl',
   'memory_loaded': False,
   'can_search': True,
   'error': None}},
 'recommendations': [],
 'system_health': 'exc

#### 5.3 Execute BM25 Index building

In [48]:
# Execute BM25 pre-building for all collections
if 'bm25_prebuilder' in globals():
    print("🚀 STARTING BM25 INDEX PRE-BUILDING PROCESS")
    print("This may take a few minutes depending on your data size...")

    # Check current status first
    current_status = bm25_prebuilder.check_index_status()
    print(f"\n📊 Current Status: {current_status['indexed_collections']}/{current_status['total_collections']} collections indexed")

    if current_status['missing_indices']:
        print(f"🔨 Missing indices for: {current_status['missing_indices']}")

        # Build all indices (parallel processing for speed)
        build_results = bm25_prebuilder.prebuild_all_indices(
            force_rebuild=False,  # Set to True to force rebuild existing indices
            parallel=True         # Use parallel processing for speed
        )

        # The summary is automatically printed by the method

    else:
        print("✅ All indices already exist!")
        print("💡 Use force_rebuild=True to rebuild existing indices")

else:
    print("❌ bm25_prebuilder not available - run the previous cell first")

🚀 STARTING BM25 INDEX PRE-BUILDING PROCESS
This may take a few minutes depending on your data size...

📊 Current Status: 0/2 collections indexed
🔨 Missing indices for: ['transcrips_barclasys', 'pra_rules']
🚀 PRE-BUILDING BM25 INDICES FOR ALL COLLECTIONS
📂 Found 2 collections: ['transcrips_barclasys', 'pra_rules']
⚡ Using parallel processing for faster builds


Building indices:   0%|          | 0/2 [00:00<?, ?collection/s]


📊 BM25 INDEX BUILD SUMMARY
✅ Successfully built: 2/2 collections
📄 Total documents indexed: 8,582
⏱️ Total build time: 0.1 seconds
📈 Average time per collection: 0.1 seconds
⚡ Parallel processing: Yes
🔄 Force rebuild: No

📋 DETAILED RESULTS:
  ✅ transcrips_barclasys: loaded_existing (5,591 docs, 0.1s)
  ✅ pra_rules: loaded_existing (2,991 docs, 0.1s)

🎯 SYSTEM STATUS:
🚀 All BM25 indices built successfully - System ready for optimal performance!


#### 5.4 Hybrid Search Engine with RRF

In [49]:
class AdvancedHybridSearchEngine:
    """Production-grade hybrid search with enhanced RRF and score normalization - FIXED VERSION"""

    def __init__(self, collections_manager):
        self.manager = collections_manager
        self.rrf_params = {
            'k': 60,
            'semantic_weight': 0.6,
            'bm25_weight': 0.4,
            'cross_boost': 1.15,
            'decay_factor': 0.95
        }

    def search_bm25_fixed(self, query: str, collection_name: str, top_k: int = 10) -> List[Dict]:
        """FIXED BM25 search with proper field mapping"""
        try:
            if collection_name not in self.manager.bm25_indices:
                self.manager.build_bm25_index(collection_name)

            bm25 = self.manager.bm25_indices.get(collection_name)
            if not bm25:
                return []

            query_tokens = [token for token in re.sub(r'[^\w\s\.]', ' ', query.lower()).split() if len(token) > 1]
            if not query_tokens:
                return []

            scores = bm25.get_scores(query_tokens)
            top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]

            collection = self.manager.get_collection(collection_name)
            if not collection:
                return []

            # Get documents and metadatas
            data = collection.get(include=['documents', 'metadatas'])
            documents = data.get('documents', [])
            metadatas = data.get('metadatas', [])

            results = []
            for idx in top_indices:
                if idx < len(documents) and scores[idx] > 0:
                    doc_id = f"doc_{idx}"
                    if idx < len(metadatas) and metadatas[idx]:
                        chunk_id = metadatas[idx].get('chunk_id', f'chunk_{idx}')
                        doc_id = chunk_id

                    results.append({
                        'text': documents[idx],  # FIXED: Map documents to text
                        'metadata': metadatas[idx] if idx < len(metadatas) else {},
                        'id': doc_id,
                        'bm25_score': float(scores[idx]),
                        'collection': collection_name,
                        'search_method': 'bm25'
                    })

            return results

        except Exception as e:
            print(f"BM25 search error for {collection_name}: {e}")
            return []

    def semantic_search_fixed(self, query: str, collection_name: str, top_k: int = 10) -> List[Dict]:
        """FIXED semantic search with proper field mapping"""
        collection = self.manager.get_collection(collection_name)
        if not collection:
            return []

        try:
            if 'embedding_model' in globals():
                query_embedding = embedding_model.encode([query])
                results = collection.query(
                    query_embeddings=query_embedding.tolist(),
                    n_results=top_k,
                    include=['documents', 'metadatas', 'distances']
                )
            else:
                results = collection.query(
                    query_texts=[query],
                    n_results=top_k,
                    include=['documents', 'metadatas', 'distances']
                )

            formatted_results = []
            if results.get('documents') and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results.get('metadatas', [[]])[0]
                distances = results.get('distances', [[]])[0]

                for i, doc in enumerate(documents):
                    doc_id = f"doc_{i}"
                    if i < len(metadatas) and metadatas[i]:
                        chunk_id = metadatas[i].get('chunk_id', f'chunk_{i}')
                        doc_id = chunk_id

                    formatted_results.append({
                        'text': doc,  # FIXED: Direct mapping to text field
                        'metadata': metadatas[i] if i < len(metadatas) else {},
                        'id': doc_id,
                        'semantic_score': 1.0 - distances[i] if i < len(distances) else 0.0,
                        'distance': distances[i] if i < len(distances) else 1.0,
                        'collection': collection_name,
                        'search_method': 'semantic'
                    })

            return formatted_results

        except Exception as e:
            print(f"Semantic search error for {collection_name}: {e}")
            return []

    # ... rest of the class methods remain the same ...

    def enhanced_reciprocal_rank_fusion(self, semantic_results: List[Dict], bm25_results: List[Dict]) -> List[Dict]:
        """Enhanced RRF with validation"""
        # Validate that all results have 'text' field
        semantic_results = [r for r in semantic_results if 'text' in r and r['text']]
        bm25_results = [r for r in bm25_results if 'text' in r and r['text']]

        # Continue with existing RRF logic...
        k = self.rrf_params['k']
        semantic_weight = self.rrf_params['semantic_weight']
        bm25_weight = self.rrf_params['bm25_weight']
        cross_boost = self.rrf_params['cross_boost']
        decay_factor = self.rrf_params['decay_factor']

        # Normalize scores first
        semantic_results = self.enhanced_score_normalization(semantic_results, 'semantic')
        bm25_results = self.enhanced_score_normalization(bm25_results, 'bm25')

        combined_scores = {}

        # Process semantic results
        for rank, result in enumerate(semantic_results):
            doc_id = self._get_document_id(result)

            decay_multiplier = decay_factor ** rank
            base_rrf = semantic_weight / (rank + 1 + k)
            rrf_score = base_rrf * decay_multiplier

            semantic_score_norm = result.get('semantic_score_normalized', 0)
            quality_bonus = semantic_score_norm * 0.1

            combined_scores[doc_id] = {
                'document': result,
                'semantic_score': result.get('semantic_score', 0),
                'semantic_score_norm': semantic_score_norm,
                'bm25_score': 0,
                'bm25_score_norm': 0,
                'semantic_rank': rank + 1,
                'bm25_rank': None,
                'rrf_score': rrf_score + quality_bonus,
                'quality_bonus': quality_bonus,
                'in_both_sets': False
            }

        # Process BM25 results
        for rank, result in enumerate(bm25_results):
            doc_id = self._get_document_id(result)

            decay_multiplier = decay_factor ** rank
            base_rrf = bm25_weight / (rank + 1 + k)
            rrf_score = base_rrf * decay_multiplier

            bm25_score_norm = result.get('bm25_score_normalized', 0)
            quality_bonus = bm25_score_norm * 0.05

            if doc_id in combined_scores:
                combined_scores[doc_id]['bm25_score'] = result.get('bm25_score', 0)
                combined_scores[doc_id]['bm25_score_norm'] = bm25_score_norm
                combined_scores[doc_id]['bm25_rank'] = rank + 1
                combined_scores[doc_id]['rrf_score'] += rrf_score + quality_bonus
                combined_scores[doc_id]['rrf_score'] *= cross_boost
                combined_scores[doc_id]['in_both_sets'] = True
                combined_scores[doc_id]['quality_bonus'] += quality_bonus
            else:
                combined_scores[doc_id] = {
                    'document': result,
                    'semantic_score': 0,
                    'semantic_score_norm': 0,
                    'bm25_score': result.get('bm25_score', 0),
                    'bm25_score_norm': bm25_score_norm,
                    'semantic_rank': None,
                    'bm25_rank': rank + 1,
                    'rrf_score': rrf_score + quality_bonus,
                    'quality_bonus': quality_bonus,
                    'in_both_sets': False
                }

        # Create final results with validation
        fused_results = []
        for doc_id, score_data in combined_scores.items():
            result = score_data['document'].copy()

            # CRITICAL: Ensure text field exists and is valid
            if 'text' not in result or not result['text']:
                print(f"Warning: Skipping result with missing/empty text field")
                continue

            base_score = score_data['rrf_score']
            diversity_bonus = 0.02 if score_data['in_both_sets'] else 0

            confidence_factors = []
            if score_data['semantic_score_norm'] > 0:
                confidence_factors.append(score_data['semantic_score_norm'])
            if score_data['bm25_score_norm'] > 0:
                confidence_factors.append(score_data['bm25_score_norm'])

            confidence = np.mean(confidence_factors) if confidence_factors else 0.5
            final_score = base_score + diversity_bonus + (confidence * 0.05)

            result.update({
                'final_score': final_score,
                'rrf_score': score_data['rrf_score'],
                'confidence': confidence,
                'semantic_score_normalized': score_data['semantic_score_norm'],
                'bm25_score_normalized': score_data['bm25_score_norm'],
                'semantic_rank': score_data['semantic_rank'],
                'bm25_rank': score_data['bm25_rank'],
                'in_both_sets': score_data['in_both_sets'],
                'quality_bonus': score_data['quality_bonus'],
                'diversity_bonus': diversity_bonus,
                'fusion_method': 'Enhanced_RRF_Fixed'
            })

            fused_results.append(result)

        # Sort by final score
        fused_results.sort(key=lambda x: x.get('final_score', 0), reverse=True)
        return fused_results

    def enhanced_score_normalization(self, results: List[Dict], score_type: str) -> List[Dict]:
        """Advanced score normalization with outlier handling"""
        if not results:
            return results

        scores = [r.get(f'{score_type}_score', 0) for r in results]
        if not scores or all(s == 0 for s in scores):
            return results

        scores_array = np.array(scores)

        if score_type == 'bm25':
            clip_value = np.percentile(scores_array, 95)
            scores_clipped = np.clip(scores_array, 0, clip_value)

            if clip_value > 0:
                normalized_scores = scores_clipped / clip_value
            else:
                normalized_scores = scores_array * 0

        elif score_type == 'semantic':
            mean_score = np.mean(scores_array)
            std_score = np.std(scores_array) + 1e-8

            z_scores = (scores_array - mean_score) / std_score
            normalized_scores = 1 / (1 + np.exp(-z_scores * 2))
        else:
            min_score, max_score = np.min(scores_array), np.max(scores_array)
            if max_score > min_score:
                normalized_scores = (scores_array - min_score) / (max_score - min_score)
            else:
                normalized_scores = scores_array * 0

        for i, result in enumerate(results):
            result[f'{score_type}_score_normalized'] = float(normalized_scores[i])
            result[f'{score_type}_score_raw'] = scores[i]

        return results

    def _get_document_id(self, result: Dict) -> str:
        """Generate consistent document ID for deduplication"""
        if 'id' in result:
            return result['id']
        elif 'metadata' in result and result['metadata'].get('chunk_id'):
            return result['metadata']['chunk_id']
        else:
            text_part = result.get('text', '')[:100]
            filename = result.get('metadata', {}).get('filename', 'unknown')
            combined = f"{filename}_{text_part}"
            return f"doc_{hash(combined)}"

    def hybrid_search(self, query: str, collection_name: str, top_k: int = 20, **kwargs) -> List[Dict]:
        """FIXED hybrid search"""
        try:
            self.ensure_bm25_indices_loaded([collection_name])

            expand_factor = 2.5
            expanded_k = int(top_k * expand_factor)

            bm25_results = self.search_bm25_fixed(query, collection_name, expanded_k)
            semantic_results = self.semantic_search_fixed(query, collection_name, expanded_k)

            if not bm25_results and not semantic_results:
                return []

            fused_results = self.enhanced_reciprocal_rank_fusion(semantic_results, bm25_results)
            return fused_results[:top_k]

        except Exception as e:
            print(f"Hybrid search error for {collection_name}: {e}")
            return []

    def multi_collection_search(self, query: str, collection_names: List[str], top_k_per_collection: int = 10) -> List[Dict]:
        """FIXED multi-collection search"""
        self.ensure_bm25_indices_loaded(collection_names)

        all_results = []

        for collection_name in collection_names:
            try:
                results = self.hybrid_search(query, collection_name, top_k_per_collection)

                valid_results = []
                for result in results:
                    if 'text' in result and result['text']:  # Validate text field
                        result['source_collection'] = collection_name
                        result['final_score'] *= 0.98
                        valid_results.append(result)

                all_results.extend(valid_results)

            except Exception as e:
                print(f"Error in collection {collection_name}: {e}")
                continue

        if len(collection_names) > 1:
            all_results = self._promote_collection_diversity(all_results, collection_names)

        all_results.sort(key=lambda x: x.get('final_score', 0), reverse=True)
        return all_results[:top_k_per_collection * len(collection_names)]

    def _promote_collection_diversity(self, results: List[Dict], collection_names: List[str]) -> List[Dict]:
        """Promote diversity across collections in results"""
        if len(collection_names) <= 1:
            return results

        collection_counts = {name: 0 for name in collection_names}
        adjusted_results = []

        for result in results:
            collection = result.get('source_collection', 'unknown')
            if collection in collection_counts:
                current_count = collection_counts[collection]
                target_per_collection = len(results) / len(collection_names)

                if current_count < target_per_collection:
                    diversity_boost = 0.05
                else:
                    diversity_boost = -0.02

                result['final_score'] = result.get('final_score', 0) + diversity_boost
                collection_counts[collection] += 1

            adjusted_results.append(result)

        return adjusted_results

    def ensure_bm25_indices_loaded(self, collection_names: List[str]) -> bool:
        """Ensure BM25 indices are loaded for all collections"""
        missing_indices = []

        for collection_name in collection_names:
            if collection_name not in self.manager.bm25_indices:
                index_path = self.manager.bm25_dir / f"{collection_name}_bm25.pkl"
                if index_path.exists():
                    try:
                        with open(index_path, 'rb') as f:
                            self.manager.bm25_indices[collection_name] = pickle.load(f)
                        print(f"📂 Loaded pre-built BM25 index for {collection_name}")
                    except Exception as e:
                        print(f"⚠️ Failed to load pre-built index for {collection_name}: {e}")
                        missing_indices.append(collection_name)
                else:
                    missing_indices.append(collection_name)

        if missing_indices:
            print(f"🔨 Building missing BM25 indices for: {missing_indices}")
            for collection_name in missing_indices:
                self.manager.build_bm25_index(collection_name)

        return len(missing_indices) == 0

# Replace the existing hybrid search engine
print("🔧 APPLYING COMPREHENSIVE FIX FOR TEXT FIELD MAPPING")
print("=" * 60)

try:
    enhanced_hybrid_search = AdvancedHybridSearchEngine(collections_manager)

    if 'rag_orchestrator' in globals():
        rag_orchestrator.hybrid_search = enhanced_hybrid_search
        print("✅ Updated RAG Orchestrator")

    if 'spec_rag' in globals():
        spec_rag.hybrid_search = enhanced_hybrid_search
        print("✅ Updated Spec RAG System")

    print("\n✅ COMPREHENSIVE FIX APPLIED!")
    print("   • Fixed field mapping: documents → text")
    print("   • Added result validation throughout pipeline")
    print("   • Enhanced error handling for missing fields")
    print("   • Maintained all RRF and scoring functionality")

except Exception as e:
    print(f"❌ Fix failed: {e}")
    import traceback
    traceback.print_exc()

🔧 APPLYING COMPREHENSIVE FIX FOR TEXT FIELD MAPPING

✅ COMPREHENSIVE FIX APPLIED!
   • Fixed field mapping: documents → text
   • Added result validation throughout pipeline
   • Enhanced error handling for missing fields
   • Maintained all RRF and scoring functionality


#### 5.5 Cross-Encoder Reranking (Optional)

In [50]:
# Cross-Encoder Reranking (lightweight implementation)
try:
    from sentence_transformers import CrossEncoder

    class CrossEncoderReranker:
        """Optional cross-encoder reranking for improved relevance"""

        def __init__(self, model_name: str = "cross-encoder/ms-marco-MiniLM-L-6-v2"):
            try:
                self.model = CrossEncoder(model_name)
                self.available = True
                print(f"✅ Cross-encoder loaded: {model_name}")
            except Exception as e:
                print(f"⚠️ Cross-encoder not available: {e}")
                self.available = False

        def rerank(self, query: str, results: List[Dict], top_k: int = 10) -> List[Dict]:
            """Rerank results using cross-encoder"""
            if not self.available or not results:
                return results[:top_k]

            try:
                # Prepare query-document pairs
                pairs = [(query, result['text'][:500]) for result in results]  # Limit text length

                # Get scores
                scores = self.model.predict(pairs)

                # Add rerank scores to results
                for i, result in enumerate(results):
                    result['rerank_score'] = float(scores[i])

                # Sort by rerank score
                reranked = sorted(results, key=lambda x: x['rerank_score'], reverse=True)
                return reranked[:top_k]

            except Exception as e:
                print(f"Reranking failed: {e}")
                return results[:top_k]

    # Try to initialize cross-encoder (optional)
    try:
        cross_encoder = CrossEncoderReranker()
    except:
        cross_encoder = None
        print("Cross-encoder reranking disabled (optional feature)")

except ImportError:
    print("Cross-encoder reranking not available (sentence-transformers required)")
    cross_encoder = None

✅ Cross-encoder loaded: cross-encoder/ms-marco-MiniLM-L-6-v2


## 6. Intelligent Query Processing and LLM Engine

#### 6.1 Ollama Configuration and Query Routing

In [51]:
# Enhanced LLM Interface with Query Routing
import subprocess
import requests
import json
import re
from typing import Dict, Any

class EnhancedLLMEngine:
    """Intelligent query routing and LLM processing for financial documents"""

    def __init__(self, model_name: str = "llama3.1:8b"):
        self.model_name = model_name
        self.base_url = "http://localhost:11434"
        self.available = self._check_ollama_availability()

        # Query classification patterns
        self.query_patterns = {
            'document_summary': ['summarize', 'summary', 'overview', 'review'],
            'compliance_analysis': ['compliance', 'regulatory', 'pra', 'rules', 'violation'],
            'comparison': ['compare', 'difference', 'versus', 'vs', 'contrast'],
            'specific_search': ['what', 'how', 'when', 'where', 'who'],
            'file_analysis': ['.pdf', 'document', 'file', 'transcript']
        }

    def _check_ollama_availability(self) -> bool:
        """Check if Ollama is available and model is installed"""
        try:
            # Check if Ollama is running
            response = requests.get(f"{self.base_url}/api/tags", timeout=5)
            if response.status_code == 200:
                models = response.json().get('models', [])
                model_names = [m['name'] for m in models]
                if self.model_name in model_names:
                    print(f"✅ Ollama available with {self.model_name}")
                    return True
                else:
                    print(f"⚠️ Model {self.model_name} not found. Available: {model_names}")
                    return False
            return False
        except:
            print("⚠️ Ollama not available - using fallback mode")
            return False

    def setup_ollama_colab(self):
        """Setup Ollama in Colab environment"""
        if self.available:
            return True

        try:
            print("🔄 Installing Ollama in Colab...")

            # Install Ollama
            subprocess.run(["curl", "-fsSL", "https://ollama.com/install.sh"],
                          shell=True, check=True, capture_output=True)

            # Start Ollama service
            subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

            # Wait a moment for service to start
            import time
            time.sleep(5)

            # Pull model
            subprocess.run(["ollama", "pull", self.model_name], check=True)

            self.available = True
            print(f"✅ Ollama setup complete with {self.model_name}")
            return True

        except Exception as e:
            print(f"❌ Ollama setup failed: {e}")
            print("Using fallback response generation")
            return False

    def classify_query(self, query: str) -> Dict[str, Any]:
        """Classify query type and extract parameters"""
        query_lower = query.lower()

        # Extract filename if present
        filename = None
        filename_match = re.search(r'([a-zA-Z0-9_-]+\.pdf)', query)
        if filename_match:
            filename = filename_match.group(1)

        # Extract word count
        word_count = None
        word_match = re.search(r'(\d+)\s*words?', query_lower)
        if word_match:
            word_count = int(word_match.group(1))

        # Classify query type
        query_type = 'specific_search'  # default
        for qtype, patterns in self.query_patterns.items():
            if any(pattern in query_lower for pattern in patterns):
                query_type = qtype
                break

        return {
            'query_type': query_type,
            'filename': filename,
            'word_count': word_count,
            'is_comparative': any(word in query_lower for word in ['compare', 'versus', 'vs', 'difference']),
            'requires_compliance': any(word in query_lower for word in ['compliance', 'regulatory', 'pra'])
        }

    def generate_response(self, prompt: str, temperature: float = 0.1) -> str:
        """Generate LLM response with fallback"""
        if not self.available:
            return "LLM not available. Please ensure Ollama is running with llama3.1:8b model."

        try:
            payload = {
                "model": self.model_name,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": temperature,
                    "num_predict": 4000,  # Max tokens
                    "top_p": 0.9,
                    "repeat_penalty": 1.1
                }
            }

            response = requests.post(f"{self.base_url}/api/generate",
                                   json=payload, timeout=60)

            if response.status_code == 200:
                return response.json().get('response', 'No response generated')
            else:
                return f"Error: HTTP {response.status_code}"

        except Exception as e:
            return f"Error generating response: {str(e)}"

# Initialize enhanced LLM engine
llm_engine = EnhancedLLMEngine()

# Auto-setup in Colab if needed
try:
    import google.colab
    if not llm_engine.available:
        print("🔄 Setting up Ollama for Colab...")
        llm_engine.setup_ollama_colab()
except ImportError:
    # Not in Colab - assume local setup
    pass

print("✅ Enhanced LLM Engine initialized")
print(f"   Model: {llm_engine.model_name}")
print(f"   Available: {llm_engine.available}")

✅ Ollama available with llama3.1:8b
✅ Enhanced LLM Engine initialized
   Model: llama3.1:8b
   Available: True


#### 6.2 Document Finder and Reference Extraction

In [52]:
# Document Finder and Reference Extraction System
class DocumentFinder:
    """Finds documents and extracts precise references with metadata"""

    def __init__(self, collections_manager: EnhancedCollectionsManager):
        self.manager = collections_manager

    def find_documents_by_name(self, filename: str, collection_names: List[str] = None) -> List[Dict]:
        """Find documents by filename across collections"""
        if not collection_names:
            collection_names = self.manager.get_available_collections()

        found_documents = []

        for collection_name in collection_names:
            collection = self.manager.get_collection(collection_name)
            if not collection:
                continue

            try:
                # Get all documents with metadata (ids are returned by default)
                data = collection.get(include=['documents', 'metadatas'])

                for i, metadata in enumerate(data.get('metadatas', [])):
                    if not metadata:
                        continue

                    doc_filename = metadata.get('filename', '')
                    if filename.lower() in doc_filename.lower():
                        found_documents.append({
                            'text': data['documents'][i],
                            'metadata': {**metadata, 'collection': collection_name},
                            'id': data.get('ids', [])[i] if data.get('ids') else f"doc_{i}",
                            'collection': collection_name
                        })

            except Exception as e:
                print(f"Error searching {collection_name}: {e}")
                continue

        return found_documents

    def get_all_filenames(self, collection_names: List[str] = None) -> Dict[str, List[str]]:
        """Get all available filenames by collection"""
        if not collection_names:
            collection_names = self.manager.get_available_collections()

        files_by_collection = {}

        for collection_name in collection_names:
            collection = self.manager.get_collection(collection_name)
            if not collection:
                continue

            try:
                data = collection.get(include=['metadatas'])
                filenames = set()

                for metadata in data.get('metadatas', []):
                    if metadata and metadata.get('filename'):
                        filenames.add(metadata['filename'])

                files_by_collection[collection_name] = sorted(list(filenames))

            except Exception as e:
                files_by_collection[collection_name] = []

        return files_by_collection

    def extract_references(self, search_results: List[Dict]) -> List[Dict]:
        """Extract structured references from search results"""
        references = []

        for result in search_results:
            metadata = result.get('metadata', {})

            reference = {
                'filename': metadata.get('filename', 'Unknown'),
                'collection': result.get('collection', metadata.get('collection', 'Unknown')),
                'page': metadata.get('page_num'),
                'chunk_id': metadata.get('chunk_id'),
                'text_snippet': result['text'][:200] + '...' if len(result['text']) > 200 else result['text'],
                'full_text': result['text']  # For processing
            }

            # Add scoring information if available
            if 'final_score' in result:
                reference['relevance_score'] = result['final_score']
            elif 'semantic_score' in result:
                reference['relevance_score'] = result['semantic_score']

            references.append(reference)

        return references

# Initialize document finder
doc_finder = DocumentFinder(collections_manager)
print("✅ Document Finder initialized")
print("   - Filename-based document search")
print("   - Structured reference extraction")
print("   - Multi-collection support")

✅ Document Finder initialized
   - Filename-based document search
   - Structured reference extraction
   - Multi-collection support


#### 6.3 Enhanced LLM Query Expansion

In [53]:
# Enhanced LLM Query Expansion with Financial Domain Optimization (Consolidated)

class AdvancedFinancialQueryExpansion:
    """Advanced query expansion specialized for financial documents - Complete Solution"""

    def __init__(self, llm_engine: EnhancedLLMEngine):
        self.llm_engine = llm_engine

        # Financial domain synonyms and related terms (expanded)
        self.financial_synonyms = {
            'revenue': ['income', 'sales', 'turnover', 'earnings', 'receipts', 'proceeds'],
            'profit': ['earnings', 'income', 'gains', 'surplus', 'return', 'yield'],
            'loss': ['deficit', 'shortfall', 'negative earnings', 'decline', 'reduction'],
            'compliance': ['regulatory', 'rules', 'requirements', 'standards', 'governance', 'adherence'],
            'risk': ['exposure', 'threat', 'vulnerability', 'hazard', 'uncertainty', 'liability'],
            'capital': ['funding', 'investment', 'financing', 'assets', 'equity', 'resources'],
            'cash flow': ['liquidity', 'working capital', 'cash generation', 'funds flow'],
            'debt': ['liability', 'borrowing', 'obligation', 'credit', 'loan'],
            'equity': ['ownership', 'shares', 'stock', 'capital', 'stake'],
            'margin': ['profitability', 'spread', 'markup', 'gross profit'],
            'growth': ['expansion', 'increase', 'development', 'improvement'],
            'performance': ['results', 'outcomes', 'metrics', 'indicators', 'KPIs'],
            'quarterly': ['Q1', 'Q2', 'Q3', 'Q4', 'three-month', 'period'],
            'annual': ['yearly', 'year-end', 'full-year', '12-month'],
            'forecast': ['projection', 'outlook', 'guidance', 'estimate', 'prediction']
        }

        # Financial abbreviations and their expansions
        self.financial_abbreviations = {
            'EBITDA': 'Earnings Before Interest Tax Depreciation Amortization',
            'ROI': 'Return on Investment',
            'ROE': 'Return on Equity',
            'P&L': 'Profit and Loss',
            'CAPEX': 'Capital Expenditure',
            'OPEX': 'Operating Expenditure',
            'NPV': 'Net Present Value',
            'IRR': 'Internal Rate of Return',
            'KPI': 'Key Performance Indicator',
            'YoY': 'Year over Year',
            'QoQ': 'Quarter over Quarter'
        }

    # Legacy compatibility method (for backward compatibility)
    def expand_query(self, query: str, expansion_type: str = 'auto') -> Dict[str, Any]:
        """Legacy method for backward compatibility - routes to comprehensive expansion"""
        result = self.expand_query_comprehensive(query, expansion_type)
        return {
            'original': result['original_query'],
            'synonyms': result['expansions'].get('synonyms', {}).get('expanded_query', query),
            'llm_expanded': result['expansions'].get('llm_contextual', {}).get('expanded_query', query),
            'combined': result['optimal_query']
        }

    def expand_query_comprehensive(self, query: str, expansion_mode: str = 'auto') -> Dict[str, Any]:
        """
        Comprehensive query expansion using multiple techniques

        Args:
            query: Original query
            expansion_mode: 'synonyms', 'llm', 'hybrid', 'auto'

        Returns:
            Dictionary with original and expanded queries plus metadata
        """
        expansion_result = {
            'original_query': query,
            'expansion_mode': expansion_mode,
            'expansions': {},
            'confidence': 0.0,
            'expansion_stats': {}
        }

        # Step 1: Synonym-based expansion
        if expansion_mode in ['synonyms', 'hybrid', 'auto']:
            synonym_expansion = self._expand_with_financial_synonyms(query)
            expansion_result['expansions']['synonyms'] = synonym_expansion

        # Step 2: Abbreviation expansion
        if expansion_mode in ['abbreviations', 'hybrid', 'auto']:
            abbrev_expansion = self._expand_abbreviations(query)
            expansion_result['expansions']['abbreviations'] = abbrev_expansion

        # Step 3: LLM-based contextual expansion
        if expansion_mode in ['llm', 'hybrid', 'auto'] and self.llm_engine.available:
            llm_expansion = self._expand_with_llm_context(query)
            expansion_result['expansions']['llm_contextual'] = llm_expansion

        # Step 4: Create optimal combined query
        optimal_query = self._create_optimal_expansion(query, expansion_result['expansions'])
        expansion_result['optimal_query'] = optimal_query

        # Step 5: Calculate expansion quality metrics
        expansion_result['confidence'] = self._calculate_expansion_confidence(expansion_result)
        expansion_result['expansion_stats'] = self._calculate_expansion_stats(query, expansion_result)

        return expansion_result

    def _expand_with_financial_synonyms(self, query: str) -> Dict[str, Any]:
        """Expand query using financial domain synonyms"""
        words = query.lower().split()
        expanded_terms = set(words)  # Start with original words
        synonym_matches = {}

        for word in words:
            # Direct matches
            if word in self.financial_synonyms:
                synonyms = self.financial_synonyms[word][:3]  # Top 3 synonyms
                expanded_terms.update(synonyms)
                synonym_matches[word] = synonyms

            # Partial matches (for compound terms)
            for key, synonyms in self.financial_synonyms.items():
                if word in key or key in word:
                    selected_synonyms = synonyms[:2]  # Top 2 for partial matches
                    expanded_terms.update(selected_synonyms)
                    synonym_matches[f"{word}→{key}"] = selected_synonyms

        return {
            'expanded_query': ' '.join(sorted(expanded_terms)),
            'synonym_matches': synonym_matches,
            'expansion_count': len(expanded_terms) - len(words)
        }

    def _expand_abbreviations(self, query: str) -> Dict[str, Any]:
        """Expand financial abbreviations"""
        expanded_query = query
        abbreviation_matches = {}

        for abbrev, full_form in self.financial_abbreviations.items():
            if abbrev.lower() in query.lower():
                # Add both abbreviated and full form
                expanded_query += f" {full_form}"
                abbreviation_matches[abbrev] = full_form

        return {
            'expanded_query': expanded_query,
            'abbreviation_matches': abbreviation_matches,
            'expansion_count': len(abbreviation_matches)
        }

    def _expand_with_llm_context(self, query: str) -> Dict[str, Any]:
        """Use LLM for contextual financial query expansion"""
        if not self.llm_engine.available:
            return {'expanded_query': query, 'llm_terms': [], 'expansion_count': 0}

        prompt = f"""You are a financial terminology expert. Generate additional search terms for this query.

ORIGINAL QUERY: {query}

CRITICAL INSTRUCTIONS:
1. Generate 3-6 highly relevant financial terms ONLY
2. Include financial synonyms and related concepts
3. Add standard abbreviations if applicable (ROI, EBITDA, etc.)
4. Keep terms SHORT (1-3 words maximum each)
5. Focus on terms likely to appear in financial documents
6. NO explanations, NO full sentences
7. Return ONLY comma-separated terms

EXAMPLE INPUT: "revenue growth"
EXAMPLE OUTPUT: income, sales increase, turnover expansion, top-line growth

YOUR OUTPUT (comma-separated terms ONLY):"""

        try:
            llm_response = self.llm_engine.generate_response(prompt, temperature=0.3)

            if llm_response and len(llm_response.strip()) > 0:
                # Parse LLM response
                additional_terms = [term.strip() for term in llm_response.split(',')]
                additional_terms = [term for term in additional_terms if term and len(term.split()) <= 3]

                # Create expanded query
                expanded_query = f"{query} {' '.join(additional_terms[:6])}"  # Limit to 6 additional terms

                return {
                    'expanded_query': expanded_query,
                    'llm_terms': additional_terms[:6],
                    'expansion_count': len(additional_terms[:6])
                }

        except Exception as e:
            print(f"LLM expansion failed: {e}")

        return {'expanded_query': query, 'llm_terms': [], 'expansion_count': 0}

    def _create_optimal_expansion(self, original_query: str, expansions: Dict) -> str:
        """Create optimal expanded query combining all expansion methods"""
        # Start with original query terms
        all_terms = set(original_query.lower().split())

        # Add synonym terms (high priority)
        if 'synonyms' in expansions:
            synonym_terms = expansions['synonyms']['expanded_query'].split()
            all_terms.update(synonym_terms[:10])  # Limit synonyms

        # Add abbreviation expansions (medium priority)
        if 'abbreviations' in expansions:
            abbrev_query = expansions['abbreviations']['expanded_query']
            # Extract only the new terms added by abbreviation expansion
            new_terms = abbrev_query.replace(original_query, '').strip().split()
            all_terms.update(new_terms[:5])  # Limit abbreviation terms

        # Add LLM terms (selective, high-quality)
        if 'llm_contextual' in expansions:
            llm_terms = expansions['llm_contextual'].get('llm_terms', [])
            # Be selective with LLM terms - only add high-value ones
            for term in llm_terms[:4]:  # Max 4 LLM terms
                if len(term.split()) <= 2:  # Prefer shorter terms
                    all_terms.add(term.lower())

        # Create balanced expanded query
        # Prioritize original terms, then add expansions
        original_terms = original_query.split()
        expansion_terms = [term for term in all_terms if term not in original_query.lower()]

        # Limit total query length for performance
        max_expansion_terms = min(8, len(expansion_terms))
        selected_expansion_terms = sorted(expansion_terms)[:max_expansion_terms]

        optimal_query = ' '.join(original_terms + selected_expansion_terms)
        return optimal_query

    def _calculate_expansion_confidence(self, expansion_result: Dict) -> float:
        """Calculate confidence in expansion quality"""
        confidence_factors = []

        # Synonym expansion confidence
        if 'synonyms' in expansion_result['expansions']:
            synonym_count = expansion_result['expansions']['synonyms']['expansion_count']
            confidence_factors.append(min(synonym_count / 5, 1.0))  # Max confidence at 5+ synonyms

        # Abbreviation expansion confidence
        if 'abbreviations' in expansion_result['expansions']:
            abbrev_count = expansion_result['expansions']['abbreviations']['expansion_count']
            confidence_factors.append(min(abbrev_count / 2, 1.0))  # Max confidence at 2+ abbreviations

        # LLM expansion confidence
        if 'llm_contextual' in expansion_result['expansions']:
            llm_count = expansion_result['expansions']['llm_contextual']['expansion_count']
            if llm_count > 0:
                confidence_factors.append(0.8)  # High confidence if LLM provided terms

        return np.mean(confidence_factors) if confidence_factors else 0.5

    def _calculate_expansion_stats(self, original_query: str, expansion_result: Dict) -> Dict[str, Any]:
        """Calculate expansion statistics"""
        original_word_count = len(original_query.split())
        optimal_word_count = len(expansion_result.get('optimal_query', original_query).split())

        return {
            'original_word_count': original_word_count,
            'optimal_word_count': optimal_word_count,
            'expansion_ratio': optimal_word_count / original_word_count if original_word_count > 0 else 1.0,
            'synonyms_added': expansion_result['expansions'].get('synonyms', {}).get('expansion_count', 0),
            'abbreviations_added': expansion_result['expansions'].get('abbreviations', {}).get('expansion_count', 0),
            'llm_terms_added': expansion_result['expansions'].get('llm_contextual', {}).get('expansion_count', 0)
        }

# Initialize the consolidated query expansion system
try:
    # Replace both old systems with this enhanced version
    advanced_query_expansion = AdvancedFinancialQueryExpansion(llm_engine)
    # Create alias for backward compatibility
    query_expansion = advanced_query_expansion

    print("✅ CONSOLIDATED Advanced Financial Query Expansion System initialized!")
    print("   🔤 Financial domain synonyms (15+ categories)")
    print("   🔣 Financial abbreviation expansion")
    print("   🤖 LLM contextual expansion")
    print("   ⚖️ Optimal query balancing")
    print("   📊 Expansion quality metrics")
    print("   🔄 Backward compatibility maintained")
    print("   ♻️ Redundant systems consolidated")

except Exception as e:
    print(f"❌ Consolidated query expansion initialization failed: {e}")


✅ CONSOLIDATED Advanced Financial Query Expansion System initialized!
   🔤 Financial domain synonyms (15+ categories)
   🔣 Financial abbreviation expansion
   🤖 LLM contextual expansion
   ⚖️ Optimal query balancing
   📊 Expansion quality metrics
   🔄 Backward compatibility maintained
   ♻️ Redundant systems consolidated


#### 6.4 Advanced RAG Orchestrator - Query Expansion and Enhancement

In [54]:
# Advanced RAG Orchestrator - Main Processing Engine
class AdvancedRAGOrchestrator:
    """Main RAG system orchestrating all components"""

    def __init__(self, hybrid_search, llm_engine: EnhancedLLMEngine,
                 doc_finder: DocumentFinder, query_expansion=None, cross_encoder=None):
        self.hybrid_search = hybrid_search
        self.llm_engine = llm_engine
        self.doc_finder = doc_finder
        self.query_expansion = query_expansion  # Add query expansion
        self.cross_encoder = cross_encoder

    def ensure_indices_ready(self, collection_names: List[str]) -> bool:
        """Ensure BM25 indices are ready for all collections"""
        if not hasattr(self.hybrid_search.manager, 'bm25_indices'):
            return False

        missing_indices = []
        for collection_name in collection_names:
            if collection_name not in self.hybrid_search.manager.bm25_indices:
                # Try to load pre-built index
                index_path = self.hybrid_search.manager.bm25_dir / f"{collection_name}_bm25.pkl"
                if index_path.exists():
                    try:
                        with open(index_path, 'rb') as f:
                            self.hybrid_search.manager.bm25_indices[collection_name] = pickle.load(f)
                        print(f"📂 Loaded pre-built index for {collection_name}")
                    except Exception as e:
                        print(f"⚠️ Failed to load index for {collection_name}: {e}")
                        missing_indices.append(collection_name)
                else:
                    missing_indices.append(collection_name)

        # Build missing indices if needed
        if missing_indices:
            print(f"🔨 Building missing indices for: {missing_indices}")
            for collection_name in missing_indices:
                self.hybrid_search.manager.build_bm25_index(collection_name)

        return len(missing_indices) == 0

    def process_query(self, query: str, collection_names: List[str],
                  use_reranking: bool = True, max_results: int = 10) -> Dict[str, Any]:
        """Main query processing pipeline with enhanced routing and reliability checks"""

        # Ensure BM25 indices are loaded for all collections
        self.ensure_indices_ready(collection_names)

        # Step 1: Classify and route query
        query_info = self.llm_engine.classify_query(query)

        # Step 2: Handle different query types
        if query_info['filename']:
            return self._handle_file_specific_query(query, query_info, collection_names, max_results)
        elif query_info['query_type'] == 'document_summary':
            return self._handle_summary_query(query, query_info, collection_names, max_results)
        elif query_info['is_comparative']:
            return self._handle_comparison_query(query, query_info, collection_names, max_results)
        else:
            return self._handle_general_query(query, query_info, collection_names, max_results, use_reranking)  # ✅ Pass it through

    def _handle_general_query(self, query: str, query_info: Dict, collection_names: List[str],
                             max_results: int, use_reranking: bool) -> Dict[str, Any]:
        """Handle general search queries with ENHANCED RRF processing and consolidated query expansion"""

        # Enhanced query expansion using the consolidated system
        expanded_query = query
        expansion_metadata = {}

        if self.query_expansion:
            try:
                # Use the comprehensive expansion method
                expansion_result = self.query_expansion.expand_query_comprehensive(query, 'auto')
                expanded_query = expansion_result.get('optimal_query', query)

                # Store expansion metadata for response
                expansion_metadata = {
                    'original_query': expansion_result.get('original_query', query),
                    'expansion_mode': expansion_result.get('expansion_mode', 'auto'),
                    'confidence': expansion_result.get('confidence', 0.0),
                    'expansion_stats': expansion_result.get('expansion_stats', {}),
                    'synonym_matches': expansion_result.get('expansions', {}).get('synonyms', {}).get('synonym_matches', {}),
                    'abbreviation_matches': expansion_result.get('expansions', {}).get('abbreviations', {}).get('abbreviation_matches', {}),
                    'llm_terms': expansion_result.get('expansions', {}).get('llm_contextual', {}).get('llm_terms', [])
                }

                print(f"🔍 Query expanded: {query} → {expanded_query}")
                if expansion_metadata.get('expansion_stats', {}).get('expansion_ratio', 1.0) > 1.2:
                    print(f"📈 Expansion ratio: {expansion_metadata['expansion_stats']['expansion_ratio']:.1f}x")

            except Exception as e:
                print(f"Query expansion failed: {e}")
                expanded_query = query
                expansion_metadata = {'error': str(e)}

        # Use the ENHANCED multi-collection search with improved RRF
        results = self.hybrid_search.multi_collection_search(
            expanded_query, collection_names, max_results
        )

        if not results:
            return {
                'status': 'no_results',
                'query': query,
                'expanded_query': expanded_query if expanded_query != query else None,
                'expansion_metadata': expansion_metadata,
                'answer': 'No relevant information found in the selected collections.',
                'references': [],
                'collections_searched': collection_names,
                'search_techniques': ['BM25', 'Semantic', 'Enhanced_RRF'],
                'fusion_method': 'Enhanced_RRF_v2'
            }

        # Enhanced results processing with confidence filtering
        final_results = []
        processing_stats = {
            'total_found': len(results),
            'high_confidence': 0,
            'medium_confidence': 0,
            'low_confidence': 0,
            'avg_rrf_score': 0,
            'avg_final_score': 0,
            'cross_collection_matches': 0,
            'semantic_only': 0,
            'bm25_only': 0,
            'both_methods': 0
        }

        rrf_scores = []
        final_scores = []
        confidence_scores = []

        for result in results:
            # Enhanced confidence calculation
            confidence = result.get('confidence', 0)
            final_score = result.get('final_score', 0)
            rrf_score = result.get('rrf_score', 0)
            in_both_sets = result.get('in_both_sets', False)

            # Confidence categorization with enhanced thresholds
            if confidence >= 0.7:
                processing_stats['high_confidence'] += 1
                confidence_level = 'high'
            elif confidence >= 0.5:
                processing_stats['medium_confidence'] += 1
                confidence_level = 'medium'
            else:
                processing_stats['low_confidence'] += 1
                confidence_level = 'low'

            # Track search method statistics
            has_semantic = result.get('semantic_score', 0) > 0
            has_bm25 = result.get('bm25_score', 0) > 0

            if has_semantic and has_bm25:
                processing_stats['both_methods'] += 1
            elif has_semantic:
                processing_stats['semantic_only'] += 1
            elif has_bm25:
                processing_stats['bm25_only'] += 1

            # Track cross-collection matches
            if in_both_sets:
                processing_stats['cross_collection_matches'] += 1

            rrf_scores.append(rrf_score)
            final_scores.append(final_score)
            confidence_scores.append(confidence)

            # Apply enhanced confidence threshold (adaptive based on query complexity)
            min_confidence = 0.3 if len(query.split()) > 5 else 0.25

            if confidence >= min_confidence:
                # Add enhanced metadata to result
                result['confidence_level'] = confidence_level
                result['search_method_used'] = 'hybrid' if (has_semantic and has_bm25) else ('semantic' if has_semantic else 'bm25')
                result['quality_indicators'] = {
                    'high_rrf': rrf_score > 0.1,
                    'cross_method_match': in_both_sets,
                    'good_length': 100 <= len(result.get('text', '')) <= 1000,
                    'has_metadata': bool(result.get('metadata', {}).get('filename'))
                }
                final_results.append(result)

        # Calculate enhanced processing statistics
        processing_stats.update({
            'total_processed': len(final_results),
            'avg_rrf_score': np.mean(rrf_scores) if rrf_scores else 0,
            'avg_final_score': np.mean(final_scores) if final_scores else 0,
            'avg_confidence': np.mean(confidence_scores) if confidence_scores else 0,
            'confidence_distribution': {
                'high': processing_stats['high_confidence'],
                'medium': processing_stats['medium_confidence'],
                'low': processing_stats['low_confidence']
            },
            'method_distribution': {
                'both_methods': processing_stats['both_methods'],
                'semantic_only': processing_stats['semantic_only'],
                'bm25_only': processing_stats['bm25_only']
            },
            'fusion_effectiveness': {
                'cross_matches': processing_stats['cross_collection_matches'],
                'fusion_boost_applied': sum(1 for r in final_results if r.get('in_both_sets', False))
            }
        })

        # Limit final results with smart selection
        if len(final_results) > max_results:
            final_results = self._ensure_result_diversity(final_results, max_results)
        else:
            final_results = final_results[:max_results]

        # Apply cross-encoder reranking if available and beneficial
        if use_reranking and self.cross_encoder and hasattr(self.cross_encoder, 'available') and self.cross_encoder.available:
            try:
                if len(final_results) > 1:
                    reranked_results = self.cross_encoder.rerank(query, final_results, len(final_results))
                    if reranked_results:
                        final_results = reranked_results
                        processing_stats['reranking_applied'] = True
            except Exception as e:
                print(f"Cross-encoder reranking failed: {e}")
                processing_stats['reranking_applied'] = False

        # Extract enhanced references with comprehensive score details
        references = []
        for i, result in enumerate(final_results):
            # Validate result has required 'text' field
            if 'text' not in result or not result.get('text'):
                continue

            metadata = result.get('metadata', {})

            ref = {
                'filename': metadata.get('filename', 'Unknown'),
                'collection': result.get('source_collection', metadata.get('collection', 'Unknown')),
                'chunk_id': metadata.get('chunk_id'),
                'text_snippet': result['text'][:200] + '...' if len(result['text']) > 200 else result['text'],
                'full_text': result['text'],
                'rank': i + 1,
                'scores': {
                    'final_score': result.get('final_score', 0),
                    'rrf_score': result.get('rrf_score', 0),
                    'confidence': result.get('confidence', 0),
                    'confidence_level': result.get('confidence_level', 'unknown'),
                    'semantic_score': result.get('semantic_score', 0),
                    'semantic_score_normalized': result.get('semantic_score_normalized', 0),
                    'bm25_score': result.get('bm25_score', 0),
                    'bm25_score_normalized': result.get('bm25_score_normalized', 0),
                    'in_both_sets': result.get('in_both_sets', False),
                    'quality_bonus': result.get('quality_bonus', 0),
                    'diversity_bonus': result.get('diversity_bonus', 0)
                },
                'quality_indicators': result.get('quality_indicators', {}),
                'search_method_used': result.get('search_method_used', 'unknown'),
                'fusion_details': {
                    'semantic_rank': result.get('semantic_rank'),
                    'bm25_rank': result.get('bm25_rank'),
                    'fusion_method': result.get('fusion_method', 'Enhanced_RRF'),
                    'cross_boost_applied': result.get('in_both_sets', False)
                }
            }

            # Add cross-encoder scores if available
            if 'rerank_score' in result:
                ref['scores']['cross_encoder_score'] = result['rerank_score']

            references.append(ref)

        # Generate enhanced contextual response
        answer = self._generate_enhanced_contextual_response(query, final_results)

        # Build comprehensive response with expansion metadata
        response = {
            'status': 'success',
            'query': query,
            'expanded_query': expanded_query if expanded_query != query else None,
            'expansion_metadata': expansion_metadata,
            'query_type': query_info['query_type'],
            'answer': answer,
            'references': references,
            'collections_searched': collection_names,
            'chunks_used': len(final_results),
            'processing_stats': processing_stats,
            'fusion_method': 'Enhanced_RRF_v2',
            'search_techniques': ['BM25', 'Semantic', 'Enhanced_RRF', 'Cross-Encoder'] if use_reranking else ['BM25', 'Semantic', 'Enhanced_RRF'],
            'system_metadata': {
                'rrf_params': self.hybrid_search.rrf_params if hasattr(self.hybrid_search, 'rrf_params') else {},
                'confidence_thresholds': {'min_applied': min_confidence, 'adaptive': len(query.split()) > 5},
                'diversity_applied': len(collection_names) > 1,
                'query_expansion_applied': bool(self.query_expansion and expanded_query != query),
                'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
            }
        }

        return response

    def _ensure_result_diversity(self, results: List[Dict], max_results: int) -> List[Dict]:
        """Ensure diversity in final result selection"""
        if len(results) <= max_results:
            return results

        # Group results by collection and confidence level
        grouped_results = {}
        for result in results:
            collection = result.get('source_collection', 'unknown')
            confidence_level = result.get('confidence_level', 'low')
            key = f"{collection}_{confidence_level}"

            if key not in grouped_results:
                grouped_results[key] = []
            grouped_results[key].append(result)

        # Select diverse results
        selected_results = []
        remaining_slots = max_results

        # First pass: take best from each group
        for group_results in grouped_results.values():
            if remaining_slots > 0:
                group_results.sort(key=lambda x: x.get('final_score', 0), reverse=True)
                selected_results.append(group_results[0])
                remaining_slots -= 1

        # Second pass: fill remaining slots with highest scores
        if remaining_slots > 0:
            remaining_results = []
            for group_results in grouped_results.values():
                remaining_results.extend(group_results[1:])

            remaining_results.sort(key=lambda x: x.get('final_score', 0), reverse=True)
            selected_results.extend(remaining_results[:remaining_slots])

        # Final sort by score
        selected_results.sort(key=lambda x: x.get('final_score', 0), reverse=True)
        return selected_results

    def _generate_enhanced_contextual_response(self, query: str, results: List[Dict]) -> str:
        """Generate enhanced contextual response with confidence indicators and query awareness"""

        if not results:
            return "No relevant information found in the selected collections."

        # Build context with enhanced confidence and source attribution
        context_parts = []
        high_confidence_sources = []
        medium_confidence_sources = []

        for i, result in enumerate(results, 1):
            # Validate result has required 'text' field
            if 'text' not in result or not result.get('text'):
                continue

            filename = result.get('metadata', {}).get('filename', 'Unknown')
            collection = result.get('source_collection', 'Unknown')
            confidence = result.get('confidence', 0)
            confidence_level = result.get('confidence_level', 'unknown')
            text = result['text']

            # Track sources by confidence
            if confidence_level == 'high':
                high_confidence_sources.append(f"Source {i}")
            elif confidence_level == 'medium':
                medium_confidence_sources.append(f"Source {i}")

            # Build context with rich metadata
            context_parts.append(
                f"[Source {i} - {filename} from {collection} (Confidence: {confidence_level}, Score: {confidence:.3f})]: {text}"
            )

        context = "\n\n".join(context_parts)

        # Enhanced prompt with strict instructions for better compliance
        prompt = f"""You are a precise financial analyst. Answer the question using ONLY the provided context.

CONTEXT WITH CONFIDENCE LEVELS:
{context[:4000]}

QUESTION: {query}

CRITICAL INSTRUCTIONS (FOLLOW EXACTLY):
1. Use ONLY information from the context above - NO external knowledge
2. For EACH fact, cite the source: (Source X - filename)
3. Prioritize high-confidence sources: {', '.join(high_confidence_sources) if high_confidence_sources else 'None'}
4. Use medium-confidence sources for supporting details: {', '.join(medium_confidence_sources) if medium_confidence_sources else 'None'}
5. If sources conflict, present both views with their confidence levels
6. If information missing, explicitly state: "Information not found in provided documents"
7. Write in clear paragraphs with proper structure
8. Be factual and precise - NO speculation or assumptions
9. End with your overall confidence assessment

FORMAT:
- Introduction paragraph
- Key findings with citations
- Supporting details with citations
- Conclusion with confidence level

YOUR ANSWER:"""

        return self.llm_engine.generate_response(prompt, temperature=0.0)

# Update the RAG orchestrator initialization with query expansion
try:
    import time  # Import time for timestamp

    rag_orchestrator = AdvancedRAGOrchestrator(
        hybrid_search=enhanced_hybrid_search,
        llm_engine=llm_engine,
        doc_finder=doc_finder,
        query_expansion=advanced_query_expansion,  # Add the consolidated query expansion
        cross_encoder=cross_encoder if 'cross_encoder' in globals() else None
    )

    print("✅ UPDATED Advanced RAG Orchestrator initialized!")
    print("   🎯 Complete query processing pipeline")
    print("   🔤 Integrated consolidated query expansion system")
    print("   📊 Enhanced RRF fusion with confidence scoring")
    print("   🔄 Multi-collection search optimization")
    print("   💎 Comprehensive source attribution")
    print("   ⚡ Cross-encoder reranking integration")
    print("   📈 Advanced quality indicators")
    print("   🎨 Result diversity optimization")
    print("   🧠 Intelligent query routing")
    print("   📝 Query expansion metadata tracking")

except Exception as e:

    print(f"❌ RAG Orchestrator initialization failed: {e}")
    print("   - advanced_query_expansion (Section 6.4)")

    print("Make sure all required components are initialized:")
    print("   - doc_finder (Section 6.2)")

    print("   - enhanced_hybrid_search (Section 5.2)")
    print("   - llm_engine (Section 6.1)")

✅ UPDATED Advanced RAG Orchestrator initialized!
   🎯 Complete query processing pipeline
   🔤 Integrated consolidated query expansion system
   📊 Enhanced RRF fusion with confidence scoring
   🔄 Multi-collection search optimization
   💎 Comprehensive source attribution
   ⚡ Cross-encoder reranking integration
   📈 Advanced quality indicators
   🎨 Result diversity optimization
   🧠 Intelligent query routing
   📝 Query expansion metadata tracking


#### 6.5 Results Processing for confidence scoring and filtering

In [55]:
# Enhanced Results Processing

class ResultsProcessor:
    """Advanced results processing with confidence scoring and filtering"""

    def __init__(self, cross_encoder=None):
        self.cross_encoder = cross_encoder
        self.confidence_thresholds = {
            'high': 0.8,
            'medium': 0.5,
            'low': 0.3
        }

    def process_results(self, results: List[Dict], query: str,
                       apply_reranking: bool = True,
                       min_confidence: float = 0.3) -> Dict[str, Any]:
        """
        Process and enhance search results

        Args:
            results: Raw search results
            query: Original query
            apply_reranking: Whether to apply cross-encoder reranking
            min_confidence: Minimum confidence threshold

        Returns:
            Processed results with confidence scores and filtering
        """
        if not results:
            return self._empty_results(query)

        processed_results = []

        for result in results:
            # Calculate confidence score
            confidence = self._calculate_confidence(result)

            # Skip low-confidence results
            if confidence < min_confidence:
                continue

            # Enhance result with additional metadata
            enhanced_result = {
                **result,
                'confidence': confidence,
                'confidence_level': self._get_confidence_level(confidence),
                'processed_text': self._process_text_snippet(result.get('text', '')),
                'relevance_indicators': self._extract_relevance_indicators(result, query)
            }

            processed_results.append(enhanced_result)

        # Apply cross-encoder reranking if available
        if apply_reranking and self.cross_encoder and processed_results:
            processed_results = self._apply_cross_encoder_reranking(query, processed_results)

        # Sort by final confidence/score
        processed_results.sort(
            key=lambda x: (x.get('final_score', 0) + x.get('confidence', 0)) / 2,
            reverse=True
        )

        return {
            'results': processed_results,
            'total_found': len(results),
            'total_processed': len(processed_results),
            'average_confidence': np.mean([r['confidence'] for r in processed_results]) if processed_results else 0,
            'query': query
        }

    def _calculate_confidence(self, result: Dict) -> float:
        """Calculate confidence score based on multiple factors"""
        factors = []

        # RRF score factor
        if 'final_score' in result:
            factors.append(min(result['final_score'] * 2, 1.0))

        # Semantic score factor
        if 'semantic_score' in result:
            factors.append(result['semantic_score'])

        # BM25 score factor (normalized)
        if 'bm25_score' in result:
            factors.append(min(result['bm25_score'] / 10, 1.0))

        # Text length factor (optimal length gets higher score)
        text_len = len(result.get('text', ''))
        if 200 <= text_len <= 1000:
            factors.append(0.8)
        elif 100 <= text_len <= 1500:
            factors.append(0.6)
        else:
            factors.append(0.4)

        # Metadata completeness factor
        metadata = result.get('metadata', {})
        if metadata.get('filename') and metadata.get('chunk_id'):
            factors.append(0.7)

        return np.mean(factors) if factors else 0.5

    def _get_confidence_level(self, confidence: float) -> str:
        """Get confidence level label"""
        if confidence >= self.confidence_thresholds['high']:
            return 'high'
        elif confidence >= self.confidence_thresholds['medium']:
            return 'medium'
        else:
            return 'low'

    def _process_text_snippet(self, text: str, max_length: int = 300) -> str:
        """Process text snippet for display"""
        if not text:
            return ""

        # Clean and truncate
        clean_text = re.sub(r'\s+', ' ', text.strip())

        if len(clean_text) <= max_length:
            return clean_text

        # Try to break at sentence boundary
        truncated = clean_text[:max_length]
        last_period = truncated.rfind('.')
        last_space = truncated.rfind(' ')

        if last_period > max_length - 50:
            return truncated[:last_period + 1]
        elif last_space > max_length - 20:
            return truncated[:last_space] + "..."
        else:
            return truncated + "..."

    def _extract_relevance_indicators(self, result: Dict, query: str) -> List[str]:
        """Extract indicators of why this result is relevant"""
        indicators = []
        text = result.get('text', '').lower()
        query_terms = query.lower().split()

        # Check for direct matches
        matches = [term for term in query_terms if term in text]
        if matches:
            indicators.append(f"Contains: {', '.join(matches[:3])}")

        # Check for semantic indicators
        if result.get('semantic_score', 0) > 0.7:
            indicators.append("High semantic similarity")

        # Check for keyword indicators
        if result.get('bm25_score', 0) > 5:
            indicators.append("Strong keyword match")

        # Check metadata indicators
        metadata = result.get('metadata', {})
        if metadata.get('doc_type') == 'financial_document':
            indicators.append("Financial document")

        return indicators

    def _apply_cross_encoder_reranking(self, query: str, results: List[Dict]) -> List[Dict]:
        """Apply cross-encoder reranking if available"""
        if not self.cross_encoder or not hasattr(self.cross_encoder, 'predict'):
            return results

        try:
            # Prepare query-document pairs
            pairs = [(query, result.get('text', '')[:500]) for result in results]

            # Get reranking scores
            rerank_scores = self.cross_encoder.predict(pairs)

            # Add reranking scores to results
            for i, result in enumerate(results):
                result['rerank_score'] = float(rerank_scores[i])
                # Combine with existing scores
                result['final_score'] = (
                    result.get('final_score', 0) * 0.7 +
                    result['rerank_score'] * 0.3
                )

            return results

        except Exception as e:
            print(f"Cross-encoder reranking failed: {e}")
            return results

    def _empty_results(self, query: str) -> Dict[str, Any]:
        """Return empty results structure"""
        return {
            'results': [],
            'total_found': 0,
            'total_processed': 0,
            'average_confidence': 0.0,
            'query': query
        }

# Initialize results processor
try:
    results_processor = ResultsProcessor(cross_encoder if 'cross_encoder' in globals() else None)
    print("✅ Enhanced Results Processor initialized!")
    print("   - Confidence scoring based on multiple factors")
    print("   - Cross-encoder reranking integration")
    print("   - Text snippet processing and cleanup")
    print("   - Relevance indicator extraction")
except Exception as e:
    print(f"❌ Results processor initialization failed: {e}")

✅ Enhanced Results Processor initialized!
   - Confidence scoring based on multiple factors
   - Cross-encoder reranking integration
   - Text snippet processing and cleanup
   - Relevance indicator extraction


## 7. Enhanced User Interface

#### 7.0 Enhanced Citation and Cross-Document Analysis Systems

In [56]:
# Enhanced Citation and Cross-Document Analysis Systems
class EnhancedCitationSystem:
    """Precise citation system with confidence scores and source verification"""

    def __init__(self, collections_manager):
        self.manager = collections_manager

    def generate_precise_citations(self, references: List[Dict]) -> List[Dict]:
        """Generate precise citations with confidence scoring"""
        citations = []

        for i, ref in enumerate(references):
            # Validate ref has required 'text' field
            if 'text' not in ref or not ref.get('text'):
                continue

            metadata = ref.get('metadata', {})

            citation = {
                'id': f"cite_{i+1}",
                'filename': metadata.get('filename', 'Unknown'),
                'collection': ref.get('collection', metadata.get('collection', 'Unknown')),
                'chunk_id': metadata.get('chunk_id'),
                'page_num': metadata.get('page_num'),
                'confidence_score': ref.get('confidence', 0.0),
                'relevance_score': ref.get('final_score', 0.0),
                'text_snippet': ref['text'][:150] + '...' if len(ref['text']) > 150 else ref['text'],
                'source_type': 'document_chunk',
                'verification_status': 'verified' if ref.get('confidence', 0) > 0.7 else 'low_confidence'
            }

            citations.append(citation)

        return citations

    def format_citation_text(self, citation: Dict) -> str:
        """Format citation for display in answers"""
        filename = citation['filename']
        collection = citation['collection']
        confidence = citation['confidence_score']

        return f"[{citation['id']}: {filename} from {collection}, confidence: {confidence:.2f}]"

# Cross-Document Analysis System
class CrossDocumentAnalyzer:
    """Analyze information across multiple documents"""

    def __init__(self, llm_engine, collections_manager):
        self.llm_engine = llm_engine
        self.manager = collections_manager

    def analyze_cross_document_patterns(self, references: List[Dict], query: str) -> Dict[str, Any]:
        """Analyze patterns and consistency across documents"""

        # Group references by document
        docs_analysis = {}
        for ref in references:
            filename = ref.get('metadata', {}).get('filename', 'Unknown')
            if filename not in docs_analysis:
                docs_analysis[filename] = {
                    'chunks': [],
                    'total_confidence': 0,
                    'collection': ref.get('collection', 'Unknown')
                }

            docs_analysis[filename]['chunks'].append(ref)
            docs_analysis[filename]['total_confidence'] += ref.get('confidence', 0)

        # Calculate document-level insights
        document_insights = []
        for filename, analysis in docs_analysis.items():
            avg_confidence = analysis['total_confidence'] / len(analysis['chunks'])

            insight = {
                'filename': filename,
                'collection': analysis['collection'],
                'chunk_count': len(analysis['chunks']),
                'average_confidence': avg_confidence,
                'coverage_assessment': 'high' if len(analysis['chunks']) > 2 else 'medium' if len(analysis['chunks']) > 1 else 'low'
            }

            document_insights.append(insight)

        return {
            'total_documents': len(docs_analysis),
            'document_insights': document_insights,
            'cross_document_consistency': self._assess_consistency(references),
            'information_completeness': 'high' if len(references) > 5 else 'medium' if len(references) > 2 else 'low'
        }

    def _assess_consistency(self, references: List[Dict]) -> str:
        """Assess consistency across document references"""
        if len(references) < 2:
            return 'insufficient_data'

        confidence_scores = [ref.get('confidence', 0) for ref in references]
        avg_confidence = np.mean(confidence_scores)
        confidence_variance = np.var(confidence_scores)

        if avg_confidence > 0.7 and confidence_variance < 0.1:
            return 'high'
        elif avg_confidence > 0.5:
            return 'medium'
        else:
            return 'low'

print("✅ Citation and Cross-Document Analysis Systems initialized!")

✅ Citation and Cross-Document Analysis Systems initialized!


#### 7.0.1 Cross-Reference Comparison System

In [57]:
# Cross-Reference Comparison System
class CrossReferenceComparator:
    """
    Handles cross-reference comparisons between documents to identify alignment and misalignment.
    Compares target documents against reference documents (e.g., regulations, policies, standards).
    """

    def __init__(self, llm_engine, collections_manager, embedding_model, hybrid_search=None):
        self.llm_engine = llm_engine
        self.manager = collections_manager
        self.embedding_model = embedding_model
        self.hybrid_search = hybrid_search

    def compare_documents(self, target_doc: str, reference_collection: str,
                         target_collection: str = None, top_k: int = 20) -> Dict[str, Any]:
        """
        Compare a target document against a reference collection (e.g., regulations).

        Args:
            target_doc: Name of the document to analyze
            reference_collection: Collection containing reference documents (e.g., 'pra_rulebook')
            target_collection: Optional collection where target document is located
            top_k: Number of reference chunks to retrieve for comparison

        Returns:
            Comprehensive comparison report with misalignments, overview, and confidence score
        """

        # Step 1: Retrieve target document content
        target_content = self._get_document_content(target_doc, target_collection)
        if not target_content:
            return {
                'status': 'error',
                'message': f"Could not find document: {target_doc}",
                'target_doc': target_doc,
                'reference_collection': reference_collection
            }

        # Step 2: Generate overview of target document
        overview = self._generate_document_overview(target_doc, target_content)

        # Step 3: Extract key claims and requirements from target document
        target_claims = self._extract_key_claims(target_content)

        # Step 4: For each claim, retrieve relevant reference passages
        comparison_results = []
        for claim in target_claims:
            # Retrieve relevant reference passages
            reference_passages = self._retrieve_relevant_references(
                claim['text'], reference_collection, top_k=20
            )

            # Analyze alignment/misalignment
            alignment_analysis = self._analyze_alignment(
                claim, reference_passages, target_doc, reference_collection
            )

            if alignment_analysis['has_misalignment']:
                comparison_results.append(alignment_analysis)

        # Step 5: Calculate overall confidence score
        confidence_score = self._calculate_confidence_score(comparison_results, len(target_claims))

        # Step 6: Format final report
        report = self._format_comparison_report(
            target_doc=target_doc,
            reference_collection=reference_collection,
            overview=overview,
            misalignments=comparison_results,
            confidence_score=confidence_score,
            total_claims_analyzed=len(target_claims)
        )

        return report

    def _get_document_content(self, doc_name: str, collection_name: str = None) -> List[Dict]:
        """Retrieve all chunks for a specific document"""
        collections_to_search = [collection_name] if collection_name else self.manager.get_available_collections()

        all_chunks = []
        for coll_name in collections_to_search:
            try:
                collection = self.manager.get_collection(coll_name)
                if collection:
                    # Get all documents from collection
                    results = collection.get()

                    # Filter for the specific document
                    for i, metadata in enumerate(results.get('metadatas', [])):
                        filename = metadata.get('filename', '')
                        if doc_name.lower() in filename.lower() or filename.lower() in doc_name.lower():
                            chunk = {
                                'text': results['documents'][i],
                                'metadata': metadata,
                                'collection': coll_name
                            }
                            all_chunks.append(chunk)
            except Exception as e:
                print(f"Error retrieving from collection {coll_name}: {e}")
                continue

        # Sort by chunk offset if available
        all_chunks.sort(key=lambda x: x['metadata'].get('offset', {}).get('start', 0))
        return all_chunks

    def _generate_document_overview(self, doc_name: str, content_chunks: List[Dict]) -> str:
        """Generate comprehensive overview of entire document using representative sampling"""
        if not content_chunks:
            return "No content available for overview."

        # Sample from beginning, middle, and end to capture entire document scope
        total_chunks = len(content_chunks)

        # Strategy: Take chunks from different sections for representative coverage
        if total_chunks <= 10:
            # Small document - use all chunks
            sample_chunks = content_chunks
        else:
            # Large document - sample strategically
            # Beginning (first 3 chunks)
            beginning = content_chunks[:3]
            # Middle (3 chunks from middle third)
            middle_start = total_chunks // 3
            middle = content_chunks[middle_start:middle_start + 3]
            # End (last 3 chunks)
            end = content_chunks[-3:]
            # Additional scattered chunks for diversity
            step = max(1, total_chunks // 12)
            scattered = [content_chunks[i] for i in range(0, total_chunks, step)][:3]

            sample_chunks = beginning + middle + end + scattered

        # Combine sample text (increased from 3000 to 5000 chars for better coverage)
        sample_text = " ".join([chunk['text'] for chunk in sample_chunks])[:5000]

        overview_prompt = f"""Analyze this document and provide a comprehensive 5-6 sentence overview covering the entire document's scope.

Document Name: {doc_name}

Representative Content (sampled from beginning, middle, and end):
{sample_text}

Instructions:
- Write 5-6 clear, factual sentences
- Cover the main topics, purpose, and scope of the entire document
- Be specific about what the document contains
- Do NOT make up information not present in the content
- Focus on factual description, not subjective interpretation

Overview:"""

        try:
            # Temperature 0.1 for more accurate, factual summaries
            # Increased max_tokens to 500 to accommodate 5-6 sentences
            response = self.llm_engine.generate_response(overview_prompt, temperature=0.1, max_tokens=500)
            return response.strip()
        except:
            return f"Overview for {doc_name}: Document contains {len(content_chunks)} sections covering various topics across {total_chunks} segments."

    def _extract_key_claims(self, content_chunks: List[Dict], max_claims: int = 15) -> List[Dict]:
        """Extract key claims, statements, and requirements from document"""
        claims = []

        # Process chunks in batches to extract claims
        # Use longer excerpts for better context and analysis
        for i, chunk in enumerate(content_chunks[:max_claims]):
            # Get full chunk text but limit to reasonable size for LLM processing
            full_text = chunk['text']

            # Extend to include more context if chunk is short
            if len(full_text) < 500 and i + 1 < len(content_chunks):
                full_text += " " + content_chunks[i + 1]['text'][:500]

            claim = {
                'id': f"claim_{i+1}",
                'text': full_text[:3000],  # Limit text for initial analysis
                'full_text': full_text[:3000],  # Store even longer version for detailed output
                'metadata': chunk['metadata'],
                'chunk_index': i
            }
            claims.append(claim)

        return claims

    def _retrieve_relevant_references(self, claim_text: str, reference_collection: str, top_k: int = 20) -> List[Dict]:
        """Retrieve relevant passages from reference collection for comparison"""
        try:
            if self.hybrid_search:
                # Use hybrid search if available
                results = self.hybrid_search.hybrid_search(
                    query=claim_text,
                    collection_name=reference_collection,
                    k=top_k
                )
            else:
                # Fall back to semantic search
                collection = self.manager.get_collection(reference_collection)
                if collection:
                    query_embedding = self.embedding_model.encode([claim_text]).tolist()
                    results_raw = collection.query(
                        query_embeddings=query_embedding,
                        n_results=top_k
                    )

                    # Format results
                    results = []
                    for i in range(len(results_raw['documents'][0])):
                        results.append({
                            'text': results_raw['documents'][0][i],
                            'metadata': results_raw['metadatas'][0][i],
                            'score': 1.0 - results_raw['distances'][0][i] if results_raw.get('distances') else 0.5
                        })
                else:
                    results = []

            return results
        except Exception as e:
            print(f"Error retrieving references: {e}")
            return []

    def _analyze_alignment(self, claim: Dict, reference_passages: List[Dict],
                          target_doc: str, reference_collection: str) -> Dict[str, Any]:
        """
        Analyze whether a claim aligns with or conflicts with reference passages.
        Only flag misalignments/conflicts.
        """

        if not reference_passages:
            return {
                'has_misalignment': True,
                'claim_id': claim['id'],
                'claim_text': claim['text'][:1000],
                'misalignment_type': 'no_reference_found',
                'explanation': 'No relevant reference material found for comparison.',
                'severity': 'medium',
                'references': []
            }

        # Prepare context from reference passages with more detail
        reference_context = "\n\n".join([
            f"Reference {i+1} (from {ref['metadata'].get('filename', 'Unknown')}, page {ref['metadata'].get('page_num', 'N/A')}): {ref['text'][:800]}"
            for i, ref in enumerate(reference_passages[:3])
        ])

        # LLM analysis prompt with strict grounding instructions
        analysis_prompt = f"""You are a compliance analyst comparing documents against regulations.

TARGET DOCUMENT: {target_doc}
TARGET STATEMENT (FULL CONTEXT):
{claim.get('full_text', claim['text'])[:2000]}

REFERENCE REGULATIONS (from {reference_collection}):
{reference_context}

TASK: Determine if the target statement conflicts with, contradicts, or misaligns with the reference regulations.

STRICT REQUIREMENTS:
1. Only identify MISALIGNMENTS or CONFLICTS - if aligned, respond "ALIGNED"
2. Base analysis ONLY on the provided reference regulations above
3. Quote specific phrases from BOTH target statement and reference regulations
4. Focus on substance and spirit of regulations, not just exact wording
5. Consider both explicit violations and implicit conflicts
6. Do NOT make up regulations or requirements not shown above
7. If insufficient reference context, state this clearly

Analyze and respond in this format:
STATUS: [ALIGNED or MISALIGNED]
SEVERITY: [high/medium/low - only if MISALIGNED]
EXPLANATION: [If MISALIGNED, provide detailed explanation with:
- Specific quote from target statement showing the concern
- Specific quote from reference regulation showing the requirement
- Clear explanation of the conflict or gap
- Page references for both documents
If ALIGNED, just say "No misalignment detected."]

Response:"""

        try:
            response = self.llm_engine.generate_response(analysis_prompt, temperature=0.1)

            # Parse response
            response_lower = response.lower()

            # Check if aligned or misaligned
            if 'status: aligned' in response_lower or 'status:aligned' in response_lower:
                return {'has_misalignment': False}

            # Extract severity
            severity = 'medium'  # default
            if 'severity: high' in response_lower or 'severity:high' in response_lower:
                severity = 'high'
            elif 'severity: low' in response_lower or 'severity:low' in response_lower:
                severity = 'low'

            # Extract explanation - more sophisticated parsing
            explanation_match = re.search(r'explanation[:\s]+(.*)', response, re.IGNORECASE | re.DOTALL)
            explanation = explanation_match.group(1).strip() if explanation_match else response.strip()

            # Clean up explanation - remove any trailing status info
            explanation = re.sub(r'\n*STATUS:.*$', '', explanation, flags=re.IGNORECASE | re.DOTALL).strip()

            # Format reference citations with longer excerpts
            formatted_references = []
            for ref in reference_passages[:3]:
                formatted_references.append({
                    'filename': ref['metadata'].get('filename', 'Unknown'),
                    'page_num': ref['metadata'].get('page_num', 'N/A'),
                    'collection': reference_collection,
                    'text_snippet': ref['text'][:600] + '...' if len(ref['text']) > 600 else ref['text'],
                    'full_text': ref['text']  # Store full text for detailed output
                })

            return {
                'has_misalignment': True,
                'claim_id': claim['id'],
                'claim_text': claim.get('full_text', claim['text'])[:1000],  # Use longer version
                'claim_source': {
                    'filename': target_doc,
                    'page_num': claim['metadata'].get('page_num', 'N/A'),
                    'chunk_id': claim['metadata'].get('chunk_id', 'N/A')
                },
                'misalignment_type': 'regulatory_conflict',
                'severity': severity,
                'explanation': explanation,
                'references': formatted_references
            }

        except Exception as e:
            print(f"Error in alignment analysis: {e}")
            return {'has_misalignment': False}

    def _calculate_confidence_score(self, misalignments: List[Dict], total_claims: int) -> float:
        """Calculate confidence score for the overall analysis"""

        if total_claims == 0:
            return 0.0

        # Base confidence on coverage and consistency
        coverage_score = min(total_claims / 15, 1.0)  # More claims = higher confidence

        # Higher confidence if we found issues (means we did thorough analysis)
        detection_score = 0.7 if len(misalignments) > 0 else 0.5

        # Consider severity distribution
        if misalignments:
            severity_scores = {
                'high': 0.9,
                'medium': 0.7,
                'low': 0.6
            }
            avg_severity_confidence = np.mean([
                severity_scores.get(m.get('severity', 'medium'), 0.7)
                for m in misalignments
            ])
        else:
            avg_severity_confidence = 0.8  # High confidence if no issues found

        # Combine scores
        overall_confidence = (coverage_score * 0.3 + detection_score * 0.3 + avg_severity_confidence * 0.4)

        return round(overall_confidence, 3)

    def _format_comparison_report(self, target_doc: str, reference_collection: str,
                                 overview: str, misalignments: List[Dict],
                                 confidence_score: float, total_claims_analyzed: int) -> Dict[str, Any]:
        """Format the final comparison report"""

        # Build report structure
        report = {
            'title': f"Cross-Reference Comparison: {target_doc} vs {reference_collection}",
            'target_document': target_doc,
            'reference_collection': reference_collection,
            'overview': overview,
            'analysis_summary': {
                'total_sections_analyzed': total_claims_analyzed,
                'misalignments_found': len(misalignments),
                'confidence_score': confidence_score,
                'confidence_level': 'High' if confidence_score > 0.75 else 'Medium' if confidence_score > 0.5 else 'Low'
            },
            'results': []
        }

        # Add misalignments if found
        if misalignments:
            report['results'] = [{
                'section': f"Misalignment #{i+1}",
                'severity': m.get('severity', 'medium').upper(),
                'target_location': {
                    'filename': m['claim_source']['filename'],
                    'page': m['claim_source']['page_num'],
                    'chunk_id': m['claim_source']['chunk_id']
                },
                'target_statement': m['claim_text'],
                'explanation': m['explanation'],
                'reference_citations': [
                    {
                        'filename': ref['filename'],
                        'page': ref['page_num'],
                        'collection': ref['collection'],
                        'excerpt': ref['text_snippet']
                    }
                    for ref in m.get('references', [])
                ]
            } for i, m in enumerate(misalignments)]
        else:
            report['results'] = [{
                'section': 'Overall Assessment',
                'message': 'No areas of misalignment were detected',
                'details': f'Analyzed {total_claims_analyzed} sections from {target_doc}. All content appears to align with {reference_collection}.'
            }]

        return report

    def detect_query_intent(self, query: str) -> Dict[str, Any]:
        """
        Detect if query is requesting cross-reference comparison.
        Returns parsed intent with target document and reference collection.
        """
        query_lower = query.lower()

        # Comparison action verbs
        comparison_verbs = [
            'compare', 'check', 'analyze', 'review', 'cross-reference',
            'cross reference', 'identify', 'validate', 'find', 'audit',
            'verify', 'assess', 'evaluate', 'examine'
        ]

        # Comparison indicators
        comparison_indicators = [
            'against', 'with', 'vs', 'versus', 'compliance', 'align',
            'discrepanc', 'conflict', 'violation', 'misalign', 'issue'
        ]

        # Check for comparison intent
        has_comparison_verb = any(verb in query_lower for verb in comparison_verbs)
        has_comparison_indicator = any(indicator in query_lower for indicator in comparison_indicators)

        if not (has_comparison_verb or has_comparison_indicator):
            return {'is_comparison_query': False}

        # Extract target document and reference collection using patterns
        intent = {
            'is_comparison_query': True,
            'target_document': None,
            'reference_collection': None,
            'original_query': query
        }

        # Pattern: "compare X against Y" or "check X for compliance with Y"
        patterns = [
            r'(?:compare|check|analyze|review)\s+([^\s]+(?:\s+[^\s]+){0,5}?)\s+(?:against|with|for\s+compliance\s+with|using)\s+(?:the\s+)?([^\s]+(?:\s+[^\s]+){0,3})',
            r'(?:does|is)\s+([^\s]+(?:\s+[^\s]+){0,5}?)\s+(?:align|comply|conform)\s+(?:with|to)\s+(?:the\s+)?([^\s]+(?:\s+[^\s]+){0,3})',
            r'(?:identify|find|flag)\s+(?:any\s+)?(?:discrepancies|conflicts|violations|issues|misalignments?)\s+(?:in|between)\s+([^\s]+(?:\s+[^\s]+){0,5}?)\s+(?:and|against|with)\s+(?:the\s+)?([^\s]+(?:\s+[^\s]+){0,3})'
        ]

        for pattern in patterns:
            match = re.search(pattern, query_lower)
            if match:
                intent['target_document'] = match.group(1).strip()
                intent['reference_collection'] = match.group(2).strip()
                break

        # Try to map reference collection to actual collection names
        if intent['reference_collection']:
            intent['reference_collection'] = self._map_to_collection_name(intent['reference_collection'])

        return intent

    def _map_to_collection_name(self, reference_text: str) -> str:
        """Map natural language reference to actual collection name"""

        reference_lower = reference_text.lower()

        # Common mappings - map to actual collection names in database
        mappings = {
            'pra': 'pra_rules',
            'rules': 'pra_rules',
            'rulebook': 'pra_rules',
            'regulations': 'pra_rules',
            'regulatory': 'pra_rules',
            'basel': 'pra_rules',
            'fca': 'pra_rules',
            'earnings': 'earnings',
            'transcript': 'earnings',
            'meeting': 'earnings',
            'analyst': 'earnings'
        }

        for key, collection in mappings.items():
            if key in reference_lower:
                return collection

        return reference_text

print("✅ Cross-Reference Comparison System initialized!")
print("   - Document-to-collection comparison")
print("   - Misalignment detection with severity scoring")
print("   - Query intent detection for comparison requests")
print("   - Confidence-scored analysis reports")

✅ Cross-Reference Comparison System initialized!
   - Document-to-collection comparison
   - Misalignment detection with severity scoring
   - Query intent detection for comparison requests
   - Confidence-scored analysis reports


#### 7.1 Enhanced Metadata Search and Summarization Systems

In [58]:
# Enhanced Metadata Search and Summarization Systems
class EnhancedMetadataSearch:
    """Advanced metadata querying and database operations with LLM-powered intent parsing"""

    def __init__(self, collections_manager, llm_engine=None):
        self.manager = collections_manager
        self.llm_engine = llm_engine

    def execute_metadata_query(self, query: str, collections: List[str]) -> Dict[str, Any]:
        """Execute complex metadata queries with LLM-powered flexible pattern matching"""
        query_lower = query.lower()

        # Count queries - flexible patterns
        count_patterns = ['count', 'how many', 'number of', 'total']
        if any(pattern in query_lower for pattern in count_patterns):
            return self._count_files_query(query, collections)

        # List/show/display queries - flexible patterns
        list_patterns = ['list', 'show', 'display', 'get', 'give me', 'tell me', 'what are']
        file_patterns = ['file', 'document', 'pdf', 'title', 'name']
        if any(list_pattern in query_lower for list_pattern in list_patterns) and \
           any(file_pattern in query_lower for file_pattern in file_patterns):
            return self._list_files_query(query, collections)

        # Temporal queries - flexible patterns
        temporal_patterns = ['year', 'quarter', 'date', 'when', 'time', 'period', '20', '19']
        if any(pattern in query_lower for pattern in temporal_patterns):
            return self._temporal_analysis_query(query, collections)

        # Collection info queries
        collection_patterns = ['collection', 'collections', 'available', 'exist', 'database']
        if any(pattern in query_lower for pattern in collection_patterns):
            return self._general_metadata_query(query, collections)

        # Default to general metadata query
        return self._general_metadata_query(query, collections)

    def _parse_query_intent_with_llm(self, query: str) -> Dict[str, Any]:
        """Use LLM to parse query intent for flexible understanding"""
        if not self.llm_engine:
            return self._parse_query_intent_heuristic(query)

        parse_prompt = f"""Analyze this file search query and extract search parameters in JSON format.

Query: "{query}"

Extract the following parameters (use null if not specified):
- limit: integer (number of files to return, default: 10)
- sort_by: "newest", "oldest", "first", "last", or null (database order)
- year_filter: string (any year mentioned, e.g., "2022", "2024", "22", "24")
- keyword_filter: string (any keyword to search for in filenames)
- title_filter: string (specific document title to match - DO NOT extract collection names like "earnings collection" or "pra rules"!)
- quarter_filter: string (quarter like "Q1", "1Q", "Q4", "4Q")

IMPORTANT SORTING RULES:
- "newest", "oldest", "most recent", "latest" → sort_by: "newest" or "oldest"
- "first", "beginning", "start" → sort_by: "first"
- "last", "end", "final" → sort_by: "last"

Year format handling:
- "2022" → year_filter: "2022"
- "22" → year_filter: "22" (will be interpreted as 2022)
- "from 2024" → year_filter: "2024"

Examples:
Query: "list first 3 files from 2022"
{{"limit": 3, "sort_by": "first", "year_filter": "2022", "keyword_filter": null, "title_filter": null, "quarter_filter": null}}

Query: "show 5 newest earnings documents"
{{"limit": 5, "sort_by": "newest", "year_filter": null, "keyword_filter": "earnings", "title_filter": null, "quarter_filter": null}}

Query: "get last 2 files from Q1 2024"
{{"limit": 2, "sort_by": "last", "year_filter": "2024", "keyword_filter": null, "title_filter": null, "quarter_filter": "Q1"}}

Query: "first file from 22"
{{"limit": 1, "sort_by": "first", "year_filter": "22", "keyword_filter": null, "title_filter": null, "quarter_filter": null}}

Query: "list all files from 2020 in earnings collection"
{{"limit": 10, "sort_by": null, "year_filter": "2020", "keyword_filter": null, "title_filter": null, "quarter_filter": null}}

Now parse the query and respond ONLY with JSON:"""

        try:
            response = self.llm_engine.generate_response(parse_prompt, temperature=0.0)
            # Extract JSON from response
            import json
            # Find JSON in the response
            json_match = re.search(r'\{[^{}]*\}', response, re.DOTALL)
            if json_match:
                parsed = json.loads(json_match.group(0))
                return parsed
            else:
                return self._parse_query_intent_heuristic(query)
        except Exception as e:
            print(f"LLM parsing failed: {e}, falling back to heuristic")
            return self._parse_query_intent_heuristic(query)

    def _parse_query_intent_heuristic(self, query: str) -> Dict[str, Any]:
        """Fallback heuristic-based query parsing with UNIVERSAL pattern matching"""
        query_lower = query.lower()

        result = {
            'limit': None,
            'sort_by': None,
            'year_filter': None,
            'keyword_filter': None,
            'title_filter': None,
            'quarter_filter': None
        }

        # 1. Extract NUMBER (limit) - match ANY number
        number_patterns = [
            r'\b(\d+)\s+(?:files?|documents?|pdfs?|items?)\b',
            r'\b(?:first|last|show|list|display|get|give)\s+(\d+)\b',
            r'\b(\d+)\s+(?:oldest|newest|most\s+recent|latest)\b'
        ]
        for pattern in number_patterns:
            match = re.search(pattern, query_lower)
            if match:
                result['limit'] = int(match.group(1))
                break

        # Handle singular file references
        if not result['limit'] and re.search(r'\b(first|last|the|one|a|an)\s+(?:file|document|pdf)\b', query_lower):
            result['limit'] = 1

        # Default limit
        if result['limit'] is None:
            result['limit'] = 10

        # 2. Detect SORTING - newest/oldest (date-based) takes priority over first/last (position-based)
        if re.search(r'\b(newest|most\s+recent|latest)\b', query_lower):
            result['sort_by'] = 'newest'
        elif re.search(r'\b(oldest|earliest)\b', query_lower):
            result['sort_by'] = 'oldest'
        elif re.search(r'\b(first|start|beginning)\b', query_lower):
            result['sort_by'] = 'first'
        elif re.search(r'\b(last|end|final)\b', query_lower):
            result['sort_by'] = 'last'

        # 3. Extract YEAR - UNIVERSAL pattern to match ANY year (2-digit or 4-digit)
        # Pattern: any 2-4 consecutive digits that could be a year
        year_matches = []

        # Look for 4-digit years (1900-2099)
        four_digit_years = re.findall(r'\b(19\d{2}|20\d{2})\b', query)
        year_matches.extend(four_digit_years)

        # Look for 2-digit years in various contexts
        # Pattern 1: Standalone 2-digit (with word boundaries or after "from", "for", "in")
        two_digit_pattern = r'\b(?:from|for|in|year)?\s*(\d{2})(?!\d)\b'
        two_digit_matches = re.findall(two_digit_pattern, query_lower)
        year_matches.extend(two_digit_matches)

        # Pattern 2: In quarter format (Q1 24, 1Q24, etc.)
        quarter_year_pattern = r'\b[Qq]?\d[Qq]?(\d{2})\b'
        quarter_year_matches = re.findall(quarter_year_pattern, query)
        year_matches.extend(quarter_year_matches)

        if year_matches:
            # Use the first year found
            year_str = year_matches[0]
            # Normalize 2-digit to 4-digit
            if len(year_str) == 2:
                year_int = int(year_str)
                # Smart century detection: 00-50 → 2000s, 51-99 → 1900s
                result['year_filter'] = str(2000 + year_int if year_int <= 50 else 1900 + year_int)
            else:
                result['year_filter'] = year_str

        # 4. Extract QUARTER - UNIVERSAL pattern (FIXED to avoid false positives)
        # IMPORTANT: Only match if there's a Q prefix/suffix or explicit "quarter" word
        # This prevents "first 3" from being interpreted as Q3
        quarter_patterns = [
            r'\b[Qq]([1-4])\b',  # Q1, q1, Q2, etc. (Q prefix)
            r'\b([1-4])[Qq]\b',  # 1Q, 2Q, etc. (Q suffix)
            r'\b(first|second|third|fourth)\s+quarter\b'  # "first quarter", etc.
        ]
        for pattern in quarter_patterns:
            match = re.search(pattern, query_lower)
            if match:
                q_str = match.group(1)
                # Normalize to Q1, Q2, Q3, Q4 format
                if 'first' in q_str:
                    result['quarter_filter'] = 'Q1'
                elif 'second' in q_str:
                    result['quarter_filter'] = 'Q2'
                elif 'third' in q_str:
                    result['quarter_filter'] = 'Q3'
                elif 'fourth' in q_str:
                    result['quarter_filter'] = 'Q4'
                elif q_str.isdigit():  # If it's just a digit (from Q1 or 1Q pattern)
                    result['quarter_filter'] = f"Q{q_str}"
                break

        # 5. Extract KEYWORD - look for content words after common prepositions
        keyword_patterns = [
            r'containing\s+["\']?([^"\']+?)["\']?(?:\s+(?:in|from|for|$))',
            r'with\s+["\']?([^"\']+?)["\']?(?:\s+(?:in|from|for|$))',
            r'matching\s+["\']?([^"\']+?)["\']?(?:\s+(?:in|from|for|$))',
            r'about\s+["\']?([^"\']+?)["\']?(?:\s+(?:in|from|for|$))',
            r'(?:files?|documents?|pdfs?)\s+(?:on|about|for|regarding)\s+([a-zA-Z]+)'
        ]
        for pattern in keyword_patterns:
            match = re.search(pattern, query_lower)
            if match:
                keyword = match.group(1).strip()
                # Clean up
                keyword = re.sub(r'\b(in|the|each|collection|database|files?|documents?|pdfs?)\b', '', keyword).strip()
                if len(keyword) > 2:
                    result['keyword_filter'] = keyword
                break

        # 6. Extract TITLE filter
        if 'title' in query_lower or 'named' in query_lower or 'called' in query_lower:
            title_match = re.search(r'(?:title|named|called)\s+["\']?([^"\']+)["\']?', query_lower)
            if title_match:
                result['title_filter'] = title_match.group(1).strip()

        return result

    def _count_files_query(self, query: str, collections: List[str]) -> Dict[str, Any]:
        """Handle file counting queries"""
        results = {}
        total_files = 0

        for collection_name in collections:
            collection = self.manager.get_collection(collection_name)
            if not collection:
                continue

            try:
                data = collection.get(include=['metadatas'])
                filenames = set()

                for metadata in data.get('metadatas', []):
                    if metadata and metadata.get('filename'):
                        filenames.add(metadata['filename'])

                results[collection_name] = len(filenames)
                total_files += len(filenames)
            except Exception as e:
                results[collection_name] = f'Error: {str(e)}'

        # Build answer
        answer_parts = [f"File count by collection:\n"]
        for collection_name, count in results.items():
            if isinstance(count, int):
                answer_parts.append(f"  • {collection_name}: {count} file(s)")
            else:
                answer_parts.append(f"  • {collection_name}: {count}")

        answer_parts.append(f"\nTotal unique files: {total_files}")
        answer = "\n".join(answer_parts)

        return {
            'query': query,
            'search_type': 'Metadata Count Query',
            'answer': answer,
            'references': [],
            'confidence': 1.0,
            'collections_searched': collections,
            'file_counts': results
        }

    def _list_files_query(self, query: str, collections: List[str]) -> Dict[str, Any]:
        """Handle intelligent file listing with LLM-powered or heuristic intent parsing"""

        # Parse query intent using LLM or heuristics
        intent = self._parse_query_intent_with_llm(query)

        # Sanitize title_filter BEFORE extracting: ignore if it's a collection name or generic term
        if intent.get('title_filter'):
            ignore_terms = ['collection', 'database', 'files', 'documents', 'earnings collection', 'pra rules', 'pra_rules', 'earnings_transcripts']
            if any(term in intent['title_filter'].lower() for term in ignore_terms):
                intent['title_filter'] = None

        limit = intent.get('limit', 10)
        sort_by = intent.get('sort_by')  # 'newest', 'oldest', 'first', 'last', or None
        year_filter = intent.get('year_filter')
        keyword_filter = intent.get('keyword_filter')
        title_filter = intent.get('title_filter')
        quarter_filter = intent.get('quarter_filter')

        # Process each collection
        results = {}

        for collection_name in collections:
            collection = self.manager.get_collection(collection_name)
            if not collection:
                continue

            try:
                # Get all files from collection (increased limit for filtering)
                data = collection.get(include=['metadatas'], limit=5000)
                file_metadata = {}

                # Build unique files with metadata
                for metadata in data.get('metadatas', []):
                    if not metadata or not metadata.get('filename'):
                        continue

                    filename = metadata['filename']
                    if filename not in file_metadata:
                        file_metadata[filename] = metadata

                filenames = list(file_metadata.keys())

                # === FILTERING PHASE ===

                # Apply keyword filter (UNIVERSAL: matches any keyword in filename)
                if keyword_filter:
                    filenames = [f for f in filenames if keyword_filter.lower() in f.lower()]

                # Apply year filter (UNIVERSAL: matches year in any format in filename)
                # FIXED: Removed \b word boundary that was preventing matches
                if year_filter:
                    # Get 2-digit and 4-digit versions of year
                    year_4digit = year_filter if len(year_filter) == 4 else str(2000 + int(year_filter))
                    year_2digit = year_4digit[-2:]

                    filtered = []
                    for f in filenames:
                        # Match 4-digit year anywhere in filename
                        if year_4digit in f:
                            filtered.append(f)
                        # Match 2-digit year in quarter format (2Q22, Q222, Q2 22, etc.)
                        elif re.search(rf'[Qq]?\d[Qq]?{year_2digit}', f):
                            filtered.append(f)
                        # Match 2-digit year with separators (_22_, -22-, .22.)
                        elif re.search(rf'[_\-\.]{year_2digit}[_\-\.]', f):
                            filtered.append(f)
                        # Match 2-digit at end before extension
                        elif re.search(rf'_{year_2digit}\.', f):
                            filtered.append(f)

                    filenames = filtered

                # Apply quarter filter (UNIVERSAL: matches quarter in any format)
                if quarter_filter:
                    # Normalize quarter to digit (Q1 → 1)
                    q_digit = quarter_filter[1]  # Extract digit from Q1, Q2, etc.

                    filtered = []
                    for f in filenames:
                        # Match various quarter formats: Q1, 1Q, q1, 1q
                        if re.search(rf'[Qq]?{q_digit}[Qq]?', f):
                            filtered.append(f)

                    filenames = filtered

                # Apply title filter (UNIVERSAL: matches any title substring)
                if title_filter:
                    filenames = [f for f in filenames if title_filter.lower() in f.lower()]

                # === SORTING PHASE ===

                if sort_by in ['newest', 'oldest']:
                    # Date-based sorting (by extracting dates from filenames)
                    def extract_date_key(filename):
                        # Try to extract year and quarter for sorting
                        # Pattern 1: Quarter + Year (1Q24, Q1 2024, etc.)
                        quarter_year_match = re.search(r'(\d)[Qq](\d{2})', filename)
                        if quarter_year_match:
                            q, yr = quarter_year_match.groups()
                            year = 2000 + int(yr) if int(yr) < 50 else 1900 + int(yr)
                            return year * 10 + int(q)  # YYYYQ format for sorting

                        # Pattern 2: 4-digit year
                        year_match = re.search(r'(19\d{2}|20\d{2})', filename)
                        if year_match:
                            return int(year_match.group(1)) * 10

                        # Pattern 3: 2-digit year (convert to 4-digit)
                        two_digit_year = re.search(r'[_\-](\d{2})[_\-\.]', filename)
                        if two_digit_year:
                            yr = int(two_digit_year.group(1))
                            year = 2000 + yr if yr <= 50 else 1900 + yr
                            return year * 10

                        # No date found - put at start or end based on sort direction
                        return 0 if sort_by == 'oldest' else 999999

                    filenames.sort(key=extract_date_key, reverse=(sort_by == 'newest'))

                elif sort_by == 'last':
                    # Position-based: reverse for "last"
                    filenames.reverse()

                # sort_by == 'first' or None: keep original database order

                # === LIMITING PHASE ===
                filenames = filenames[:limit]
                results[collection_name] = filenames

            except Exception as e:
                results[collection_name] = [f'Error: {str(e)}']

        # === BUILD ANSWER ===
        answer_parts = []

        # Create descriptor for what was returned
        descriptors = []
        if limit:
            descriptors.append(f"{limit}")
        if sort_by:
            descriptors.append(sort_by)
        if year_filter:
            # Show original year format from query
            descriptors.append(f"from {year_filter}")
        if quarter_filter:
            descriptors.append(quarter_filter)
        if keyword_filter:
            descriptors.append(f"with '{keyword_filter}'")
        if title_filter:
            descriptors.append(f"titled '{title_filter}'")

        descriptor = " ".join(descriptors) if descriptors else "files"

        for collection_name, filenames in results.items():
            answer_parts.append(f"\n{collection_name} ({descriptor}):")
            if isinstance(filenames, list) and filenames and not filenames[0].startswith('Error'):
                for i, filename in enumerate(filenames, 1):
                    answer_parts.append(f"  {i}. {filename}")
            elif isinstance(filenames, list) and filenames:
                answer_parts.append(f"  {filenames[0]}")  # Error message
            else:
                answer_parts.append(f"  No files found matching criteria")

        answer = "\n".join(answer_parts)

        return {
            'query': query,
            'search_type': 'File List Query',
            'answer': answer,
            'references': [],
            'confidence': 1.0,
            'collections_searched': collections,
            'files_listed': results,
            'parsed_intent': intent  # Include for debugging
        }

    def _temporal_analysis_query(self, query: str, collections: List[str]) -> Dict[str, Any]:
        """Handle temporal analysis queries"""
        temporal_data = {}

        for collection_name in collections:
            collection = self.manager.get_collection(collection_name)
            if not collection:
                continue

            try:
                data = collection.get(include=['metadatas'])
                years = {}
                quarters = {}

                for metadata in data.get('metadatas', []):
                    if not metadata:
                        continue

                    # Extract year and quarter
                    filename = metadata.get('filename', '')

                    # Look for year patterns
                    year_match = re.search(r'(19|20)\d{2}', filename)
                    if year_match:
                        year = year_match.group(0)
                        years[year] = years.get(year, 0) + 1

                    # Look for quarter patterns
                    quarter_match = re.search(r'(\dQ\d{2})', filename)
                    if quarter_match:
                        quarter = quarter_match.group(1)
                        quarters[quarter] = quarters.get(quarter, 0) + 1

                temporal_data[collection_name] = {
                    'years': years,
                    'quarters': quarters
                }
            except Exception as e:
                temporal_data[collection_name] = {'error': str(e)}

        # Build answer
        answer_parts = ["Temporal Analysis:\n"]

        for collection_name, data in temporal_data.items():
            answer_parts.append(f"\n{collection_name}:")

            if 'error' in data:
                answer_parts.append(f"  Error: {data['error']}")
            else:

                if data['years']:
                    sorted_years = dict(sorted(data['years'].items()))
                    year_summary = [f"{year}: {count} doc(s)" for year, count in sorted_years.items()]
                    answer_parts.append(f"  Years: {', '.join(year_summary)}")

                if data['quarters']:
                    sorted_quarters = dict(sorted(data['quarters'].items()))
                    quarter_summary = [f"{q}: {count} doc(s)" for q, count in sorted_quarters.items()]
                    answer_parts.append(f"  Quarters: {', '.join(quarter_summary)}")

                if not data['years'] and not data['quarters']:
                    answer_parts.append("  No temporal metadata available")

        answer = "\n".join(answer_parts)

        return {
            'query': query,
            'search_type': 'Temporal Analysis Query',
            'answer': answer,
            'references': [],
            'confidence': 1.0,
            'collections_searched': collections,
            'temporal_data': temporal_data
        }

    def _general_metadata_query(self, query: str, collections: List[str]) -> Dict[str, Any]:
        """Handle general metadata queries - fallback for unmatched patterns"""
        query_lower = query.lower()

        # Collect general metadata from all collections
        metadata_summary = {}
        total_documents = 0
        all_metadata_fields = set()

        for collection_name in collections:
            collection = self.manager.get_collection(collection_name)
            if not collection:
                continue

            try:
                data = collection.get(include=['metadatas'])
                metadatas = data.get('metadatas', [])

                # Collect unique files
                unique_files = set()
                metadata_fields = set()

                for metadata in metadatas:
                    if not metadata:
                        continue

                    if metadata.get('filename'):
                        unique_files.add(metadata['filename'])

                    # Track all metadata fields
                    metadata_fields.update(metadata.keys())
                    all_metadata_fields.update(metadata.keys())

                metadata_summary[collection_name] = {
                    'total_chunks': len(metadatas),
                    'unique_files': len(unique_files),
                    'metadata_fields': sorted(metadata_fields)
                }
                total_documents += len(unique_files)

            except Exception as e:
                metadata_summary[collection_name] = {'error': str(e)}

        # Build informative answer
        answer_parts = ["Database Metadata Summary:\n"]
        answer_parts.append(f"Total unique documents: {total_documents}")
        answer_parts.append(f"Collections searched: {len(collections)}\n")

        for collection_name, data in metadata_summary.items():
            if 'error' in data:
                answer_parts.append(f"\n{collection_name}: Error - {data['error']}")
            else:
                answer_parts.append(f"\n{collection_name}:")
                answer_parts.append(f"  • Unique files: {data['unique_files']}")
                answer_parts.append(f"  • Total chunks: {data['total_chunks']}")
                answer_parts.append(f"  • Available metadata: {', '.join(data['metadata_fields'])}")

        answer_parts.append("\n\n📝 Available Query Types:")
        answer_parts.append("  ✓ Listing: 'list first 3 files', 'show first file', 'display last 5 files'")
        answer_parts.append("  ✓ Counting: 'count files', 'how many documents'")
        answer_parts.append("  ✓ Sorting: 'newest files', 'oldest documents', 'most recent', 'last file'")
        answer_parts.append("  ✓ Filtering: 'files from 2020', 'files from 22', 'documents containing earnings', 'Q1 files'")
        answer_parts.append("  ✓ Temporal: 'show files by year', 'count by quarter'")

        answer = "\n".join(answer_parts)

        return {
            'query': query,
            'search_type': 'General Metadata Query',
            'answer': answer,
            'references': [],
            'confidence': 0.8,
            'collections_searched': collections,
            'metadata_summary': metadata_summary
        }

# Enhanced Summarizer - IMPROVED with structured, professional formatting
class EnhancedSummarizer:
    """Enhanced summarization with structured output, proper formatting, and precise citations"""

    def __init__(self, llm_engine, citation_system):
        self.llm_engine = llm_engine
        self.citation_system = citation_system

    def _extract_query_focus(self, query: str, filename: str) -> str:
        """Extract the specific topic focus from query"""
        query_lower = query.lower()
        # Remove summarization keywords
        focus = re.sub(r'\b(summarize|summary|summarise|overview|review|focusing|on|about)\b', '', query_lower, flags=re.IGNORECASE)
        # Remove filename
        focus = re.sub(r'\b' + re.escape(filename.lower()) + r'\b', '', focus)
        # Remove word count
        focus = re.sub(r'\b\d+\s*words?\b', '', focus, flags=re.IGNORECASE)
        # Clean up
        focus = ' '.join(focus.split()).strip()

        if len(focus) < 3:
            return "all key information and main points"
        return focus

    def generate_summary(self, references: List[Dict], query: str, target_words: int = 300) -> Dict[str, Any]:
        """Generate structured, professional summary with consistent formatting"""

        if not references:
            return {
                'summary': 'No documents found to summarize.',
                'word_count': 0,
                'citations_used': [],
                'confidence': 0.0,
                'sources_count': 0
            }

        # Get filename and extract query focus
        filename = references[0].get('metadata', {}).get('filename', 'document')
        query_focus = self._extract_query_focus(query, filename)

        # Build unique page citations (one per page)
        page_citations = {}
        for i, ref in enumerate(references[:15]):
            if 'text' not in ref or not ref.get('text'):
                continue

            metadata = ref.get('metadata', {})
            page = metadata.get('page_num', metadata.get('page', 'N/A'))

            # Only one citation per unique page
            if page not in page_citations:
                page_citations[page] = {
                    'id': f'cite_{len(page_citations)+1}',
                    'filename': filename,
                    'page': page,
                    'text': ref['text'],
                    'collection': ref.get('collection', metadata.get('collection', 'Unknown'))
                }

        citations = list(page_citations.values())

        # Prepare context with page references
        context_parts = []
        for citation in citations:
            page_ref = f" [Page {citation['page']}]" if citation['page'] != 'N/A' else ""
            context_parts.append(f"[{citation['id']}{page_ref}]: {citation['text'][:800]}")

        context = "\n\n".join(context_parts)

        # Create PROFESSIONAL STRUCTURED SUMMARY PROMPT
        prompt = f"""You are a professional financial analyst creating an executive summary. Create a STRUCTURED, PROFESSIONAL summary following this EXACT format:

CONTEXT WITH PAGE REFERENCES:
{context[:4500]}

DOCUMENT: {filename}
TOPIC FOCUS: {query_focus}
TARGET LENGTH: {target_words} words total

===== MANDATORY OUTPUT FORMAT =====

DOCUMENT SUMMARY: {filename}

EXECUTIVE OVERVIEW (3-4 sentences):
[Provide a concise overview of the document's main purpose, scope, and key themes. Each sentence must include a citation in the format [cite_X, Page Y].]

KEY FINDINGS:
• [First major finding with specific details and metrics] [cite_X, Page Y]
• [Second major finding with specific details and metrics] [cite_X, Page Y]
• [Third major finding with specific details and metrics] [cite_X, Page Y]
• [Fourth major finding - include as many as relevant to reach target length] [cite_X, Page Y]

[Additional sections if content warrants:]

STRATEGIC IMPLICATIONS / RECOMMENDATIONS (if present):
• [Key strategic point or recommendation] [cite_X, Page Y]
• [Additional strategic points] [cite_X, Page Y]

METHODOLOGY / APPROACH (if discussed in document):
• [Key methodological point] [cite_X, Page Y]

===== END FORMAT =====

CRITICAL REQUIREMENTS:
1. Use the EXACT structure shown above with clear section headers
2. Focus content on: {query_focus}
3. EVERY statement MUST have a citation: [cite_X, Page Y]
4. Use bullet points (•) for all findings and details
5. Executive overview should be 3-4 complete sentences
6. Each finding should be substantive (15-30 words minimum per bullet)
7. Target {target_words} words total across all sections
8. Use ONLY information from context - NO external knowledge
9. Include specific metrics, numbers, dates, and names when available
10. Each bullet point must be self-contained and complete
11. Maintain professional tone throughout
12. Ensure proper citations for every factual claim

FORMATTING RULES:
- Section headers in CAPS with colon
- Bullet points with • symbol
- Citations as [cite_X, Page Y] at end of each statement
- Single blank line between sections
- No introduction like "Here is the summary"
- Start directly with "DOCUMENT SUMMARY: [filename]"

EXAMPLE:
DOCUMENT SUMMARY: Quarterly_Earnings_Report_Q2_2022.pdf

EXECUTIVE OVERVIEW (3-4 sentences):
The Q2 2022 earnings report presents strong financial performance with revenue growth of 15% year-over-year, reaching $2.5 billion [cite_1, Page 3]. The company achieved record operating margins of 22% through strategic cost optimization initiatives and operational improvements [cite_2, Page 5]. Key growth drivers included the digital services segment expansion and successful product launches in three new markets [cite_3, Page 8]. Management provided updated guidance for FY2022, raising revenue projections by 8% based on strong demand signals [cite_4, Page 12].

KEY FINDINGS:
• Total revenue reached $2.5 billion, representing 15% year-over-year growth driven primarily by 28% expansion in digital services and 12% increase in enterprise solutions [cite_1, Page 3]
• Operating margin improved to 22%, up from 19% in Q2 2021, attributed to cost optimization initiatives that reduced operational expenses by $45 million [cite_2, Page 5]
• Customer acquisition costs decreased by 12% to $185 per customer while retention rates increased to 94%, indicating improved efficiency in marketing spend [cite_3, Page 9]
• Cash flow from operations increased 18% to $420 million, supporting the announced share buyback program of $500 million over the next 12 months [cite_4, Page 11]

STRATEGIC IMPLICATIONS:
• Management plans to invest $200 million in AI and machine learning capabilities over the next 18 months to enhance product offerings [cite_5, Page 14]
• International expansion strategy targets three new markets in Southeast Asia with projected revenue contribution of $150 million by Q4 2023 [cite_6, Page 16]

NOW CREATE YOUR PROFESSIONAL SUMMARY FOLLOWING THIS EXACT FORMAT:"""

        summary_text = self.llm_engine.generate_response(prompt, temperature=0.0)
        actual_words = len(summary_text.split())

        # High confidence for summaries (92-98%)
        word_accuracy = max(0.8, 1.0 - min(abs(actual_words - target_words) / target_words, 0.2))
        page_coverage = min(len(citations) / 8, 1.0)
        query_bonus = 0.03 if len(query_focus) > 10 else 0.01

        overall_confidence = 0.90 + (word_accuracy * 0.05) + (page_coverage * 0.02) + query_bonus
        overall_confidence = min(overall_confidence, 0.98)

        return {
            'summary': summary_text,
            'word_count': actual_words,
            'target_words': target_words,
            'word_accuracy': word_accuracy,
            'citations_used': citations,  # Unique pages only
            'confidence': overall_confidence,  # 92-98%
            'sources_count': len(references),
            'unique_pages': len(citations),
            'query_focus': query_focus
        }

print("✅ Metadata Search and Summarization Systems initialized with STRUCTURED output formatting!")

✅ Metadata Search and Summarization Systems initialized with STRUCTURED output formatting!


#### 7.2 Reliability Enforcement System

In [59]:
# Reliability Enforcement System
class ReliabilityEnforcer:
    """Ensures response grounding and reliability"""

    def __init__(self, llm_engine):
        self.llm_engine = llm_engine

    def validate_response_grounding(self, query: str, answer: str, references: List[Dict]) -> Dict[str, Any]:
        """Validate that response is properly grounded in source materials"""

        if not references:
            return {
                'is_grounded': False,
                'grounding_score': 0.0,
                'issues': ['No source references provided'],
                'confidence': 0.0
            }

        # Extract key claims from the answer
        claims = self._extract_key_claims(answer)

        # Check grounding for each claim
        grounding_results = []
        for claim in claims:
            grounding_result = self._check_claim_grounding(claim, references)
            grounding_results.append(grounding_result)

        # Calculate overall grounding score
        grounding_scores = [r['grounding_score'] for r in grounding_results]
        overall_grounding = np.mean(grounding_scores) if grounding_scores else 0.0

        # Identify issues
        issues = []
        for result in grounding_results:
            if result['grounding_score'] < 0.5:
                issues.append(f"Weak grounding for: {result['claim'][:50]}...")

        is_grounded = overall_grounding > 0.6 and len(issues) == 0

        return {
            'is_grounded': is_grounded,
            'grounding_score': overall_grounding,
            'claim_analysis': grounding_results,
            'issues': issues,
            'confidence': min(overall_grounding, 1.0),
            'total_claims_analyzed': len(claims)
        }

    def _extract_key_claims(self, answer: str) -> List[str]:
        """Extract key factual claims from answer"""
        # Simple sentence-based extraction
        sentences = re.split(r'[.!?]+', answer)

        # Filter out short sentences and common phrases
        claims = []
        for sentence in sentences:
            sentence = sentence.strip()
            if (len(sentence) > 20 and
                not sentence.lower().startswith(('however', 'therefore', 'moreover', 'furthermore'))):
                claims.append(sentence)

        return claims[:5]  # Limit to top 5 claims

    def _check_claim_grounding(self, claim: str, references: List[Dict]) -> Dict[str, Any]:
        """Check if a specific claim is grounded in references"""

        # Simple text overlap analysis
        claim_words = set(claim.lower().split())

        max_overlap = 0
        best_reference = None

        for ref in references:
            # Validate ref has required 'text' field
            if 'text' not in ref or not ref.get('text'):
                continue

            ref_words = set(ref['text'].lower().split())
            overlap = len(claim_words.intersection(ref_words))
            overlap_ratio = overlap / len(claim_words) if claim_words else 0

            if overlap_ratio > max_overlap:
                max_overlap = overlap_ratio
                best_reference = ref

        return {
            'claim': claim,
            'grounding_score': max_overlap,
            'best_reference': best_reference.get('metadata', {}).get('filename', 'Unknown') if best_reference else None,
            'word_overlap_ratio': max_overlap
        }

print("✅ Reliability Enforcement System initialized!")

✅ Reliability Enforcement System initialized!


#### 7.3 Main Enhanced RAG System

In [60]:
# Main Enhanced RAG System
class SpecCompliantRAGSystem:
    """Enhanced RAG system meeting all specification requirements with integrated improvements"""

    def __init__(self, collections_manager, llm_engine, embedding_model, cross_encoder=None):
        self.collections_manager = collections_manager
        self.llm_engine = llm_engine
        self.embedding_model = embedding_model
        self.cross_encoder = cross_encoder or self._init_cross_encoder()

        # Initialize hybrid search engine first (needed by cross_reference_comparator)
        self.hybrid_search = self._init_hybrid_search()

        # Initialize enhanced components
        self.citation_system = EnhancedCitationSystem(collections_manager)
        self.cross_doc_analyzer = CrossDocumentAnalyzer(llm_engine, collections_manager)
        self.metadata_search = EnhancedMetadataSearch(collections_manager, llm_engine)
        self.enhanced_summarizer = EnhancedSummarizer(llm_engine, self.citation_system)
        self.reliability_enforcer = ReliabilityEnforcer(llm_engine)
        self.cross_reference_comparator = CrossReferenceComparator(llm_engine, collections_manager, embedding_model, self.hybrid_search)

        # Enhanced capabilities flag
        self.enhanced_mode = True

    def _init_cross_encoder(self):
        """Initialize cross-encoder if available"""
        try:
            from sentence_transformers import CrossEncoder
            return CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
        except:
            return None

    def _init_hybrid_search(self):
        """Initialize hybrid search engine"""
        try:
            # Use the global hybrid search if available
            if 'enhanced_hybrid_search' in globals():
                return globals()['enhanced_hybrid_search']
            else:
                # Create basic hybrid search
                return AdvancedHybridSearchEngine(self.collections_manager)
        except:
            return None

    def analyze_query_type(self, query: str) -> Dict[str, Any]:
        """Analyze query to determine processing approach - Enhanced with flexible patterns"""
        query_lower = query.lower()

        # Metadata/database query patterns - Very flexible to catch various metadata queries
        metadata_action_words = [
            'count', 'list', 'show', 'display', 'get', 'find', 'search for',
            'how many', 'what are', 'which', 'tell me', 'give me'
        ]

        metadata_target_words = [
            'file', 'files', 'document', 'documents', 'pdf', 'pdfs',
            'collection', 'collections', 'title', 'titles', 'name', 'names',
            'date', 'dates', 'year', 'years', 'quarter', 'quarters',
            'author', 'authors', 'source', 'sources', 'metadata',
            'available', 'exist', 'exists', 'database', 'stored'
        ]

        # Check for metadata query patterns
        has_action = any(action in query_lower for action in metadata_action_words)
        has_target = any(target in query_lower for target in metadata_target_words)

        # Additional patterns that strongly indicate metadata queries
        metadata_phrases = [
            'in the database', 'in each collection', 'per collection', 'from each',
            'all the', 'total', 'first', 'last', 'latest', 'oldest', 'newest',
            'by collection', 'by year', 'by date', 'by author', 'by title'
        ]
        has_metadata_phrase = any(phrase in query_lower for phrase in metadata_phrases)

        # If it's asking about metadata (not content), route to database query
        if (has_action and has_target) or has_metadata_phrase:
            # Additional check: avoid false positives for content questions
            content_indicators = [
                'about', 'regarding', 'explain', 'describe', 'what does',
                'why', 'how does', 'when did', 'where', 'details about'
            ]
            # If asking "what does X say about Y" it's content search, not metadata
            is_content_question = any(indicator in query_lower for indicator in content_indicators) and \
                                   not any(phrase in query_lower for phrase in ['what are the files', 'what files', 'which files'])

            if not is_content_question:
                return {
                    'type': 'database_query',
                    'confidence': 0.9,
                    'subtype': 'metadata_query'
                }

        # Cross-reference comparison patterns (check before summarization)
        comparison_intent = self.cross_reference_comparator.detect_query_intent(query)
        if comparison_intent['is_comparison_query']:
            return {
                'type': 'cross_reference_comparison',
                'confidence': 0.9,
                'subtype': 'document_comparison',
                'target_document': comparison_intent.get('target_document'),
                'reference_collection': comparison_intent.get('reference_collection'),
                'original_query': query
            }

        # Summarization patterns
        if any(pattern in query_lower for pattern in ['summarize', 'summary', 'overview']):
            # Extract filename if present
            filename_match = re.search(r'([a-zA-Z0-9_-]+\.pdf)', query)
            filename = filename_match.group(1) if filename_match else None

            # Extract word count
            word_match = re.search(r'(\d+)\s*words?', query_lower)
            word_count = int(word_match.group(1)) if word_match else 300

            return {
                'type': 'summarization',
                'confidence': 0.8,
                'filename': filename,
                'word_count': word_count,
                'subtype': 'document_summary' if filename else 'general_summary'
            }

        # Standard search (default)
        return {
            'type': 'search',
            'confidence': 0.7,
            'subtype': 'general_search'
        }

    def process_query(self, query: str, collections: List[str], use_reranking: bool = True) -> Dict[str, Any]:
        """Main query processing with enhanced routing and reliability checks"""

        if not collections:
            return self._create_error_response(query, 'No collections selected', collections)

        try:
            # Analyze query type
            analysis = self.analyze_query_type(query)

            # Route to enhanced handlers - pass use_reranking to search handler
            if analysis['type'] == 'database_query':
                result = self.metadata_search.execute_metadata_query(query, collections)

            elif analysis['type'] == 'cross_reference_comparison':
                result = self._handle_cross_reference_comparison(query, collections, analysis)

            elif analysis['type'] == 'summarization':
                result = self._handle_enhanced_summarization(query, collections, analysis)

            else:
                result = self._handle_enhanced_search(query, collections, analysis, use_reranking)

            # Apply reliability validation for content-based responses (EXCLUDE summaries and comparisons)
            if result.get('search_type') not in ['Metadata Count Query', 'Database Query', 'File List Query', 'Temporal Analysis Query', 'Enhanced Document Summarization', 'Cross-Reference Comparison']:
                validation = self.reliability_enforcer.validate_response_grounding(
                    query, result.get('answer', ''), result.get('references', [])
                )
                result['reliability_validation'] = validation

                if not validation['is_grounded']:
                    result['confidence'] *= 0.5
                    result['answer'] = f"⚠️ RELIABILITY WARNING: This response may not be fully grounded in the source documents.\n\n{result['answer']}"

            # Add system metadata
            result['enhanced_mode'] = self.enhanced_mode
            result['processing_timestamp'] = time.strftime('%Y-%m-%d %H:%M:%S')

            return result

        except Exception as e:
            return self._create_error_response(query, str(e), collections)

    def _handle_enhanced_search(self, query: str, collections: List[str], analysis: Dict, use_reranking: bool = True) -> Dict[str, Any]:
        """Enhanced search with cross-document analysis and precise citations"""

        if not self.hybrid_search:
            return self._create_error_response(query, 'Hybrid search not available', collections)

        # Ensure BM25 indices are ready
        self.hybrid_search.ensure_bm25_indices_loaded(collections)

        # Perform multi-collection search
        search_results = self.hybrid_search.multi_collection_search(query, collections, 15)

        if not search_results:
            return {
                'query': query,
                'search_type': 'Enhanced Search',
                'answer': 'No relevant information found in the selected collections.',
                'references': [],
                'confidence': 0.0,
                'collections_searched': collections,
                'search_method': 'Enhanced Hybrid RRF'
            }

        # Enhanced results processing with confidence filtering
        final_results = []
        processing_stats = {
            'total_found': len(search_results),
            'high_confidence': 0,
            'medium_confidence': 0,
            'low_confidence': 0,
            'avg_rrf_score': 0,
            'avg_final_score': 0,
            'cross_collection_matches': 0,
            'semantic_only': 0,
            'bm25_only': 0,
            'both_methods': 0,
            'reranking_applied': False
        }

        rrf_scores = []
        final_scores = []
        confidence_scores = []

        for result in search_results:
            # Enhanced confidence calculation
            confidence = result.get('confidence', 0)
            final_score = result.get('final_score', 0)
            rrf_score = result.get('rrf_score', 0)
            in_both_sets = result.get('in_both_sets', False)

            # Confidence categorization with enhanced thresholds
            if confidence >= 0.7:
                processing_stats['high_confidence'] += 1
                confidence_level = 'high'
            elif confidence >= 0.5:
                processing_stats['medium_confidence'] += 1
                confidence_level = 'medium'
            else:
                processing_stats['low_confidence'] += 1
                confidence_level = 'low'

            # Track search method statistics
            has_semantic = result.get('semantic_score', 0) > 0
            has_bm25 = result.get('bm25_score', 0) > 0

            if has_semantic and has_bm25:
                processing_stats['both_methods'] += 1
            elif has_semantic:
                processing_stats['semantic_only'] += 1
            elif has_bm25:
                processing_stats['bm25_only'] += 1

            # Track cross-collection matches
            if in_both_sets:
                processing_stats['cross_collection_matches'] += 1

            rrf_scores.append(rrf_score)
            final_scores.append(final_score)
            confidence_scores.append(confidence)

            # Apply enhanced confidence threshold (adaptive based on query complexity)
            min_confidence = 0.3 if len(query.split()) > 5 else 0.25

            if confidence >= min_confidence:
                # Add enhanced metadata to result
                result['confidence_level'] = confidence_level
                result['search_method_used'] = 'hybrid' if (has_semantic and has_bm25) else ('semantic' if has_semantic else 'bm25')
                result['quality_indicators'] = {
                    'high_rrf': rrf_score > 0.1,
                    'cross_method_match': in_both_sets,
                    'good_length': 100 <= len(result.get('text', '')) <= 1000,
                    'has_metadata': bool(result.get('metadata', {}).get('filename'))
                }
                final_results.append(result)

        # Calculate enhanced processing statistics
        processing_stats.update({
            'total_processed': len(final_results),
            'avg_rrf_score': np.mean(rrf_scores) if rrf_scores else 0,
            'avg_final_score': np.mean(final_scores) if final_scores else 0,
            'avg_confidence': np.mean(confidence_scores) if confidence_scores else 0,
            'confidence_distribution': {
                'high': processing_stats['high_confidence'],
                'medium': processing_stats['medium_confidence'],
                'low': processing_stats['low_confidence']
            },
            'method_distribution': {
                'both_methods': processing_stats['both_methods'],
                'semantic_only': processing_stats['semantic_only'],
                'bm25_only': processing_stats['bm25_only']
            },
            'fusion_effectiveness': {
                'cross_matches': processing_stats['cross_collection_matches'],
                'fusion_boost_applied': sum(1 for r in final_results if r.get('in_both_sets', False))
            }
        })

        # Limit final results with smart selection
        if len(final_results) > 10:
            final_results = self._ensure_result_diversity(final_results, 10)
        else:
            final_results = final_results[:10]

        # 🔧 Apply cross-encoder reranking if available and requested
        if use_reranking and self.cross_encoder and hasattr(self.cross_encoder, 'available') and self.cross_encoder.available:
            try:
                if len(final_results) > 1:
                    reranked_results = self.cross_encoder.rerank(query, final_results, len(final_results))
                    if reranked_results:
                        final_results = reranked_results
                        processing_stats['reranking_applied'] = True
            except Exception as e:
                print(f"Cross-encoder reranking failed: {e}")
                processing_stats['reranking_applied'] = False

        # Generate enhanced citations
        citations = self.citation_system.generate_precise_citations(final_results)

        # Perform cross-document analysis
        cross_analysis = self.cross_doc_analyzer.analyze_cross_document_patterns(final_results, query)

        # Generate contextual response
        answer = self._generate_enhanced_response(query, final_results, citations)

        # Build comprehensive references
        references = []
        for i, result in enumerate(final_results):
            # Validate result has required 'text' field
            if 'text' not in result or not result.get('text'):
                continue

            metadata = result.get('metadata', {})
            citation = citations[i] if i < len(citations) else {}

            ref = {
                'filename': metadata.get('filename', 'Unknown'),
                'collection': result.get('source_collection', metadata.get('collection', 'Unknown')),
                'chunk_id': metadata.get('chunk_id'),
                'text_snippet': result['text'][:200] + '...' if len(result['text']) > 200 else result['text'],
                'relevance_score': result.get('final_score', 0),
                'confidence': result.get('confidence', 0),
                'citation_id': citation.get('id', f'cite_{i+1}'),
                'verification_status': citation.get('verification_status', 'unverified')
            }
            references.append(ref)

        # Calculate overall confidence
        confidence_scores = [r.get('confidence', 0) for r in final_results[:5]]
        overall_confidence = np.mean(confidence_scores) if confidence_scores else 0.0

        return {
            'query': query,
            'search_type': 'Enhanced Hybrid Search',
            'answer': answer,
            'references': references,
            'confidence': overall_confidence,
            'collections_searched': collections,
            'chunks_analyzed': len(search_results),
            'cross_document_analysis': cross_analysis,
            'citations_generated': len(citations),
            'search_method': 'Enhanced RRF with Cross-Document Analysis',
            'processing_stats': processing_stats
        }

    def _handle_enhanced_summarization(self, query: str, collections: List[str], analysis: Dict) -> Dict[str, Any]:
        """Enhanced summarization with length control and citations"""

        filename = analysis.get('filename')
        word_count = analysis.get('word_count', 300)

        if filename:
            # Find specific document
            doc_finder = DocumentFinder(self.collections_manager)
            documents = doc_finder.find_documents_by_name(filename, collections)

            if not documents:
                return {
                    'query': query,
                    'search_type': 'Document Summarization',
                    'answer': f'Document "{filename}" not found in the selected collections.',
                    'references': [],
                    'confidence': 0.0,
                    'collections_searched': collections
                }

            # Use ALL document chunks for complete summarization
            search_results = documents  # Use entire document, not just 10 chunks

        else:
            # General topic summarization - search for relevant content
            if not self.hybrid_search:
                return self._create_error_response(query, 'Hybrid search not available', collections)

            # Extract topic from summarization query
            topic_query = re.sub(r'\bsummariz\w*\b', '', query, flags=re.IGNORECASE)
            topic_query = re.sub(r'\b\d+\s*words?\b', '', topic_query, flags=re.IGNORECASE).strip()

            if not topic_query:
                topic_query = "main topics and key information"

            search_results = self.hybrid_search.multi_collection_search(topic_query, collections, 12)

        if not search_results:
            return {
                'query': query,
                'search_type': 'Summarization',
                'answer': 'No content found to summarize.',
                'references': [],
                'confidence': 0.0,
                'collections_searched': collections
            }

        # Generate enhanced summary
        summary_result = self.enhanced_summarizer.generate_summary(search_results, query, word_count)

        # Build SINGLE document reference with page range
        filename = search_results[0].get('metadata', {}).get('filename', 'Unknown') if search_results else 'Unknown'
        collection = search_results[0].get('collection', collections[0]) if search_results else collections[0]

        # Collect unique pages from citations
        pages = []
        seen_pages = set()
        for citation in summary_result.get('citations_used', []):
            page = citation.get('page')
            if page and page != 'N/A' and page not in seen_pages:
                pages.append(page)
                seen_pages.add(page)

        # Sort pages and create page range string
        if pages:
            try:
                pages.sort(key=int)  # Try numeric sort
            except:
                pages.sort()  # Fall back to string sort

            if len(pages) == 1:
                page_range = f"Page {pages[0]}"
            elif len(pages) == 2:
                page_range = f"Pages {pages[0]}, {pages[1]}"
            else:
                page_range = f"Pages {pages[0]}-{pages[-1]} ({len(pages)} pages)"
        else:
            page_range = "Pages not available"

        # Single document reference
        references = [{
            'filename': filename,
            'collection': collection,
            'page_range': page_range,
            'pages': pages,
            'citation_id': 'doc_1',
            'confidence_score': summary_result['confidence']
        }]

        return {
            'query': query,
            'search_type': 'Enhanced Document Summarization',
            'answer': summary_result['summary'],
            'references': references,  # SINGLE DOCUMENT REFERENCE
            'confidence': summary_result['confidence'],
            'collections_searched': collections,
            'summary_stats': {
                'target_words': summary_result['target_words'],
                'actual_words': summary_result['word_count'],
                'word_accuracy': summary_result.get('word_accuracy', 0),
                'sources_used': summary_result.get('sources_count', 0),
                'unique_pages': len(pages),
                'query_focus': summary_result.get('query_focus', 'general'),
                'requested_file': filename
            }
        }

    def _handle_cross_reference_comparison(self, query: str, collections: List[str], analysis: Dict) -> Dict[str, Any]:
        """Handle cross-reference comparison queries"""

        target_doc = analysis.get('target_document')
        reference_collection = analysis.get('reference_collection')

        # If target document or reference collection not detected, return helpful error
        if not target_doc or not reference_collection:
            return {
                'query': query,
                'search_type': 'Cross-Reference Comparison',
                'answer': '''To perform a cross-reference comparison, please specify:
1. The target document name (e.g., "Q1 2024 earnings transcript")
2. The reference collection (e.g., "PRA rules", "regulations database")

Example queries:
- "Compare the Q1 earnings transcript against the PRA rules"
- "Check compliance of the 2024 policy document with regulatory standards"
- "Identify discrepancies between contract_v2.pdf and our corporate guidelines"''',
                'references': [],
                'confidence': 0.0,
                'collections_searched': collections,
                'error': 'Could not identify target document or reference collection'
            }

        # Perform the comparison
        comparison_report = self.cross_reference_comparator.compare_documents(
            target_doc=target_doc,
            reference_collection=reference_collection,
            target_collection=collections[0] if collections else None
        )

        # Check for errors in comparison
        if comparison_report.get('status') == 'error':
            return {
                'query': query,
                'search_type': 'Cross-Reference Comparison',
                'answer': f"Error: {comparison_report.get('message', 'Unknown error')}",
                'references': [],
                'confidence': 0.0,
                'collections_searched': collections
            }

        # Format the report into a readable answer
        answer = self._format_comparison_answer(comparison_report)

        # Extract references from the comparison report
        references = []
        for result in comparison_report.get('results', []):
            if 'reference_citations' in result:
                for ref_cite in result['reference_citations']:
                    references.append({
                        'filename': ref_cite.get('filename', 'Unknown'),
                        'page_num': ref_cite.get('page', 'N/A'),
                        'collection': ref_cite.get('collection', reference_collection),
                        'text': ref_cite.get('excerpt', '')
                    })

        return {
            'query': query,
            'search_type': 'Cross-Reference Comparison',
            'answer': answer,
            'references': references,
            'confidence': comparison_report.get('analysis_summary', {}).get('confidence_score', 0.0),
            'collections_searched': collections,
            'comparison_report': comparison_report,
            'target_document': target_doc,
            'reference_collection': reference_collection
        }

    def _format_comparison_answer(self, report: Dict) -> str:
        """Format comparison report into readable answer with comprehensive details"""

        lines = []

        # Title
        lines.append(f"{'='*80}")
        lines.append(f"{report.get('title', 'Cross-Reference Comparison Report')}")
        lines.append(f"{'='*80}\n")

        # Overview
        lines.append(f"**DOCUMENT OVERVIEW:**")
        lines.append(f"{report.get('overview', 'No overview available')}\n")

        # Analysis Summary
        summary = report.get('analysis_summary', {})
        lines.append(f"**ANALYSIS SUMMARY:**")
        lines.append(f"• Sections Analyzed: {summary.get('total_sections_analyzed', 0)}")
        lines.append(f"• Misalignments Found: {summary.get('misalignments_found', 0)}")
        lines.append(f"• Confidence Score: {summary.get('confidence_score', 0.0):.2f} ({summary.get('confidence_level', 'Unknown')})\n")

        # Results
        lines.append(f"**RESULTS:**\n")

        results = report.get('results', [])
        if not results:
            lines.append("No results available.")
        elif len(results) == 1 and results[0].get('message'):
            # Case: No misalignments found
            lines.append(f"{results[0].get('message', '')}")
            lines.append(f"{results[0].get('details', '')}")
        else:
            # Case: Misalignments found - Enhanced with more detail
            for i, result in enumerate(results, 1):
                lines.append(f"\n{'─'*80}")
                lines.append(f"{result.get('section', f'Section {i}')} | Severity: {result.get('severity', 'UNKNOWN')}")
                lines.append(f"{'─'*80}")

                # Target location
                target_loc = result.get('target_location', {})
                lines.append(f"\n📄 TARGET DOCUMENT: {target_loc.get('filename', 'Unknown')}")
                lines.append(f"📍 LOCATION: Page {target_loc.get('page', 'N/A')}, Chunk {target_loc.get('chunk_id', 'N/A')}\n")

                # Target statement - Extended display (no truncation unless > 1000 chars)
                target_stmt = result.get('target_statement', 'N/A')
                lines.append(f"📋 TARGET STATEMENT:")
                lines.append(f"   {'-'*76}")
                if len(target_stmt) > 1000:
                    lines.append(f"   {target_stmt[:1000]}...")
                    lines.append(f"   [Statement truncated - {len(target_stmt)} characters total]")
                else:
                    lines.append(f"   {target_stmt}")
                lines.append(f"   {'-'*76}\n")

                # Explanation - Full display, no truncation
                explanation = result.get('explanation', 'No explanation provided')
                lines.append(f"⚠️  ANALYSIS & EXPLANATION:")
                lines.append(f"   {'-'*76}")
                # Split explanation into paragraphs for better readability
                explanation_paras = explanation.split('\n')
                for para in explanation_paras:
                    if para.strip():
                        # Wrap long lines at word boundaries
                        wrapped = self._wrap_text(para.strip(), 76)
                        for line in wrapped:
                            lines.append(f"   {line}")
                lines.append(f"   {'-'*76}\n")

                # Reference citations - Extended with more detail
                ref_cites = result.get('reference_citations', [])
                if ref_cites:
                    lines.append(f"📚 RELEVANT REGULATIONS & REFERENCES:")
                    for j, ref in enumerate(ref_cites, 1):
                        lines.append(f"\n   [{j}] {ref.get('filename', 'Unknown')}, Page {ref.get('page', 'N/A')}")
                        lines.append(f"       Collection: {ref.get('collection', 'Unknown')}")
                        lines.append(f"       {'-'*70}")
                        # Display longer excerpts (up to 500 chars)
                        excerpt = ref.get('excerpt', 'No excerpt')
                        if len(excerpt) > 500:
                            lines.append(f"       {excerpt[:500]}...")
                        else:
                            lines.append(f"       {excerpt}")
                        lines.append(f"       {'-'*70}")

                lines.append("")  # Blank line between sections

        # Confidence note
        lines.append(f"\n{'='*80}")
        lines.append(f"**CONFIDENCE & RELIABILITY NOTE:**")
        lines.append(f"This analysis achieved a confidence score of {summary.get('confidence_score', 0.0):.2f}")
        lines.append(f"({summary.get('confidence_level', 'Unknown')} confidence).")
        lines.append(f"\nAll findings are based strictly on the content available in the database.")
        lines.append(f"The analysis compares actual text from the target document against specific")
        lines.append(f"regulations retrieved from the reference collection. Users should review the")
        lines.append(f"cited page numbers and excerpts to verify the identified concerns.")
        lines.append(f"{'='*80}")

        return "\n".join(lines)

    def _wrap_text(self, text: str, width: int) -> list:
        """Wrap text at word boundaries for better readability"""
        words = text.split()
        lines = []
        current_line = []
        current_length = 0

        for word in words:
            word_length = len(word) + 1  # +1 for space
            if current_length + word_length > width and current_line:
                lines.append(' '.join(current_line))
                current_line = [word]
                current_length = word_length
            else:
                current_line.append(word)
                current_length += word_length

        if current_line:
            lines.append(' '.join(current_line))

        return lines

    def _generate_enhanced_response(self, query: str, results: List[Dict], citations: List[Dict]) -> str:
        """Generate enhanced contextual response with proper citations"""

        # Build context with citations
        context_parts = []
        for i, result in enumerate(results[:5]):
            # Validate result has required 'text' field
            if 'text' not in result or not result.get('text'):
                continue

            citation = citations[i] if i < len(citations) else {'id': f'cite_{i+1}'}
            filename = citation.get('filename', 'Unknown')
            context_parts.append(f"[{citation['id']} - {filename}]: {result['text'][:600]}")

        context = "\n\n".join(context_parts)

        # Enhanced prompt with strict citation requirements
        prompt = f"""You are a precise analyst. Answer using ONLY the provided context with proper citations.

CONTEXT WITH CITATIONS:
{context[:4000]}

QUESTION: {query}

MANDATORY REQUIREMENTS:
1. Use ONLY information from context - NO external knowledge
2. Cite EVERY fact with format: (cite_X)
3. Write in clear, professional paragraphs
4. If multiple sources address the same point, cite all: (cite_1, cite_2)
5. If information insufficient, state: "Limited information in provided sources"
6. Be comprehensive but stay focused on the question
7. Structure: Introduction → Key Points (cited) → Conclusion

EXAMPLE:
"The company reported strong revenue growth (cite_1). Operating margins improved to 15% (cite_2), while cash flow remained stable (cite_3)."

YOUR CITED ANSWER:"""

        return self.llm_engine.generate_response(prompt, temperature=0.0)

    def _create_error_response(self, query: str, error: str, collections: List[str]) -> Dict[str, Any]:
        """Create standardized error response"""
        return {
            'query': query,
            'search_type': 'Error',
            'answer': f'Error processing query: {error}',
            'references': [],
            'confidence': 0.0,
            'collections_searched': collections,
            'enhanced_mode': self.enhanced_mode,
            'error_details': error,
            'processing_timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
        }

# Initialize Enhanced RAG System
try:
    # Ensure all required components are available
    required_components = ['collections_manager', 'llm_engine', 'embedding_model']
    missing_components = [comp for comp in required_components if comp not in globals()]

    if missing_components:
        print(f"❌ Missing required components: {missing_components}")
        print("Please ensure all previous sections have been run successfully.")
    else:
        # Initialize the enhanced RAG system
        spec_rag = SpecCompliantRAGSystem(
            collections_manager=collections_manager,
            llm_engine=llm_engine,
            embedding_model=embedding_model,
            cross_encoder=cross_encoder if 'cross_encoder' in globals() else None
        )

        print("✅ ENHANCED SpecCompliantRAGSystem initialized successfully!")
        print("🚀 Features Available:")
        print("   • Enhanced citation system with confidence scoring")
        print("   • Cross-document analysis and pattern recognition")
        print("   • Advanced metadata search and database queries")
        print("   • Precise summarization with word count control")
        print("   • Response reliability validation and grounding")
        print("   • Multi-collection hybrid search with RRF fusion")
        print("   • Comprehensive error handling and logging")
        print("   • Integrated query expansion and optimization")
        print("   • Enhanced confidence scoring throughout pipeline")
        print("   • Complete backward compatibility maintained")

        print(f"\n📊 System Status:")
        print(f"   • Enhanced mode: {spec_rag.enhanced_mode}")
        print(f"   • Collections available: {len(collections_manager.get_available_collections())}")
        print(f"   • Hybrid search: {'✅ Available' if spec_rag.hybrid_search else '❌ Not available'}")
        print(f"   • Cross-encoder: {'✅ Available' if spec_rag.cross_encoder else '❌ Not available'}")
        print(f"   • LLM engine: {'✅ Available' if llm_engine.available else '❌ Not available'}")

except Exception as e:
    print(f"❌ Enhanced RAG System initialization failed: {e}")
    print("Please check that all required components from previous sections are available.")

✅ ENHANCED SpecCompliantRAGSystem initialized successfully!
🚀 Features Available:
   • Enhanced citation system with confidence scoring
   • Cross-document analysis and pattern recognition
   • Advanced metadata search and database queries
   • Precise summarization with word count control
   • Response reliability validation and grounding
   • Multi-collection hybrid search with RRF fusion
   • Comprehensive error handling and logging
   • Integrated query expansion and optimization
   • Enhanced confidence scoring throughout pipeline
   • Complete backward compatibility maintained

📊 System Status:
   • Enhanced mode: True
   • Collections available: 2
   • Hybrid search: ✅ Available
   • Cross-encoder: ✅ Available
   • LLM engine: ✅ Available


In [61]:
# PATCH: Fix title_filter sanitization for EnhancedMetadataSearch
import types

# Store original method
original_list_files_query = spec_rag.metadata_search._list_files_query
original_parse = spec_rag.metadata_search._parse_query_intent_with_llm

def patched_parse_query_intent(self, query: str) -> Dict[str, Any]:
    """Patched version that sanitizes title_filter"""
    # Call original parse
    result = original_parse(query)

    # Sanitize title_filter: ignore if it's a collection name or generic term
    if result.get('title_filter'):
        ignore_terms = ['collection', 'database', 'files', 'documents', 'earnings collection',
                       'pra rules', 'pra_rules', 'earnings_transcripts']
        title_lower = result['title_filter'].lower()
        if any(term in title_lower for term in ignore_terms):
            result['title_filter'] = None

    return result

# Apply patch to the parse method
spec_rag.metadata_search._parse_query_intent_with_llm = types.MethodType(patched_parse_query_intent, spec_rag.metadata_search)

print("✅ Applied title_filter sanitization patch!")

✅ Applied title_filter sanitization patch!


#### 7.4 Enhanced Multi-Collection Selector UI Component

In [62]:
# Enhanced Multi-Collection Selector UI Component
class AdvancedCollectionSelector:
    """Advanced collection selector with file counts and metadata"""

    def __init__(self, collections_manager):
        self.manager = collections_manager
        self.selected_collections = []

    def create_selector_widget(self):
        """Create advanced collection selector with real-time stats"""

        available_collections = self.manager.get_available_collections()

        # Create checkboxes with collection info
        collection_checkboxes = {}
        collection_info = {}

        for collection_name in available_collections:
            try:
                collection = self.manager.get_collection(collection_name)
                count = collection.count()

                # Get sample filenames
                data = collection.get(include=['metadatas'], limit=5)
                filenames = set()
                for metadata in data.get('metadatas', []):
                    if metadata and metadata.get('filename'):
                        filenames.add(metadata['filename'])

                info_text = f"{collection_name} ({count:,} chunks, {len(filenames)} files)"
                collection_info[collection_name] = {
                    'count': count,
                    'files': len(filenames),
                    'sample_files': list(filenames)[:3]
                }

            except Exception as e:
                info_text = f"{collection_name} (error: {str(e)[:50]})"
                collection_info[collection_name] = {'error': str(e)}

            checkbox = widgets.Checkbox(
                value=False,
                description=info_text,
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='100%')
            )
            collection_checkboxes[collection_name] = checkbox

        # Create selection summary
        selection_summary = widgets.HTML(value="<p><em>No collections selected</em></p>")

        def update_selection(*args):
            selected = [name for name, cb in collection_checkboxes.items() if cb.value]
            self.selected_collections = selected

            if selected:
                total_chunks = sum(collection_info[name].get('count', 0) for name in selected)
                total_files = sum(collection_info[name].get('files', 0) for name in selected)

                summary_html = f"""
                <div style='background: #f0f9ff; padding: 10px; border-radius: 5px; border: 1px solid #0ea5e9;'>
                    <strong>Selected Collections ({len(selected)}):</strong><br/>
                    • Total chunks: {total_chunks:,}<br/>
                    • Total files: {total_files}<br/>
                    • Collections: {', '.join(selected)}
                </div>
                """
            else:
                summary_html = "<p><em>No collections selected</em></p>"

            selection_summary.value = summary_html

        # Bind events
        for cb in collection_checkboxes.values():
            cb.observe(update_selection, names='value')

        # Create UI
        selector_ui = widgets.VBox([
            widgets.HTML("<h4>📂 Select Collections for RAG Queries:</h4>"),
            widgets.VBox(list(collection_checkboxes.values())),
            selection_summary
        ])

        return selector_ui, collection_checkboxes

# 2. Real-time Progress and Status System
class RAGProgressTracker:
    """Real-time progress tracking for RAG operations"""

    def __init__(self):
        self.progress_bar = widgets.FloatProgress(
            value=0, min=0, max=100,
            description='Progress:',
            bar_style='info',
            layout=widgets.Layout(width='100%')
        )

        self.status_label = widgets.HTML(value="Ready")

        self.cancel_flag = threading.Event()
        self.cancel_button = widgets.Button(
            description='Cancel',
            button_style='danger',
            disabled=True,
            layout=widgets.Layout(width='100px')
        )

        self.cancel_button.on_click(self._cancel_operation)

    def _cancel_operation(self, button):
        """Cancel current operation"""
        self.cancel_flag.set()
        self.update_status("Cancelling...", 0)
        button.disabled = True

    def start_operation(self, operation_name: str):
        """Start tracking an operation"""
        self.cancel_flag.clear()
        self.cancel_button.disabled = False
        self.update_status(f"Starting {operation_name}...", 10)

    def update_progress(self, progress: float, message: str = ""):
        """Update progress (0-100)"""
        self.progress_bar.value = progress
        if message:
            self.status_label.value = f"<span style='color: #0066cc;'>{message}</span>"

    def update_status(self, message: str, progress: float = None):
        """Update status message"""
        self.status_label.value = f"<span style='color: #0066cc;'>{message}</span>"
        if progress is not None:
            self.progress_bar.value = progress

    def complete_operation(self, message: str = "Operation completed"):
        """Mark operation as complete"""
        self.progress_bar.value = 100
        self.cancel_button.disabled = True
        self.status_label.value = f"<span style='color: #008800;'>✅ {message}</span>"

    def error_operation(self, message: str = "Operation failed"):
        """Mark operation as failed"""
        self.cancel_button.disabled = True
        self.status_label.value = f"<span style='color: #cc0000;'>❌ {message}</span>"

    def get_widgets(self):
        """Get progress widgets for display"""
        return widgets.VBox([
            self.progress_bar,
            widgets.HBox([self.status_label, self.cancel_button])
        ])

# 3. Enhanced Results Display System
class EnhancedResultsDisplay:
    """Enhanced results display with expandable details and scoring"""

    def __init__(self):
        # Create output widget with styling
        self.results_area = widgets.Output(
            layout=widgets.Layout(
                width='100%',
                border='1px solid #e2e8f0',
                padding='15px',
                border_radius='8px',
                background_color='#f8fafc',
                overflow='auto',
                max_height='600px'
            )
        )

        # Initialize with a placeholder message
        with self.results_area:
            print("💡 Ready to search! Enter a query and click Search to see results here.")

    def display_results(self, result: Dict[str, Any]):
        """Display results with enhanced formatting"""

        with self.results_area:
            clear_output(wait=False)

            # Display main answer
            print("🎯 ANSWER:")
            print("=" * 50)
            print(result.get('answer', 'No answer generated'))
            print()

            # Display confidence and reliability
            confidence = result.get('confidence', 0)
            reliability = result.get('reliability_validation', {})

            print(f"📊 CONFIDENCE: {confidence:.1%}")
            if reliability:
                grounding_score = reliability.get('grounding_score', 0)
                print(f"🔗 GROUNDING: {grounding_score:.1%}")
                if not reliability.get('is_grounded', True):
                    print("⚠️  WARNING: Response may not be fully grounded in sources")
            print()

            # Display search metadata
            search_type = result.get('search_type', 'Unknown')
            collections_searched = result.get('collections_searched', [])
            print(f"🔍 SEARCH TYPE: {search_type}")
            print(f"📂 COLLECTIONS: {', '.join(collections_searched)}")

            # Display processing stats if available
            if 'processing_stats' in result:
                stats = result['processing_stats']
                print(f"📈 RESULTS: {stats.get('total_processed', 0)} processed from {stats.get('total_found', 0)} found")
                print(f"🎯 CONFIDENCE DIST: High: {stats.get('confidence_distribution', {}).get('high', 0)}, "
                      f"Medium: {stats.get('confidence_distribution', {}).get('medium', 0)}, "
                      f"Low: {stats.get('confidence_distribution', {}).get('low', 0)}")

            # Display cross-document analysis
            if 'cross_document_analysis' in result:
                cross_analysis = result['cross_document_analysis']
                print(f"📚 DOCUMENTS ANALYZED: {cross_analysis.get('total_documents', 0)}")
                print(f"🔗 CONSISTENCY: {cross_analysis.get('cross_document_consistency', 'Unknown')}")

            print()

            # Display references with PROMINENT PAGE NUMBERS
            references = result.get('references', [])
            if references:
                print(f"📎 SOURCES ({len(references)}):")
                print("=" * 30)

                for i, ref in enumerate(references[:10], 1):
                    filename = ref.get('filename', 'Unknown')
                    collection = ref.get('collection', 'Unknown')

                    # Handle both formats: single page or page range
                    page = ref.get('page', 'N/A')
                    page_range = ref.get('page_range', '')
                    citation_id = ref.get('citation_id', f'cite_{i}')

                    # Build page reference - PROMINENT
                    if page_range:
                        page_ref = f" 📄 {page_range}"
                    elif page != 'N/A':
                        page_ref = f" 📄 Page {page}"
                    else:
                        page_ref = ""

                    # Display scores if available
                    scores_info = []
                    if 'confidence_score' in ref:
                        scores_info.append(f"Confidence: {ref['confidence_score']:.1%}")
                    if 'scores' in ref:
                        scores = ref['scores']
                        if scores.get('final_score'):
                            scores_info.append(f"Score: {scores['final_score']:.3f}")
                    elif ref.get('relevance_score'):
                        scores_info.append(f"Relevance: {ref['relevance_score']:.3f}")

                    scores_text = f" ({', '.join(scores_info)})" if scores_info else ""

                    # Print with citation ID and page number PROMINENT
                    print(f"{i}. [{citation_id}]{page_ref} - {filename}")
                    if collection != 'Unknown':
                        print(f"   Collection: {collection}{scores_text}")

                    # Show snippet
                    snippet = ref.get('text_snippet', '')
                    if snippet:
                        print(f"   └─ {snippet}")
                    print()

            # Display summary stats if available
            if 'summary_stats' in result:
                stats = result['summary_stats']
                print(f"📝 SUMMARY STATS:")
                print(f"   Words: {stats.get('actual_words', 0)}/{stats.get('target_words', 0)} "
                      f"(Accuracy: {stats.get('word_accuracy', 0):.1%})")
                print(f"   Sources: {stats.get('sources_used', 0)}")
                if stats.get('unique_pages'):
                    print(f"   Unique Pages: {stats.get('unique_pages', 0)}")
                if stats.get('query_focus'):
                    print(f"   Query Focus: {stats.get('query_focus', 'N/A')}")
                if stats.get('file_specific'):
                    print(f"   File: {stats.get('requested_file', 'N/A')}")

    def get_widget(self):
        """Get results display widget"""
        return self.results_area

print("✅ Additional RAG UI Components defined!")

✅ Additional RAG UI Components defined!


## 8. Launch the complete RAG system interface

#### 8.1 Complete Functional RAG System UI

In [63]:
# Complete Functional RAG System UI
class ComprehensiveRAGInterface:
    """Complete RAG system interface with all capabilities"""

    def __init__(self, spec_rag_system):
        self.rag_system = spec_rag_system
        self.setup_ui_components()
        self.query_thread = None
        self.animation_active = False

    def setup_ui_components(self):
        """Initialize all UI components"""

        # 1. Compact Collection Selector
        collections = self.rag_system.collections_manager.get_available_collections()
        collection_checkboxes = []
        self.collection_map = {}

        for col_name in collections:
            cb = widgets.Checkbox(
                value=True,
                description=col_name,
                layout=widgets.Layout(width='auto'),
                style={'description_width': 'initial'}
            )
            collection_checkboxes.append(cb)
            self.collection_map[cb] = col_name

        self.compact_selector = widgets.HBox(
            collection_checkboxes,
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        # 2. Query Input
        self.query_input = widgets.Textarea(
            value='',
            placeholder='Enter your query here ...',
            layout=widgets.Layout(width='95%', height='60px', margin='10px auto')
        )

        # 3. Answer Output
        self.answer_output = widgets.HTML(
            value=(
                "<div style='max-height:800px; overflow:auto; font-family: monospace; white-space: pre-wrap; "
                "color: #334155; padding: 10px; border: 1px solid #e2e8f0; border-radius: 6px;'>"
                "Full answer (with references and stats) will appear here. This box supports scrolling/overflow."
                "</div>"
            ),
            layout=widgets.Layout(width='95%', margin='6px auto')
        )

        # 4. Search Controls
        self.search_button = widgets.Button(
            description='⏳ Search',
            button_style='primary',
            layout=widgets.Layout(width='100px', height='30px')
        )

        self.progress_bar = widgets.FloatProgress(
            value=0,
            min=0,
            max=100,
            layout=widgets.Layout(width='400px', height='25px')
        )

        self.status_label = widgets.HTML(value="Ready")

        self.cancel_button = widgets.Button(
            description='Cancel',
            button_style='danger',
            layout=widgets.Layout(width='80px', height='30px'),
            disabled=True
        )

        # Progress tracker
        self.progress_tracker = RAGProgressTracker()

        # 5. Advanced Options
        self.advanced_options = widgets.Accordion([
            widgets.VBox([
                widgets.Checkbox(
                    value=True,
                    description='Use Cross-Encoder Reranking',
                    style={'description_width': 'initial'}
                ),
                widgets.Checkbox(
                    value=True,
                    description='Use Query Expansion',
                    style={'description_width': 'initial'}
                )
            ])
        ])
        self.advanced_options.set_title(0, '⚙️ Advanced Options')
        self.advanced_options.selected_index = None

        # 6. Instructions
        self.instructions = widgets.HTML(value="""
        <div style='background-color: #f8fafc; padding: 10px; border-radius: 6px; font-size: 12px; margin-top: 10px;'>
            <b>💡 Tips:</b><br/>
            • <b>Summarize:</b> "Provide summary of 300 words for barclays_1Q20_analyst_meeting_transcript.pdf"<br/>
            • <b>Database:</b> "List all files in each collection in the database for year 2022"<br/>
            • <b>Cross-Reference:</b> "Compare the barclays_1Q20_analyst_meeting_transcript.pdf against the PRA rules for compliance issues"
        </div>
        """)

        # Setup event handlers
        self.search_button.on_click(self._on_search_clicked)
        self.cancel_button.on_click(self._on_cancel_clicked)

    def _on_cancel_clicked(self, button):
        """Handle cancel button click"""
        self.progress_tracker.cancel_flag.set()
        self.animation_active = False
        self.cancel_button.disabled = True
        self.search_button.disabled = False
        self.search_button.description = '⏳ Search'
        self.status_label.value = "<span style='color: #cc6600;'>⚠️ Search cancelled</span>"

    def _on_search_clicked(self, button):
        """Handle search button click"""
        query = self.query_input.value.strip()
        if not query:
            self.status_label.value = "<span style='color: #cc0000;'>⚠️ Please enter a query</span>"
            return

        # Get selected collections
        collections = []
        for cb, col_name in self.collection_map.items():
            if cb.value:
                collections.append(col_name)

        if not collections:
            self.status_label.value = "<span style='color: #cc0000;'>⚠️ Please select at least one collection</span>"
            return

        # Clear output
        from html import escape as _escape
        msg = 'Running search... (full answer will appear here when ready)\n'
        self.answer_output.value = "<div style='white-space: pre-wrap; font-family: monospace;'>{}</div>".format(_escape(msg))

        # Update UI state
        self.search_button.disabled = True
        self.animation_active = True
        self.cancel_button.disabled = False
        self.progress_bar.value = 0
        self.status_label.value = "<span style='color: #0066cc;'>Starting search...</span>"
        self.progress_tracker.cancel_flag.clear()

        # Start animation and search
        import threading
        animation_thread = threading.Thread(target=self._animate_button)
        animation_thread.daemon = True
        animation_thread.start()

        search_thread = threading.Thread(target=self._execute_search, args=(query, collections))
        search_thread.daemon = True
        search_thread.start()

    def _animate_button(self):
        """Animate the search button with spinning hourglass"""
        import time
        spinners = ['⏳', '⌛']
        idx = 0
        while self.animation_active:
            self.search_button.description = f'{spinners[idx]} Searching...'
            idx = (idx + 1) % len(spinners)
            time.sleep(0.5)

    def _execute_search(self, query, collections):
        """Execute search in background thread"""
        try:
            # Get advanced options settings
            use_reranking = self.advanced_options.children[0].children[0].value  # Cross-encoder checkbox
            use_query_expansion = self.advanced_options.children[0].children[1].value  # Query expansion checkbox

            self.progress_bar.value = 20
            self.status_label.value = "<span style='color: #0066cc;'>Initializing search...</span>"

            if self.progress_tracker.cancel_flag.is_set():
                return

            self.progress_bar.value = 40
            self.status_label.value = "<span style='color: #0066cc;'>Searching collections...</span>"

            # Pass reranking parameter to process_query
            result = self.rag_system.process_query(
                query,
                collections,
                use_reranking=use_reranking  # ✅ Now properly passed
            )

            if self.progress_tracker.cancel_flag.is_set():
                return

            self.progress_bar.value = 80
            self.status_label.value = "<span style='color: #0066cc;'>Processing results...</span>"

            # Format output
            import html as _html
            content_lines = []
            content_lines.append("🎯 ANSWER")
            content_lines.append("="*80)
            content_lines.append(result.get('answer', 'No answer generated'))
            content_lines.append("")

            confidence = result.get('confidence', 0)
            reliability = result.get('reliability_validation', {})

            content_lines.append(f"📊 CONFIDENCE: {confidence:.1%}")
            if reliability:
                grounding_score = reliability.get('grounding_score', 0)
                content_lines.append(f"🔗 GROUNDING: {grounding_score:.1%}")
                if not reliability.get('is_grounded', True):
                    content_lines.append("⚠️  WARNING: Response may not be fully grounded in sources")
            content_lines.append("")

            search_type = result.get('search_type', 'Unknown')
            collections_searched = result.get('collections_searched', [])
            content_lines.append(f"🔍 SEARCH TYPE: {search_type}")
            content_lines.append(f"📂 COLLECTIONS: {', '.join(collections_searched)}")
            content_lines.append("")

            references = result.get('references', [])
            if references:
                content_lines.append(f"📎 SOURCES ({len(references)}):")
                content_lines.append("="*50)
                for i, ref in enumerate(references[:15], 1):
                    filename = ref.get('filename', 'Unknown')
                    collection = ref.get('collection', 'Unknown')

                    page = ref.get('page', 'N/A')
                    page_range = ref.get('page_range', '')
                    citation_id = ref.get('citation_id', f'cite_{i}')

                    if page_range:
                        page_ref = f" 📄 {page_range}"
                    elif page != 'N/A':
                        page_ref = f" 📄 Page {page}"
                    else:
                        page_ref = ""

                    content_lines.append(f"{i}. [{citation_id}]{page_ref} - {filename}")
                    if collection != 'Unknown':
                        content_lines.append(f"   Collection: {collection}")

                    snippet = ref.get('text_snippet', '')
                    if snippet:
                        content_lines.append(f"   └─ {snippet}")
                    content_lines.append("")

            content_text = "\n".join(content_lines)
            escaped = _html.escape(content_text)
            self.answer_output.value = f"<div style='max-height:800px; overflow:auto; font-family: monospace; white-space: pre-wrap; color: #334155; padding: 10px; border: 1px solid #e2e8f0; border-radius: 6px;'>{escaped}</div>"

            self.progress_bar.value = 100
            self.status_label.value = "<span style='color: #008800;'>✅ Search completed successfully</span>"
            self.cancel_button.disabled = True

        except Exception as e:
            error_msg = f"Search failed: {str(e)}"
            self.progress_bar.value = 0
            self.status_label.value = f"<span style='color: #cc0000;'>❌ {error_msg}</span>"
            self.cancel_button.disabled = True

            from html import escape as _escape
            self.answer_output.value = f"<div style='white-space: pre-wrap; font-family: monospace; color:#cc0000;'>{_escape(error_msg)}</div>"

        finally:
            self.animation_active = False
            self.search_button.disabled = False
            self.search_button.description = '⏳ Search'

    def display(self):
        header = widgets.HTML("""
        <div style='background: linear-gradient(135deg, #002147 0%, #003d82 100%); color: white; padding: 10px; border-radius: 8px; margin-bottom: 20px; text-align: center;'>
            <h2 style='margin: 0; font-size: 24px;'>RAG System Interface</h2>
        </div>
        """)

        main_content = widgets.VBox([
            self.compact_selector,
            widgets.HTML("<div style='height: 10px;'></div>"),

            # Search controls ABOVE query input
            widgets.HBox([
                self.search_button,
                widgets.HTML("<div style='width: 20px;'></div>"),
                self.progress_bar,
                widgets.HTML("<div style='width: 10px;'></div>"),
                self.cancel_button,
            ], layout=widgets.Layout(justify_content='center', margin='10px 0')),

            widgets.HBox([self.status_label], layout=widgets.Layout(justify_content='center')),
            widgets.HTML("<div style='height: 10px;'></div>"),

            # Query input
            self.query_input,

            # Answer output
            self.answer_output,
            widgets.HTML("<div style='height: 10px;'></div>"),

            self.advanced_options,
            self.instructions
        ], layout=widgets.Layout(align_items='center'))

        complete_interface = widgets.VBox([
            header,
            main_content
        ], layout=widgets.Layout(
            width='95%',
            max_width='1000px',
            margin='0 auto',
            padding='20px',
            background_color='white',
            border_radius='12px',
            box_shadow='0 2px 8px rgba(0,0,0,0.1)'
        ))

        from IPython.display import display
        display(complete_interface)

print("✅ Complete RAG System UI initialized!")


✅ Complete RAG System UI initialized!


#### 8.2 Launch the complete RAG system interface

In [64]:
# Initialize and Launch Complete RAG Interface
if 'spec_rag' not in globals():
    print("spec_rag system not available. Please run Section 7 first.")
else:
    # Initialize the comprehensive interface
    comprehensive_rag_interface = ComprehensiveRAGInterface(spec_rag)

    # Display the interface ONLY (no print statements after)
    display(comprehensive_rag_interface.display())

None